<a href="https://colab.research.google.com/github/tnc-br/ddf_common/blob/xgb2/dataset_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title Imports and modules.
%pip install opencv-python
%pip install matplotlib
%pip install pandas

from osgeo import gdal, gdal_array
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import dataclass
import matplotlib.animation as animation
from matplotlib import rc
from typing import List
from numpy.random import MT19937, RandomState, SeedSequence
import pandas as pd
from tqdm import tqdm
from io import StringIO
import xgboost as xgb
import os
import math
import glob

rc('animation', html='jshtml')

import sys
!if [ ! -d "/content/ddf_common_stub" ] ; then git clone -b test https://github.com/tnc-br/ddf_common_stub.git; fi
sys.path.append("/content/ddf_common_stub/")
import ddfimport
ddfimport.ddf_source_control_pane()
# ddfimport.ddf_import_common()

executing checkout_branch xgb2...
Mounted at /content/gdrive
b"fatal: destination path 'ddf_common' already exists and is not an empty directory.\nRepository already exists.\n"
b'From https://github.com/tnc-br/ddf_common\n   b5ea7d9..ac5bea5  partition_data        -> origin/partition_data\n   357b178..d699202  partition_data_random -> origin/partition_data_random\n   2a83735..fcc4974  ttest-greta           -> origin/ttest-greta\nAlready up to date.\n'
b''
xgb2 branch checked out at "/content/gdrive/MyDrive/xgb2/ddf_common". You may now use ddf_common imports and change common files.


interactive(children=(Text(value='required', description='Commit Msg', placeholder='Enter commit message'), Bu…

In [4]:
import dataset
import raster
import importlib

importlib.reload(dataset)
importlib.reload(raster)

<module 'raster' from '/content/gdrive/MyDrive/xgb2/ddf_common/raster.py'>

# preprocess_sample_data Tests

In [3]:
# @title Load example CSV
jun23_reference_data = """ID,Cod Lab,Code,Species,Scientific_name,Genus,Family,Point,Origin,State,date sample havest,lat,long,d15N_wood,%N_wood,d13C_wood,%C_wood,d13C_cel,%C_cel,d18O_cel_CENA,d18O_cel_Davis,d18O_cel_analysis,d18O_cel,VPD,RH,PET,DEM,PA,Mean Annual Temperature,Mean Annual Precipitation,Iso_Oxi_Stack_mean_TERZER,predkrig_br_lat_ISORG,brisoscape_mean_ISORIX,isoscape_fullmodel_d18O_prec_REGRESSION
1,ZAC 730,mad53,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,itacoatiara,amazonas,Jul-22,-2.499,-59.121,3.2,0.1,-32.4,48.8,-30.2,41.9,,24.92,,24.92,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
2,ZAC 731,mad53,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,itacoatiara,amazonas,Jul-22,-2.499,-59.121,2.7,0.1,-27.3,49.3,-25.9,42.8,,23.46,,23.46,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
3,ZAC 732,mad53,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,itacoatiara,amazonas,Jul-22,-2.499,-59.121,2.8,0.1,-26.3,50.2,-24.9,43,,23.81,,23.81,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
4,ZAC 733,mad53,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,itacoatiara,amazonas,Jul-22,-2.499,-59.121,3.7,0.2,-27.5,50.3,-25.2,44.4,,23.11,,23.11,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
5,ZAC 734,mad53,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,itacoatiara,amazonas,Jul-22,-2.499,-59.121,3.7,0.2,-28.3,48.8,-25.8,41.3,,22.94,,22.94,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
6,ZAC 735,mad54,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,itacoatiara,amazonas,Jul-22,-2.496,-59.126,-1,0.1,-30.8,49.5,-28.2,42.3,,25.48,,25.48,0.76667,0.78997,98.28333,103,1000.65411,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
7,ZAC 736,mad54,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,itacoatiara,amazonas,Jul-22,-2.496,-59.126,1.1,0.1,-28.4,49.4,-27.3,43.6,,25.41,,25.41,0.76667,0.78997,98.28333,103,1000.65411,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
8,ZAC 737,mad54,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,itacoatiara,amazonas,Jul-22,-2.496,-59.126,-0.1,0.1,-28.2,50.5,-26.7,43.3,,25.19,,25.19,0.76667,0.78997,98.28333,103,1000.65411,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
9,ZAC 738,mad54,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,itacoatiara,amazonas,Jul-22,-2.496,-59.126,0.2,0.1,-28.3,50.6,-26.9,44.1,,24.77,,24.77,0.76667,0.78997,98.28333,103,1000.65411,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
10,ZAC 739,mad54,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,itacoatiara,amazonas,Jul-22,-2.496,-59.126,1.2,0.1,-28.4,49.8,-26.7,42.8,,26.3,,26.3,0.76667,0.78997,98.28333,103,1000.65411,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
11,ZAC 740,mad55,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,itacoatiara,amazonas,Jul-22,-2.495,-59.12,2.8,0.1,-27.9,49.6,-26.8,43.8,,25.38,,25.38,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
12,ZAC 741,mad55,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,itacoatiara,amazonas,Jul-22,-2.495,-59.12,3.9,0.1,-28,49.5,-26.9,43.4,,25.45,,25.45,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
13,ZAC 742,mad55,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,itacoatiara,amazonas,Jul-22,-2.495,-59.12,3.3,0.1,-28.8,49.4,-28,41.1,,25.78,,25.78,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
14,ZAC 743,mad55,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,itacoatiara,amazonas,Jul-22,-2.495,-59.12,2.8,0.1,-29,50.1,-27.3,42,,26.15,,26.15,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
15,ZAC 744,mad55,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,itacoatiara,amazonas,Jul-22,-2.495,-59.12,2.6,0.1,-28.8,49.9,-27,42,,26.64,,26.64,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
16,ZAC 745,mad62,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.496,-59.125,0.9,0.2,-25.6,51.4,-24.8,41.5,,27.28,,27.28,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
17,ZAC 746,mad62,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.496,-59.125,1.5,0.2,-25.9,51.2,-24.6,40.6,,27.24,,27.24,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
18,ZAC 747,mad62,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.496,-59.125,1.1,0.2,-25.3,53,-24.5,40.3,,27.42,,27.42,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
19,ZAC 748,mad62,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.496,-59.125,0.1,0.3,-26.9,51.8,-25.9,41.1,,27.34,,27.34,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
20,ZAC 749,mad62,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.496,-59.125,2.2,0.3,-27.7,51,-24.8,41.1,,27.47,,27.47,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
21,ZAC 750,mad63,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.495,-59.123,0.1,0.3,-27.4,51.2,-26.7,40.8,,26.01,,26.01,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
22,ZAC 751,mad63,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.495,-59.123,0.8,0.2,-27.1,51.6,-25.8,40.7,,26.46,,26.46,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
23,ZAC 752,mad63,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.495,-59.123,1,0.3,-27.4,52.5,-25.9,40.6,,26.45,,26.45,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
24,ZAC 753,mad63,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.495,-59.123,1.9,0.5,-28.6,51.3,-27.1,40.5,,24.86,,24.86,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
25,ZAC 754,mad63,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.495,-59.123,1.6,0.5,-28.8,50.3,-27.8,38.4,,25.2,,25.2,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
26,ZAC 755,mad64,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.495,-59.122,1.1,0.3,-32.5,52.1,-31.8,44,,24.69,,24.69,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
27,ZAC 756,mad64,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.495,-59.122,1,0.3,-27.6,51.6,-26.4,40.3,,26.65,,26.65,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
28,ZAC 757,mad64,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.495,-59.122,1.2,0.3,-27.4,51.6,-26.3,40.1,,29.19,,29.19,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
29,ZAC 758,mad64,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.495,-59.122,2.7,0.4,-28.9,50.4,-27.1,39.8,,26.16,,26.16,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
30,ZAC 759,mad64,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.495,-59.122,2.4,0.4,-29,48.9,-27.4,39.6,,25.97,,25.97,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
31,ZAC 715,mad65,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.496,-59.125,1.7,0.2,-26.5,48.6,-25.3,41.6,,22.73,,22.73,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
32,ZAC 716,mad65,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.496,-59.125,2,0.2,-26.7,51.1,-25.7,43.8,,22.58,,22.58,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
33,ZAC 717,mad65,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.496,-59.125,2,0.2,-27.7,52,-25.6,43,,22.67,,22.67,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
34,ZAC 718,mad65,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.496,-59.125,1.7,0.3,-28.7,51.6,-25.6,42.1,,23.69,,23.69,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
35,ZAC 719,mad65,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.496,-59.125,1.4,0.3,-28.8,49.6,-26.4,40.5,,25.54,,25.54,0.775,0.78866,98.45,139,996.36792,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
36,ZAC 720,mad66,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.485,-59.125,4.3,0.1,-27.5,49.7,-28.6,41.5,,23.15,,23.15,0.775,0.78866,98.45,120,998.62823,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
37,ZAC 721,mad66,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.485,-59.125,4.8,0.2,-27.7,51,-26.2,40.9,,23.95,,23.95,0.775,0.78866,98.45,120,998.62823,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
38,ZAC 722,mad66,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.485,-59.125,4.6,0.2,-27.5,52,-25.9,42.6,,21.47,,21.47,0.775,0.78866,98.45,120,998.62823,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
39,ZAC 723,mad66,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.485,-59.125,4,0.2,-27.8,51.8,-25.9,41.3,,22.82,,22.82,0.775,0.78866,98.45,120,998.62823,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
40,ZAC 724,mad66,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.485,-59.125,3.2,0.2,-27.8,52.7,-26,44,,22.86,,22.86,0.775,0.78866,98.45,120,998.62823,26.79167,2264,-3.70363,-4.17673,-3.98193,-3.8401
41,ZAC 725,mad67,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.492,-59.123,3.6,0.2,-26.5,49.6,-24.8,41.8,,23.72,,23.72,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
42,ZAC 726,mad67,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.492,-59.123,3.9,0.1,-26,50.9,-24.3,41.6,,23.85,,23.85,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
43,ZAC 727,mad67,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.492,-59.123,2.6,0.2,-26.1,50.1,-24.3,41.8,,24.3,,24.3,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
44,ZAC 728,mad67,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.492,-59.123,2.6,0.3,-27.3,51.9,-25.4,44.7,,23.31,,23.31,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
45,ZAC 729,mad67,angelim_vermelho,Dinizia excelsa Ducke,dinizia,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.492,-59.123,3.7,0.4,-28,49.7,-25.3,41,,24.48,,24.48,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
46,ZAC 805,mad68,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.495,-59.124,-1,1,-28.3,52.7,-25.7,40.7,,24.91,,24.91,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
47,ZAC 806,mad68,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.495,-59.124,-0.6,0.6,-28,51.9,-26.1,40.6,,25.81,,25.81,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
48,ZAC 807,mad68,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.495,-59.124,0.5,0.5,-27.3,51.8,-25.4,40.5,,25.59,,25.59,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
49,ZAC 808,mad68,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.495,-59.124,0.6,0.8,-28.7,49.6,-27,40.4,,25.76,,25.76,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
50,ZAC 809,mad68,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.495,-59.124,1.6,0.8,-29.2,49,-27.6,39.6,,24.78,,24.78,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
51,ZAC 810,mad69,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.496,-59.121,2.2,0.7,-26.9,51.4,-25.4,42.5,,25.6,,25.6,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
52,ZAC 811,mad69,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.496,-59.121,1.9,0.4,-26.9,50.2,-25.7,41.6,,25.17,,25.17,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
53,ZAC 812,mad69,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.496,-59.121,2.4,0.8,-27.8,51.1,-25.2,41.5,,24.9,,24.9,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
54,ZAC 813,mad69,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.496,-59.121,2.8,0.5,-26.9,50.7,-25,41.7,,26.24,,26.24,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
55,ZAC 814,mad69,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.496,-59.121,1.8,0.9,-28.2,51.3,-26.4,42.8,,24.42,,24.42,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
56,ZAC 815,mad70,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.485,-59.124,-0.3,0.6,-29.8,49.9,-28.2,40.6,,24.17,,24.17,0.775,0.78866,98.45,120,998.62823,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
57,ZAC 816,mad70,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.485,-59.124,0.3,1,-29.4,50,-27.9,41,,25.39,,25.39,0.775,0.78866,98.45,120,998.62823,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
58,ZAC 817,mad70,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.485,-59.124,0.1,0.5,-27.1,51.2,-26.2,40.7,,24.72,,24.72,0.775,0.78866,98.45,120,998.62823,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
59,ZAC 818,mad70,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.485,-59.124,0.7,0.7,-26.8,52.5,-25.4,42.3,,24.91,,24.91,0.775,0.78866,98.45,120,998.62823,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
60,ZAC 819,mad70,angelim_pedra,Hymenolobium modestum Ducke,hymenaea,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.485,-59.124,0.6,0.9,-27.3,50.6,-25.8,41.3,,24.72,,24.72,0.775,0.78866,98.45,120,998.62823,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
61,ZAC 670,mad74,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,0,itacoatiara,amazonas,Jul-22,-2.496,-59.121,2.9,0.3,-28.1,51.7,-27.7,42,,27.36,,27.36,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
62,ZAC 671,mad74,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,25,itacoatiara,amazonas,Jul-22,-2.496,-59.121,3.1,0.3,-27.3,52.3,-26.4,41,,28.75,,28.75,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
63,ZAC 672,mad74,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,50,itacoatiara,amazonas,Jul-22,-2.496,-59.121,3.5,0.3,-26.3,50.8,-24.5,41.6,,27.88,,27.88,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
64,ZAC 673,mad74,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,75,itacoatiara,amazonas,Jul-22,-2.496,-59.121,3.1,0.4,-27.3,51.8,-25.9,41.5,,27.65,,27.65,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
65,ZAC 674,mad74,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,100,itacoatiara,amazonas,Jul-22,-2.496,-59.121,3.4,0.5,-27.8,49.4,-26.4,41.6,,26.93,,26.93,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
66,ZAC 675,mad75,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,0,itacoatiara,amazonas,Jul-22,-2.495,-59.122,4.9,0.3,-30.5,51.1,-25.8,40.7,,25.99,,25.99,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
67,ZAC 676,mad75,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,25,itacoatiara,amazonas,Jul-22,-2.495,-59.122,6.5,0.2,-27.1,51.7,-30.6,41,,27.14,,27.14,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
68,ZAC 677,mad75,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,50,itacoatiara,amazonas,Jul-22,-2.495,-59.122,5.5,0.2,-26.2,50.5,-25.5,41,,27.02,,27.02,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
69,ZAC 678,mad75,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,75,itacoatiara,amazonas,Jul-22,-2.495,-59.122,4.7,0.3,-27,49.4,-25.9,39.3,,27.36,,27.36,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
70,ZAC 679,mad75,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,100,itacoatiara,amazonas,Jul-22,-2.495,-59.122,7.9,0.3,-27.5,48.5,-26.1,39.6,,26.59,,26.59,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
71,ZAC 680,mad76,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,0,itacoatiara,amazonas,Jul-22,-2.495,-59.121,3.2,0.3,-27.4,49.8,-26.4,40.9,,26.9,,26.9,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
72,ZAC 681,mad76,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,25,itacoatiara,amazonas,Jul-22,-2.495,-59.121,4.1,0.3,-25.1,50.8,-24,40.2,,27.15,,27.15,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
73,ZAC 682,mad76,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,50,itacoatiara,amazonas,Jul-22,-2.495,-59.121,3.8,0.4,-25.8,50.5,-24.7,42,,25.97,,25.97,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
74,ZAC 683,mad76,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,75,itacoatiara,amazonas,Jul-22,-2.495,-59.121,3.8,0.4,-27.6,49.5,-25.9,42.6,,26.32,,26.32,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
75,ZAC 684,mad76,guariuba,Clarisia racemosa Ruiz & Pav.,clarisia,meliaceae,100,itacoatiara,amazonas,Jul-22,-2.495,-59.121,3.9,0.6,-27.9,48.3,-26.4,40.4,,27.09,,27.09,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
76,ZAC 685,mad56,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,0,itacoatiara,amazonas,Jul-22,-2.497,-59.116,2.3,0.4,-29,45.7,-29.11,42.29,,23.36,,23.36,0.775,0.78866,98.45,121,998.50916,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
77,ZAC 686,mad56,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,25,itacoatiara,amazonas,Jul-22,-2.497,-59.116,2.5,0.5,-26,47.1,-24.96,43.28,,23.99,,23.99,0.775,0.78866,98.45,121,998.50916,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
78,ZAC 687,mad56,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,50,itacoatiara,amazonas,Jul-22,-2.497,-59.116,2.6,0.4,-25.8,45.8,-24.82,42.04,,23.92,,23.92,0.775,0.78866,98.45,121,998.50916,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
79,ZAC 688,mad56,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,75,itacoatiara,amazonas,Jul-22,-2.497,-59.116,3.4,0.4,-26.2,45.7,-25.6,43.49,,24.43,,24.43,0.775,0.78866,98.45,121,998.50916,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
80,ZAC 689,mad56,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,100,itacoatiara,amazonas,Jul-22,-2.497,-59.116,5.1,0.2,-27.1,45.5,-26.86,43.34,,23.44,,23.44,0.775,0.78866,98.45,121,998.50916,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
81,ZAC 690,mad57,desconhecida,desconhecido,desconhecido,desconhecido,0,itacoatiara,amazonas,Jul-22,-2.495,-59.124,2.8,0.4,-26.8,46.9,-26.33,43.84,,25.3,,25.3,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
82,ZAC 691,mad57,desconhecida,desconhecido,desconhecido,desconhecido,25,itacoatiara,amazonas,Jul-22,-2.495,-59.124,2.6,0.4,-26.3,47.4,-26.14,43.35,,25.01,,25.01,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
83,ZAC 692,mad57,desconhecida,desconhecido,desconhecido,desconhecido,50,itacoatiara,amazonas,Jul-22,-2.495,-59.124,3.6,0.4,-25.9,46.5,-25.6,42.8,,24.68,,24.68,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
84,ZAC 693,mad57,desconhecida,desconhecido,desconhecido,desconhecido,75,itacoatiara,amazonas,Jul-22,-2.495,-59.124,3.7,0.6,-26.8,48.5,-26.34,42.96,,24.67,,24.67,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
85,ZAC 694,mad57,desconhecida,desconhecido,desconhecido,desconhecido,100,itacoatiara,amazonas,Jul-22,-2.495,-59.124,5,0.3,-27.6,44.4,-26.94,41.77,,24.31,,24.31,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
86,ZAC 695,mad58,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,0,itacoatiara,amazonas,Jul-22,-2.493,-59.121,3.3,0.5,-25.8,47.5,-25.44,43.13,,26.01,,26.01,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
87,ZAC 696,mad58,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,25,itacoatiara,amazonas,Jul-22,-2.493,-59.121,3.4,0.4,-25.4,46.7,-24.78,43.22,,24.48,,24.48,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
88,ZAC 697,mad58,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,50,itacoatiara,amazonas,Jul-22,-2.493,-59.121,3.1,0.5,-25.7,47.2,-25.26,41.74,,24.57,,24.57,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
89,ZAC 698,mad58,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,75,itacoatiara,amazonas,Jul-22,-2.493,-59.121,3,0.6,-25.2,47.4,-23.77,39.91,,24.47,,24.47,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
90,ZAC 699,mad58,pequia_marfim,Aspidosperma desmanthum Benth. ex Müll.Arg.,aspidosperma,apocynaceae,100,itacoatiara,amazonas,Jul-22,-2.493,-59.121,4.3,0.2,-27.1,46.1,-25.27,40.16,,24.92,,24.92,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
91,ZAC 790,mad59,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,0,itacoatiara,amazonas,Jul-22,-2.498,-59.118,-0.5,0.1,-28.9,50,-26.48,40.58,,25.77,,25.77,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
92,ZAC 791,mad59,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,25,itacoatiara,amazonas,Jul-22,-2.498,-59.118,0.7,0.2,-29.6,48.4,-25.19,40.2,,24.92,,24.92,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
93,ZAC 792,mad59,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,50,itacoatiara,amazonas,Jul-22,-2.498,-59.118,1.4,0.1,-27.1,52.4,-25.86,42.1,,24.43,,24.43,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
94,ZAC 793,mad59,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,75,itacoatiara,amazonas,Jul-22,-2.498,-59.118,3.3,0.2,-26.6,52.1,-25.23,42.03,,25.09,,25.09,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
95,ZAC 794,mad59,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,100,itacoatiara,amazonas,Jul-22,-2.498,-59.118,3.6,0.2,-28.3,49.3,-26.56,40.75,,25.32,,25.32,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
96,ZAC 795,mad60,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,0,itacoatiara,amazonas,Jul-22,-2.482,-59.126,0.2,0.2,-29.9,49.2,-29.25,42.89,,25.06,,25.06,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
97,ZAC 796,mad60,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,25,itacoatiara,amazonas,Jul-22,-2.482,-59.126,-0.5,0.2,-29.1,51.7,-27.36,41.24,,26.37,,26.37,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
98,ZAC 797,mad60,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,50,itacoatiara,amazonas,Jul-22,-2.482,-59.126,3.4,0.3,-27.6,51.3,-25.36,41.64,,25.68,,25.68,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
99,ZAC 798,mad60,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,75,itacoatiara,amazonas,Jul-22,-2.482,-59.126,3.4,0.3,-26.9,52.8,-25.4,41.41,,25.1,,25.1,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
100,ZAC 799,mad60,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,100,itacoatiara,amazonas,Jul-22,-2.482,-59.126,3.4,0.3,-28.9,50.6,-26.17,40.3,,26.06,,26.06,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
101,ZAC 800,mad61,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,0,itacoatiara,amazonas,Jul-22,-2.483,-59.126,0.1,0.1,-27.1,50.1,-26.16,42.27,,26.45,,26.45,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
102,ZAC 801,mad61,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,25,itacoatiara,amazonas,Jul-22,-2.483,-59.126,2.6,0.2,-27.6,50.3,-25.53,42.05,,26.63,,26.63,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
103,ZAC 802,mad61,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,50,itacoatiara,amazonas,Jul-22,-2.483,-59.126,-1.1,0.1,-27.2,51.3,-25.27,40.93,,26.17,,26.17,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
104,ZAC 803,mad61,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,75,itacoatiara,amazonas,Jul-22,-2.483,-59.126,-1.6,0.1,-28.2,50.6,-25.79,41.45,,25.88,,25.88,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
105,ZAC 804,mad61,louro_gamela,Ocotea rubra Mez,ocotea,lauraceae,100,itacoatiara,amazonas,Jul-22,-2.483,-59.126,4.4,0.4,-29.4,49,-26.79,40.27,,25.88,,25.88,0.76667,0.78997,98.28333,91,1002.08618,26.725,2264,-3.69263,-4.18364,-3.98193,-3.8401
106,ZAC 700,mad71,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,0,itacoatiara,amazonas,Jul-22,-2.495,-59.119,4,0.4,-30.2,49.5,-28.34,43.57,,24.99,,24.99,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
107,ZAC 701,mad71,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,25,itacoatiara,amazonas,Jul-22,-2.495,-59.119,3.4,0.4,-28.5,49.7,-26.82,42.28,,24.1,,24.1,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
108,ZAC 702,mad71,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,50,itacoatiara,amazonas,Jul-22,-2.495,-59.119,3.1,0.4,-28.7,49.9,-27.11,43.29,,24.54,,24.54,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
109,ZAC 703,mad71,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,75,itacoatiara,amazonas,Jul-22,-2.495,-59.119,2.9,0.4,-28.1,49.3,-26.45,43.19,,25.61,,25.61,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
110,ZAC 704,mad71,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,100,itacoatiara,amazonas,Jul-22,-2.495,-59.119,5.5,0.5,-31.4,49.2,-29.56,42.82,,,,,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
111,ZAC 705,mad72,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,0,itacoatiara,amazonas,Jul-22,-2.494,-59.119,4.4,0.4,-28.9,49.9,-27.72,46.93,,24.74,,24.74,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
112,ZAC 706,mad72,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,25,itacoatiara,amazonas,Jul-22,-2.494,-59.119,3.4,0.5,-28.5,51.9,-27.6,44.46,,24.9,,24.9,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
113,ZAC 707,mad72,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,50,itacoatiara,amazonas,Jul-22,-2.494,-59.119,3.5,0.5,-27.4,49.6,-26.05,43.95,,25.68,,25.68,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
114,ZAC 708,mad72,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,75,itacoatiara,amazonas,Jul-22,-2.494,-59.119,4,0.6,-28.2,52.9,-26.69,46.01,,24.24,,24.24,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
115,ZAC 709,mad72,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,100,itacoatiara,amazonas,Jul-22,-2.494,-59.119,6.2,0.5,-28.6,48.5,-27.12,44.11,,23.75,,23.75,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
116,ZAC 710,mad73,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,0,itacoatiara,amazonas,Jul-22,-2.486,-59.126,0.6,0.3,-31.8,47.2,-30.55,43.68,,23.8,,23.8,0.76667,0.78997,98.28333,98,1001.25061,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
117,ZAC 711,mad73,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,25,itacoatiara,amazonas,Jul-22,-2.486,-59.126,1.3,0.3,-29.9,48.3,-28.64,40.06,,22.94,,22.94,0.76667,0.78997,98.28333,98,1001.25061,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
118,ZAC 712,mad73,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,50,itacoatiara,amazonas,Jul-22,-2.486,-59.126,0.5,0.4,-28.6,48.6,-27.74,42.54,,23.02,,23.02,0.76667,0.78997,98.28333,98,1001.25061,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
119,ZAC 713,mad73,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,75,itacoatiara,amazonas,Jul-22,-2.486,-59.126,1.4,0.5,-27.6,49.3,-26.65,42.84,,23.92,,23.92,0.76667,0.78997,98.28333,98,1001.25061,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
120,ZAC 714,mad73,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,100,itacoatiara,amazonas,Jul-22,-2.486,-59.126,5.4,0.3,-30.1,47.1,-28.54,40.75,,24.73,,24.73,0.76667,0.78997,98.28333,98,1001.25061,26.725,2264,-3.70363,-4.17673,-3.98193,-3.8401
121,ZAC 775,mad77,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.496,-59.118,4.9,0.2,-25.1,49,-24.45,44.71,,26.14,,26.14,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
122,ZAC 776,mad77,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.496,-59.118,5.4,0.2,-25.8,51.1,-25.67,45.17,,25.9,,25.9,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
123,ZAC 777,mad77,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.496,-59.118,3.8,0.2,-25.6,51.3,-25.38,45.74,,25.14,,25.14,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
124,ZAC 778,mad77,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.496,-59.118,4.1,0.2,-25.4,51.3,-25.07,46.15,,26.34,,26.34,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
125,ZAC 779,mad77,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.496,-59.118,4.5,0.2,-27.5,45.3,-27.12,44.89,,27.2,,27.2,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
126,ZAC 780,mad78,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.496,-59.12,5,0.2,-29.5,49.7,-28.78,45.51,,25.61,,25.61,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
127,ZAC 781,mad78,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.496,-59.12,5.6,0.1,-28.1,49.8,-27.62,44.17,,26.19,,26.19,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
128,ZAC 782,mad78,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.496,-59.12,5,0.2,-27.4,49.2,-27.07,44.49,,26.61,,26.61,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
129,ZAC 783,mad78,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.496,-59.12,3.9,0.2,-28.1,50.3,-27.21,45.59,,26.5,,26.5,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
130,ZAC 784,mad78,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.496,-59.12,4.1,0.2,-28.2,48.2,-27.9,45.94,,25.76,,25.76,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
131,ZAC 785,mad79,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,itacoatiara,amazonas,Jul-22,-2.493,-59.121,4.3,0.1,-28.6,49,-28.54,45.34,,26.5,,26.5,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
132,ZAC 786,mad79,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,itacoatiara,amazonas,Jul-22,-2.493,-59.121,4.6,0.1,-26.7,49.8,-25.65,46.12,,26.74,,26.74,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
133,ZAC 787,mad79,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,itacoatiara,amazonas,Jul-22,-2.493,-59.121,4.3,0.2,-26.3,47.1,-25.92,44.47,,25.28,,25.28,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
134,ZAC 788,mad79,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,itacoatiara,amazonas,Jul-22,-2.493,-59.121,4.2,0.2,-27.2,50.1,-26.18,45.03,,23.4,,23.4,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
135,ZAC 789,mad79,cumaru,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,itacoatiara,amazonas,Jul-22,-2.493,-59.121,3.8,0.3,-27.8,48.6,-27.18,44.66,,25.96,,25.96,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
136,ZAC 760,mad80,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,0,itacoatiara,amazonas,Jul-22,-2.494,-59.123,3.5,0.1,-29.3,45.7,-26.81,44.06,,26.52,,26.52,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
137,ZAC 761,mad80,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,25,itacoatiara,amazonas,Jul-22,-2.494,-59.123,3.8,0.1,-27.3,46.7,-26.54,41.68,,,,,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
138,ZAC 762,mad80,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,50,itacoatiara,amazonas,Jul-22,-2.494,-59.123,5.6,0.1,-27.1,46.4,-26.51,40.47,,,,,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
139,ZAC 763,mad80,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,75,itacoatiara,amazonas,Jul-22,-2.494,-59.123,4.4,0.1,-27.5,46.8,-26.84,40.84,,,,,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
140,ZAC 764,mad80,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,100,itacoatiara,amazonas,Jul-22,-2.494,-59.123,2.7,0.3,-29.6,43.8,-27.15,40.03,,,,,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
141,ZAC 765,mad81,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,0,itacoatiara,amazonas,Jul-22,-2.497,-59.121,5.1,0.2,-27.5,43.8,-27.04,43.85,,,,,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
142,ZAC 766,mad81,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,25,itacoatiara,amazonas,Jul-22,-2.497,-59.121,5.7,0.1,-27.9,46.1,-26.63,40.85,,26,,26,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
143,ZAC 767,mad81,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,50,itacoatiara,amazonas,Jul-22,-2.497,-59.121,4.4,0.1,-28,46.3,-26.42,40.66,,25.5,,25.5,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
144,ZAC 768,mad81,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,75,itacoatiara,amazonas,Jul-22,-2.497,-59.121,6.9,0.1,-27.9,46.9,-26.66,40.2,,24.79,,24.79,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
145,ZAC 769,mad81,mandioqueira,Qualea paraensis Ducke,qualea,vochysiaceae,100,itacoatiara,amazonas,Jul-22,-2.497,-59.121,5,0.2,-28.3,45.6,-26.97,40.34,,25.03,,25.03,0.775,0.78866,98.45,139,996.36792,26.79167,2253,-3.70363,-4.17673,-3.98193,-3.8401
146,ZAC 770,mad82,qualea_sp,Qualea spp.,qualea,vochysiaceae,0,itacoatiara,amazonas,Jul-22,-2.483,-59.124,1.8,0.1,-29.7,44.9,-29.26,43.87,,25.66,,25.66,0.775,0.78866,98.45,96,1001.48932,26.79167,2253,-3.69263,-4.18364,-3.98193,-3.8401
147,ZAC 771,mad82,qualea_sp,Qualea spp.,qualea,vochysiaceae,25,itacoatiara,amazonas,Jul-22,-2.483,-59.124,1.2,0.1,-27.9,47.7,-27.45,44.13,,25.85,,25.85,0.775,0.78866,98.45,96,1001.48932,26.79167,2253,-3.69263,-4.18364,-3.98193,-3.8401
148,ZAC 772,mad82,qualea_sp,Qualea spp.,qualea,vochysiaceae,50,itacoatiara,amazonas,Jul-22,-2.483,-59.124,1.3,0.1,-26.6,48.4,-26.89,42.88,,25.91,,25.91,0.775,0.78866,98.45,96,1001.48932,26.79167,2253,-3.69263,-4.18364,-3.98193,-3.8401
149,ZAC 773,mad82,qualea_sp,Qualea spp.,qualea,vochysiaceae,75,itacoatiara,amazonas,Jul-22,-2.483,-59.124,1.7,0.1,-27.8,47.1,-27.7,43.45,,26.65,,26.65,0.775,0.78866,98.45,96,1001.48932,26.79167,2253,-3.69263,-4.18364,-3.98193,-3.8401
150,ZAC 774,mad82,qualea_sp,Qualea spp.,qualea,vochysiaceae,100,itacoatiara,amazonas,Jul-22,-2.483,-59.124,0.8,0.2,-29.5,45,-28.19,39.26,,25.34,,25.34,0.775,0.78866,98.45,96,1001.48932,26.79167,2253,-3.69263,-4.18364,-3.98193,-3.8401
151,,mad0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,sao gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,23.196075,,,23.196075,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
152,,mad0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,sao gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,23.411125,,,23.411125,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
153,,mad0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,sao gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,22.883275,,,22.883275,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
154,,mad0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,sao gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,23.851,,,23.851,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
155,,mad0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,sao gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,23.684825,,,23.684825,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
156,,mad01,breu,Protium hebetatum Daly,protium,burseraceae,0,sao gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,26.148125,,,26.148125,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
157,,mad01,breu,Protium hebetatum Daly,protium,burseraceae,25,sao gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,25.1804,,,25.1804,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
158,,mad01,breu,Protium hebetatum Daly,protium,burseraceae,50,sao gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,25.287925,,,25.287925,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
159,,mad01,breu,Protium hebetatum Daly,protium,burseraceae,75,sao gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,26.128575,,,26.128575,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
160,,mad01,breu,Protium hebetatum Daly,protium,burseraceae,100,sao gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,25.8842,,,25.8842,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
161,ZAC 455,mad1,breu,Protium hebetatum Daly,protium,burseraceae,0,maues,amazonas,Mar-14,-3.995544837,-57.58927257,1.7,0.2,-30.2,45.8,-29.4,40.1,26.5489,27.65,-0.04147441137,27.65,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
162,ZAC 456,mad1,breu,Protium hebetatum Daly,protium,burseraceae,25,maues,amazonas,Mar-14,-3.995544837,-57.58927257,1.9,0.1,-29.3,47.5,-28.1,40.2,26.891025,26.86,0.001153730659,26.86,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
163,ZAC 457,mad1,breu,Protium hebetatum Daly,protium,burseraceae,50,maues,amazonas,Mar-14,-3.995544837,-57.58927257,2.9,0.2,-27.5,47.5,-27,40.1,26.88125,26.84,0.001534526854,26.84,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
164,ZAC 458,mad1,breu,Protium hebetatum Daly,protium,burseraceae,75,maues,amazonas,Mar-14,-3.995544837,-57.58927257,3.6,0.2,-28.3,47.4,-27.2,40.4,26.8226,25.61,0.04520814537,25.61,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
165,ZAC 459,mad1,breu,Protium hebetatum Daly,protium,burseraceae,100,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4,0.2,-30.1,46.5,-27.5,40.5,26.0797,26.78,-0.02685230275,26.78,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
166,,mad2,breu,Protium hebetatum Daly,protium,burseraceae,0,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,4,0.2,-29.3,48.5,-29.3,41.5,,,,,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
167,,mad2,breu,Protium hebetatum Daly,protium,burseraceae,25,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,3.1,0.1,-27.4,42.5,-27,41,,,,,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
168,,mad2,breu,Protium hebetatum Daly,protium,burseraceae,50,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,6.2,0.3,-28.6,47.8,-28.2,41.1,,,,,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
169,,mad2,breu,Protium hebetatum Daly,protium,burseraceae,75,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,2.1,0.1,-29.1,47.5,-27.4,41.4,,,,,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
170,,mad2,breu,Protium hebetatum Daly,protium,burseraceae,100,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,3.4,0.2,-28.3,47.1,-27.1,41.1,,,,,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
171,ZAC 415,mad3,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3,0.2,-29.7,50,-28.6,40.4,24.37885,22.45,0.07911981082,22.45,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
172,ZAC 416,mad3,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.6,0.2,-27.6,47.6,-26.9,41.1,24.3984,25.18,-0.03203488753,25.18,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
173,ZAC 417,mad3,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,2.7,0.2,-26.9,47.3,-25.9,41.3,23.55775,25.69,-0.09051161507,25.69,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
174,ZAC 418,mad3,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.4,0.2,-28.5,44.7,-27.2,40.7,24.88715,24.46,0.01716347593,24.46,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
175,ZAC 419,mad3,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.3,0.3,-29,47.8,-27.7,40.4,24.779625,25.26,-0.01938588659,25.26,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
176,ZAC 505,mad4,breu,Protium hebetatum Daly,protium,burseraceae,0,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,4.4,0.2,-30.4,44.3,-29.5,40.3,,25.52,,25.52,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
177,ZAC 506,mad4,breu,Protium hebetatum Daly,protium,burseraceae,25,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,3.5,0.2,-29.3,47.7,-28.4,39.5,,25.58,,25.58,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
178,ZAC 507,mad4,breu,Protium hebetatum Daly,protium,burseraceae,50,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,3.5,0.2,-28.5,46.6,-27.1,40.6,,26.22,,26.22,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
179,ZAC 508,mad4,breu,Protium hebetatum Daly,protium,burseraceae,75,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,3.9,0.2,-28.8,46.8,-27.4,40.5,,26.02,,26.02,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
180,ZAC 509,mad4,breu,Protium hebetatum Daly,protium,burseraceae,100,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,4.2,0.2,-29.1,46.1,-27.6,41,,26.49,,26.49,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
181,ZAC 575,mad5,breu,Protium hebetatum Daly,protium,burseraceae,0,maues,amazonas,Mar-14,-3.995544837,-57.58927257,2.3,0.2,-29.9,44.8,-28.7,40.6,,27.51,,27.51,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
182,ZAC 576,mad5,breu,Protium hebetatum Daly,protium,burseraceae,25,maues,amazonas,Mar-14,-3.995544837,-57.58927257,2.6,0.2,-29.8,46.1,-28.6,40.6,,27.22,,27.22,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
183,ZAC 577,mad5,breu,Protium hebetatum Daly,protium,burseraceae,50,maues,amazonas,Mar-14,-3.995544837,-57.58927257,2.1,0.2,-29.2,46.2,-27.5,39.7,,27.92,,27.92,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
184,ZAC 578,mad5,breu,Protium hebetatum Daly,protium,burseraceae,75,maues,amazonas,Mar-14,-3.995544837,-57.58927257,2.5,0.2,-29.5,45.5,-27.5,40,,26.64,,26.64,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
185,ZAC 579,mad5,breu,Protium hebetatum Daly,protium,burseraceae,100,maues,amazonas,Mar-14,-3.995544837,-57.58927257,2.3,0.2,-28,45.9,-26.3,40.2,,26.81,,26.81,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
186,,mad6,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,sao_gabriel_cachoeira,amazonas,Sep-14,-0.121229892,-67.01310259,3.4,0.1,-33.5,46.3,-31.7,41.2,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
187,,mad6,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,sao_gabriel_cachoeira,amazonas,Sep-14,-0.121229892,-67.01310259,3.5,0.1,-33.4,47.4,-31.7,41.6,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
188,,mad6,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,sao_gabriel_cachoeira,amazonas,Sep-14,-0.121229892,-67.01310259,2.9,0.1,-29,45.9,-28,42.5,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
189,,mad6,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,sao_gabriel_cachoeira,amazonas,Sep-14,-0.121229892,-67.01310259,3.1,0.1,-28.6,47.7,-27.1,40.9,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
190,,mad6,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,sao_gabriel_cachoeira,amazonas,Sep-14,-0.121229892,-67.01310259,5.9,0.1,-29.5,46.4,-27.8,40.5,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
191,ZAC 585,mad7,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,maues,amazonas,Mar-14,-3.995544837,-57.58927257,,,,,,,,27.14,,27.14,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
192,ZAC 586,mad7,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,maues,amazonas,Mar-14,-3.995544837,-57.58927257,,,,,,,,26.45,,26.45,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
193,ZAC 587,mad7,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,maues,amazonas,Mar-14,-3.995544837,-57.58927257,,,,,,,,26.67,,26.67,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
194,ZAC 588,mad7,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,maues,amazonas,Mar-14,-3.995544837,-57.58927257,,,,,,,,25.7,,25.7,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
195,ZAC 589,mad7,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,maues,amazonas,Mar-14,-3.995544837,-57.58927257,,,,,,,,27.18,,27.18,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
196,ZAC 475,mad8,breu,Protium hebetatum Daly,protium,burseraceae,0,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,2.3,0.1,-30.7,46.9,-29.7,41.3,26.284975,26.2,0.003232835489,26.2,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
197,ZAC 476,mad8,breu,Protium hebetatum Daly,protium,burseraceae,25,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.7,0.1,-31.3,47.3,-29.2,40.3,25.39545,24.46,0.03683533861,24.46,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
198,ZAC 477,mad8,breu,Protium hebetatum Daly,protium,burseraceae,50,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.5,0.1,-29.9,47.4,-28.3,40.8,25.092425,25.09,0.00009664271189,25.09,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
199,ZAC 478,mad8,breu,Protium hebetatum Daly,protium,burseraceae,75,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.7,0.1,-30.8,47.6,-29,40.3,24.7112,24.93,-0.008854284697,24.93,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
200,ZAC 479,mad8,breu,Protium hebetatum Daly,protium,burseraceae,100,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.4,0.1,-31.3,47.4,-30.2,41,25.1022,25.3,-0.007879787429,25.3,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
201,ZAC 535,mad9,ucuuba_puna,Iryanthera sp 5,iryanthera,myristicaceae,0,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,2.8,0.3,-31.4,51.6,-29.8,40.5,,25.58,,25.58,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
202,ZAC 536,mad9,ucuuba_puna,Iryanthera sp 5,iryanthera,myristicaceae,25,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,4.3,0.2,-30.1,46.1,-29.2,40.8,,24.62,,24.62,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
203,ZAC 537,mad9,ucuuba_puna,Iryanthera sp 5,iryanthera,myristicaceae,50,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,4.1,0.2,-30.6,44.2,-29.5,41,,25.13,,25.13,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
204,ZAC 538,mad9,ucuuba_puna,Iryanthera sp 5,iryanthera,myristicaceae,75,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,4.1,0.2,-29.1,41.5,-28.3,40.4,,24.85,,24.85,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
205,ZAC 539,mad9,ucuuba_puna,Iryanthera sp 5,iryanthera,myristicaceae,100,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,3.7,0.3,-31.8,46.1,-29.7,40.4,,24.31,,24.31,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
206,ZAC 465,mad10,breu,Protium hebetatum Daly,protium,burseraceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,5.2,0.2,-30.4,47.4,-29.6,40.8,27.516625,25.82,0.06165817937,25.82,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
207,ZAC 466,mad10,breu,Protium hebetatum Daly,protium,burseraceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,5.2,0.2,-30.4,45.7,-28.6,40.1,26.597775,25.89,0.02661030857,25.89,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
208,ZAC 467,mad10,breu,Protium hebetatum Daly,protium,burseraceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,2.9,0.2,-30.3,45.1,-28.4,40.8,26.3534,26.39,-0.001388815105,26.39,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
209,ZAC 468,mad10,breu,Protium hebetatum Daly,protium,burseraceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.5,0.2,-30.5,46.5,-28.1,39.9,25.190175,25.82,-0.02500280367,25.82,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
210,ZAC 469,mad10,breu,Protium hebetatum Daly,protium,burseraceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,5.2,0.3,-30.5,46,-28,39.8,25.111975,25.76,-0.02580541753,25.76,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
211,,mad11,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,2.3,0.2,-30.1,46.3,-28.5,39.8,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
212,,mad11,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,2.5,0.2,-29.5,46.4,-28,40.5,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
213,,mad11,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,2.2,0.2,-29,41.2,-27.4,40.3,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
214,,mad11,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.2,0.3,-30.6,51.2,-28.5,41.1,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
215,,mad11,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.7,0.3,-30.8,46,-29.5,41.9,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
216,ZAC 470,mad12,breu,Protium subserratum (Engl.) Engl.,protium,burseraceae,0,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,2.8,0.2,-31.1,44.9,-28.9,39.9,24.88715,23.95,0.03765597909,23.95,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
217,ZAC 471,mad12,breu,Protium subserratum (Engl.) Engl.,protium,burseraceae,25,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.4,0.2,-30.6,45.6,-28.8,39.7,23.606625,23.54,0.002822300943,23.54,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
218,ZAC 472,mad12,breu,Protium subserratum (Engl.) Engl.,protium,burseraceae,50,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.2,0.2,-31.9,45.4,-29.2,39.3,23.90965,23.35,0.0234068671,23.35,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
219,ZAC 473,mad12,breu,Protium subserratum (Engl.) Engl.,protium,burseraceae,75,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.4,0.2,-32.1,45.4,-30.2,37.2,22.89305,22.93,-0.001614026965,22.93,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
220,ZAC 474,mad12,breu,Protium subserratum (Engl.) Engl.,protium,burseraceae,100,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,4.9,0.2,-32.7,45.6,-31.2,40.3,23.723925,23.41,0.01323242254,23.41,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
221,ZAC 640,mad13,breu,Protium sp.2,protium,burseraceae,0,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,3.9,0.1,-31.1,45.1,-30.3,40.3,,25.49,,25.49,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
222,ZAC 641,mad13,breu,Protium sp.2,protium,burseraceae,25,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,4.1,0.1,-30.6,46.1,-28.9,37.8,,25.65,,25.65,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
223,ZAC 642,mad13,breu,Protium sp.2,protium,burseraceae,50,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,3.4,0.1,-30.1,45.8,-28.2,40.6,,25.31,,25.31,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
224,ZAC 643,mad13,breu,Protium sp.2,protium,burseraceae,75,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,3.2,0.1,-30.3,46,-29,40.6,,25.24,,25.24,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
225,ZAC 644,mad13,breu,Protium sp.2,protium,burseraceae,100,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,3.2,0.1,-30.2,45.4,-28.9,40.3,,25.38,,25.38,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
226,,mad14,ucuuba_puna,Iryanthera cf. macrophylla (Benth.) Warb.,iryanthera,myristicaceae,0,atalaia_do_norte,amazonas,Sep-13,-4.303698078,-70.29106779,2.4,0.1,-30.3,46.5,-30.5,40.3,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
227,,mad14,ucuuba_puna,Iryanthera cf. macrophylla (Benth.) Warb.,iryanthera,myristicaceae,25,atalaia_do_norte,amazonas,Sep-13,-4.303698078,-70.29106779,2.5,0.1,-30.7,45.1,-30.5,39.8,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
228,,mad14,ucuuba_puna,Iryanthera cf. macrophylla (Benth.) Warb.,iryanthera,myristicaceae,50,atalaia_do_norte,amazonas,Sep-13,-4.303698078,-70.29106779,2.3,0.1,-30.4,45.2,-30,39.2,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
229,,mad14,ucuuba_puna,Iryanthera cf. macrophylla (Benth.) Warb.,iryanthera,myristicaceae,75,atalaia_do_norte,amazonas,Sep-13,-4.303698078,-70.29106779,3.3,0.1,-29.4,45.7,-29.2,39.6,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
230,,mad14,ucuuba_puna,Iryanthera cf. macrophylla (Benth.) Warb.,iryanthera,myristicaceae,100,atalaia_do_norte,amazonas,Sep-13,-4.303698078,-70.29106779,3.2,0.1,-29.9,41.6,-28.9,39.4,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
231,ZAC 605,mad15,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.4,0.3,-28.6,45.5,-27.5,40.5,,25.58,,25.58,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
232,ZAC 606,mad15,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.2,0.2,-28.5,42.8,-27.7,40.8,,25.15,,25.15,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
233,ZAC 607,mad15,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.5,0.2,-28.8,45.7,-27.7,38.7,,24.67,,24.67,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
234,ZAC 608,mad15,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.3,0.2,-29.1,45.2,-27.4,41,,25.25,,25.25,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
235,ZAC 609,mad15,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,5.5,0.2,-28.9,45.8,-27.5,40.3,,25.19,,25.19,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
236,ZAC 480,mad16,breu,Protium sp.2,protium,burseraceae,0,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,3.3,0.1,-30.4,45.6,-29.6,40.1,22.1306,21.03,0.04973204522,21.03,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
237,ZAC 481,mad16,breu,Protium sp.2,protium,burseraceae,25,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,1.8,0.1,-31.8,45.2,-30.5,39.9,25.74735,24.94,0.03135662505,24.94,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
238,ZAC 482,mad16,breu,Protium sp.2,protium,burseraceae,50,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,3.8,0.1,-31,45.1,-30.3,39.2,24.1247,25.91,-0.07400299278,25.91,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
239,ZAC 483,mad16,breu,Protium sp.2,protium,burseraceae,75,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,3.2,0.1,-29.8,45.2,-28.7,39.9,24.545025,25.54,-0.04053672791,25.54,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
240,ZAC 484,mad16,breu,Protium sp.2,protium,burseraceae,100,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,3.2,0.1,-30.7,44,-30.1,40.2,23.01035,25.49,-0.1077623765,25.49,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
241,ZAC 515,mad17,breu_vermelho,Protium cf. hebetatum Daly,protium,burseraceae,0,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.5,0.2,-27.6,45.4,-27.2,40.2,,26.38,,26.38,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
242,ZAC 516,mad17,breu_vermelho,Protium cf. hebetatum Daly,protium,burseraceae,25,maues,amazonas,Mar-14,-3.995544837,-57.58927257,3.5,0.2,-27.1,47.2,-26,40.1,,25.67,,25.67,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
243,ZAC 517,mad17,breu_vermelho,Protium cf. hebetatum Daly,protium,burseraceae,50,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.5,0.2,-28,45,-27,40.4,,25.92,,25.92,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
244,ZAC 518,mad17,breu_vermelho,Protium cf. hebetatum Daly,protium,burseraceae,75,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.3,0.2,-27.8,45,-27.2,40.1,,26.12,,26.12,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
245,ZAC 519,mad17,breu_vermelho,Protium cf. hebetatum Daly,protium,burseraceae,100,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.1,0.2,-29.3,46.1,-28,40.2,,24.44,,24.44,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
246,ZAC 615,mad18,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.8,0.3,-28.5,46.5,-27.4,39.9,,25.61,,25.61,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
247,ZAC 616,mad18,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.4,0.2,-29.7,47.5,-28.6,39.2,,24.62,,24.62,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
248,ZAC 617,mad18,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.6,0.2,-28.7,45.2,-27.9,38.3,,25.11,,25.11,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
249,ZAC 618,mad18,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.3,0.4,-29.4,45.5,-28.4,39.4,,25.41,,25.41,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
250,ZAC 619,mad18,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.4,0.4,-29.8,45.9,-28.1,38.9,,24.79,,24.79,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
251,ZAC 610,mad19,ucuuba_puna,Iryanthera sp.6,iryanthera,myristicaceae,0,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,3.7,0.2,-31.2,45.3,-29.9,39,,24.17,,24.17,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
252,ZAC 611,mad19,ucuuba_puna,Iryanthera sp.6,iryanthera,myristicaceae,25,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,3.9,0.2,-31.6,45.2,-30.1,39.1,,23.65,,23.65,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
253,ZAC 612,mad19,ucuuba_puna,Iryanthera sp.6,iryanthera,myristicaceae,50,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,4.1,0.2,-31.6,45.9,-29.4,39.4,,25.02,,25.02,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
254,ZAC 613,mad19,ucuuba_puna,Iryanthera sp.6,iryanthera,myristicaceae,75,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,3.6,0.3,-31.8,46,-30.4,38.8,,23.72,,23.72,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
255,ZAC 614,mad19,ucuuba_puna,Iryanthera sp.6,iryanthera,myristicaceae,100,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,3.2,0.3,-31.9,46.2,-30.1,41.1,,23.65,,23.65,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
256,ZAC 460,mad20,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,5,0.1,-29.9,43.4,-28.5,38.5,25.8842,25.28,0.02334242511,25.28,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
257,ZAC 461,mad20,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,5.3,0.1,-30.1,45.5,-29.5,38.3,24.7894,25.5,-0.02866547799,25.5,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
258,ZAC 462,mad20,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,4.1,0.1,-29.8,45.2,-28.7,37.7,24.45705,25.2,-0.03037774384,25.2,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
259,ZAC 463,mad20,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,5.5,0.1,-30.5,45.6,-28.8,37.8,23.90965,24.97,-0.0443482025,24.97,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
260,ZAC 464,mad20,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,manicore,amazonas,Mar-14,-6.009706576,-61.8686565,5.6,0.2,-29.8,45.5,-28.7,38.1,24.388625,25.25,-0.03531871928,25.25,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
261,ZAC 600,mad21,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,23.21,,23.21,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
262,ZAC 601,mad21,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,23.49,,23.49,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
263,ZAC 602,mad21,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,23.96,,23.96,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
264,ZAC 603,mad21,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,23.23,,23.23,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
265,ZAC 604,mad21,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,23.1,,23.1,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
266,ZAC 525,mad22,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.2,0.2,-31,45.9,-30,39.1,,24.75,,24.75,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
267,ZAC 526,mad22,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.9,0.2,-30.3,44.8,-29.3,38.5,,24.72,,24.72,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
268,ZAC 527,mad22,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.7,0.2,-29.9,44.5,-28.3,38.5,,25.39,,25.39,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
269,ZAC 528,mad22,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.9,0.2,-30,45.5,-29.2,39.6,,25.71,,25.71,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
270,ZAC 529,mad22,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.6,0.3,-30.9,46.2,-29.2,36.8,,24.46,,24.46,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
271,ZAC 490,mad23,breu,Protium cf. elegans Engl.,protium,burseraceae,0,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,3.4,0.2,-32.3,45,-30.4,39.2,22.785525,23.47,-0.03003990472,23.47,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
272,ZAC 491,mad23,breu,Protium cf. elegans Engl.,protium,burseraceae,25,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,3.1,0.1,-32,44.8,-31.2,40,24.18335,24.05,0.005514124387,24.05,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
273,ZAC 492,mad23,breu,Protium cf. elegans Engl.,protium,burseraceae,50,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,1.5,0.1,-31.8,45.1,-30.5,40.9,27.125625,23.8,0.1226008617,23.8,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
274,ZAC 493,mad23,breu,Protium cf. elegans Engl.,protium,burseraceae,75,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,3.2,0.1,-31.3,45.1,-29.9,40.2,23.391575,23.65,-0.01104778109,23.65,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
275,ZAC 494,mad23,breu,Protium cf. elegans Engl.,protium,burseraceae,100,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,4.5,0.1,-31.7,45.2,-30.1,40,23.71415,23.87,-0.006572025563,23.87,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
276,ZAC 430,mad24,ucuuba_puna,Iryanthera sp.1,iryanthera,myristicaceae,0,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,24.075825,23.15,0.03845454932,23.15,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
277,ZAC 431,mad24,ucuuba_puna,Iryanthera sp.2,iryanthera,myristicaceae,25,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,22.93215,23.54,-0.02650645491,23.54,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
278,ZAC 432,mad24,ucuuba_puna,Iryanthera sp.3,iryanthera,myristicaceae,50,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,24.857825,24.58,0.0111765611,24.58,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
279,ZAC 433,mad24,ucuuba_puna,Iryanthera sp.4,iryanthera,myristicaceae,75,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,23.75325,23.83,-0.003231136792,23.83,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
280,ZAC 434,mad24,ucuuba_puna,Iryanthera sp.5,iryanthera,myristicaceae,100,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,23.372025,23.9,-0.02259004087,23.9,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
281,ZAC 550,mad25,breu,Protium hebetatum Daly,protium,burseraceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.9,0.1,-29.8,46.2,-28.5,39.1,,23.87,,23.87,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
282,ZAC 551,mad25,breu,Protium hebetatum Daly,protium,burseraceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.5,0.1,-27.6,46.3,-26.5,39.4,,24.13,,24.13,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
283,ZAC 552,mad25,breu,Protium hebetatum Daly,protium,burseraceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.4,0.1,-27.5,46,-26,38.7,,23.67,,23.67,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
284,ZAC 553,mad25,breu,Protium hebetatum Daly,protium,burseraceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.2,0.2,-28.3,46.6,-26.8,39.4,,23.2,,23.2,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
285,ZAC 554,mad25,breu,Protium hebetatum Daly,protium,burseraceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.1,0.1,-29.8,41.2,-28.5,40,,23.84,,23.84,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
286,ZAC 500,mad26,breu,Protium hebetatum Daly,protium,burseraceae,0,maues,amazonas,Mar-14,-3.995544837,-57.58927257,3.2,0.2,-30.6,45.2,-30.3,40.1,27.203825,26.84,0.01337403839,26.84,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
287,ZAC 501,mad26,breu,Protium hebetatum Daly,protium,burseraceae,25,maues,amazonas,Mar-14,-3.995544837,-57.58927257,2.9,0.2,-27.8,45.3,-26.8,39,24.408175,26.65,-0.09184730116,26.65,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
288,ZAC 502,mad26,breu,Protium hebetatum Daly,protium,burseraceae,50,maues,amazonas,Mar-14,-3.995544837,-57.58927257,2.5,0.2,-26.7,45.3,-26.5,39.9,26.265425,26.77,-0.01921061624,26.77,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
289,ZAC 503,mad26,breu,Protium hebetatum Daly,protium,burseraceae,75,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.4,0.1,-29.7,45.4,-28.9,40.4,26.9399,26.69,0.009276203698,26.69,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
290,ZAC 504,mad26,breu,Protium hebetatum Daly,protium,burseraceae,100,maues,amazonas,Mar-14,-3.995544837,-57.58927257,3.8,0.1,-29.3,45.6,-28.3,36.1,27.2136,26.74,0.01740306317,26.74,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
291,,mad27,breu,Protium sp.4,protium,burseraceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.7,0.2,-30.1,47,-27.5,40.6,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
292,,mad27,breu,Protium sp.4,protium,burseraceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.3,0.2,-30.1,44.5,-27.1,38.2,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
293,,mad27,breu,Protium sp.4,protium,burseraceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.4,0.2,-28.7,47.5,-27.6,42,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
294,,mad27,breu,Protium sp.4,protium,burseraceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.5,0.2,-28.5,48,-29.2,39.2,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
295,,mad27,breu,Protium sp.4,protium,burseraceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,2.9,0.2,-28.6,46.8,-29.1,39.4,,,,,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
296,ZAC 625,mad28,ucuuba_puna,Iryanthera cf. paradoxa (Schwacke) Warb.,iryanthera,myristicaceae,0,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,1.4,0.2,-30.5,44.9,-29,40.2,,24.77,,24.77,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
297,ZAC 626,mad28,ucuuba_puna,Iryanthera cf. paradoxa (Schwacke) Warb.,iryanthera,myristicaceae,25,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2.7,0.1,-31,45,-29.8,40.9,,23.33,,23.33,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
298,ZAC 627,mad28,ucuuba_puna,Iryanthera cf. paradoxa (Schwacke) Warb.,iryanthera,myristicaceae,50,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,1.7,0.1,-30.6,43.3,-29.4,40.1,,23.99,,23.99,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
299,ZAC 628,mad28,ucuuba_puna,Iryanthera cf. paradoxa (Schwacke) Warb.,iryanthera,myristicaceae,75,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2.4,0.2,-30.3,44.9,-28.8,41.2,,24.14,,24.14,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
300,ZAC 629,mad28,ucuuba_puna,Iryanthera cf. paradoxa (Schwacke) Warb.,iryanthera,myristicaceae,100,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,1.9,0.2,-31,45.4,-29.7,39.9,,23.96,,23.96,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
301,ZAC 485,mad29,breu,Protium hebetatum Daly,protium,burseraceae,0,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.7,0.2,-30.3,44.7,-29.2,39.2,24.603675,25.34,-0.0299274397,25.34,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
302,ZAC 486,mad29,breu,Protium hebetatum Daly,protium,burseraceae,25,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.3,0.2,-30.5,44.7,-29.7,38.9,24.14425,24.65,-0.02094701637,24.65,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
303,ZAC 487,mad29,breu,Protium hebetatum Daly,protium,burseraceae,50,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.5,0.2,-29.8,45.6,-29.2,41,24.486375,23.27,0.04967558489,23.27,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
304,ZAC 488,mad29,breu,Protium hebetatum Daly,protium,burseraceae,75,maues,amazonas,Mar-14,-3.995544837,-57.58927257,3.1,0.1,-30.1,45.9,-29,39.5,24.603675,24.52,0.003400914701,24.52,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
305,ZAC 489,mad29,breu,Protium hebetatum Daly,protium,burseraceae,100,maues,amazonas,Mar-14,-3.995544837,-57.58927257,4.5,0.2,-29.9,44.9,-28.3,39.1,24.505925,24.1,0.01656436148,24.1,0.85167,0.77358,99.99167,61,1005.67358,27.16667,2273,-3.51406,-3.96502,-3.78967,-3.73041
306,ZAC 495,mad30,breu,Protium hebetatum Daly,protium,burseraceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.3,0.2,-29.8,44.4,-29.9,35.9,25.463875,24.69,0.03039109326,24.69,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
307,ZAC 496,mad30,breu,Protium hebetatum Daly,protium,burseraceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,0.6,0.1,-30.8,46.8,-29.7,39.9,24.271325,26.71,-0.1004755612,26.71,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
308,ZAC 497,mad30,breu,Protium hebetatum Daly,protium,burseraceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,1.3,0.1,-32.4,45.8,-30.9,40,26.226325,26.53,-0.0115790146,26.53,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
309,ZAC 498,mad30,breu,Protium hebetatum Daly,protium,burseraceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,1,0.1,-31.4,46.9,-30.1,39.6,25.287925,25.47,-0.007200076716,25.47,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
310,ZAC 499,mad30,breu,Protium hebetatum Daly,protium,burseraceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.4,0.2,-30.5,47,-29.3,37.7,25.39545,25.08,0.01242151645,25.08,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
311,ZAC 555,mad31,breu,Protium hebetatum Daly,protium,burseraceae,0,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,5.3,0.1,-32.1,46.2,-31.3,40.4,,25.47,,25.47,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
312,ZAC 556,mad31,breu,Protium hebetatum Daly,protium,burseraceae,25,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.9,0.1,-32.5,46.2,-31.7,40.7,,24.95,,24.95,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
313,ZAC 557,mad31,breu,Protium hebetatum Daly,protium,burseraceae,50,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,4.1,0.2,-30.9,46.9,-29.1,39.4,,25.27,,25.27,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
314,ZAC 558,mad31,breu,Protium hebetatum Daly,protium,burseraceae,75,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,3.9,0.1,-31.8,46.3,-30.7,39.2,,25.75,,25.75,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
315,ZAC 559,mad31,breu,Protium hebetatum Daly,protium,burseraceae,100,sao_gabriel_cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,5.6,0.2,-32.7,44.7,-31.4,40.5,,26.18,,26.18,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
316,ZAC 560,mad32,ucuuba_puna,Iryanthera cf. laevis Markgr.,iryanthera,myristicaceae,0,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,3.4,0.2,-29.4,46.9,-29.8,40.3,,26.36,,26.36,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
317,ZAC 561,mad32,ucuuba_puna,Iryanthera cf. laevis Markgr.,iryanthera,myristicaceae,25,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,2,0.2,-30.6,46.8,-29.7,39.7,,26.02,,26.02,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
318,ZAC 562,mad32,ucuuba_puna,Iryanthera cf. laevis Markgr.,iryanthera,myristicaceae,50,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,3.1,0.2,-29.5,46.5,-28.4,39.3,,26.22,,26.22,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
319,ZAC 563,mad32,ucuuba_puna,Iryanthera cf. laevis Markgr.,iryanthera,myristicaceae,75,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,2.2,0.2,-31.2,48.4,-27.7,39.4,,25.4,,25.4,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
320,ZAC 564,mad32,ucuuba_puna,Iryanthera cf. laevis Markgr.,iryanthera,myristicaceae,100,manicore,amazonas,Dec-14,-6.009706576,-61.8686565,3.5,0.2,-30.9,46.4,-29.5,39.3,,24.91,,24.91,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
321,ZAC 440,mad33,breu,Protium hebetatum Daly,protium,burseraceae,0,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,26.500025,26.37,0.004906599145,26.37,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
322,ZAC 441,mad33,breu,Protium hebetatum Daly,protium,burseraceae,25,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,26.187225,25.91,0.01058626869,25.91,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
323,ZAC 442,mad33,breu,Protium hebetatum Daly,protium,burseraceae,50,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,26.76395,25.89,0.03265399913,25.89,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
324,ZAC 443,mad33,breu,Protium hebetatum Daly,protium,burseraceae,75,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,25.776675,25.06,0.02780323684,25.06,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
325,ZAC 444,mad33,breu,Protium hebetatum Daly,protium,burseraceae,100,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,26.588,25.74,0.03189408756,25.74,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
326,ZAC 425,mad34,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,0,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,2.8,0.2,-33,49,-31.4,39.5,23.469775,24.54,-0.0456001389,24.54,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
327,ZAC 426,mad34,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,25,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.1,0.2,-32.4,50.1,-30.8,38.6,24.036725,24.55,-0.0213537826,24.55,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
328,ZAC 427,mad34,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,50,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.6,0.3,-30.3,48.2,-28,39.9,23.75325,24.09,-0.01417700736,24.09,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
329,ZAC 428,mad34,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,75,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,4.5,0.5,-29.8,43.1,-28.8,39.9,22.140375,22.14,0.0000169373825,22.14,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
330,ZAC 429,mad34,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,100,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,4.3,0.3,-30,47,-28.8,40.2,23.978075,23.53,0.01868686289,23.53,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
331,ZAC 580,mad35,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,0,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,25.5,,25.5,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
332,ZAC 581,mad35,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,25,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,25.1,,25.1,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
333,ZAC 582,mad35,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,50,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,24.8,,24.8,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
334,ZAC 583,mad35,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,75,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,25.56,,25.56,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
335,ZAC 584,mad35,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,100,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,24.42,,24.42,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
336,,mad37,ucuuba_puna,Iryanthera cf. juruensis Warb.,iryanthera,myristicaceae,0,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,1.7,0.2,-32.4,47,-31.1,39.6,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
337,,mad37,ucuuba_puna,Iryanthera cf. juruensis Warb.,iryanthera,myristicaceae,25,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,1.7,0.2,-31.2,46.4,-29.1,39.4,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
338,,mad37,ucuuba_puna,Iryanthera cf. juruensis Warb.,iryanthera,myristicaceae,50,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,1.1,0.1,-32.1,46.5,-30.7,39.6,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
339,,mad37,ucuuba_puna,Iryanthera cf. juruensis Warb.,iryanthera,myristicaceae,75,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,1.4,0.1,-32.4,46.5,-31.2,39.5,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
340,,mad37,ucuuba_puna,Iryanthera cf. juruensis Warb.,iryanthera,myristicaceae,100,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,0.9,0.2,-31.5,45.5,-31,39.5,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
341,ZAC 510,mad38,breu,Protium sp.3,protium,burseraceae,0,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.9,0.2,-31.7,44.8,-29.6,39.9,,26.72,,26.72,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
342,ZAC 511,mad38,breu,Protium sp.3,protium,burseraceae,25,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.9,0.1,-30.5,45.4,-30.5,39.9,,26.04,,26.04,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
343,ZAC 512,mad38,breu,Protium sp.3,protium,burseraceae,50,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,4.3,0.1,-30.4,45.6,-31.2,40.1,,25.01,,25.01,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
344,ZAC 513,mad38,breu,Protium sp.3,protium,burseraceae,75,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,5,0.2,-28.6,45.9,-30.7,39.8,,26.36,,26.36,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
345,ZAC 514,mad38,breu,Protium sp.3,protium,burseraceae,100,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,5,0.2,-30.4,43.8,-29.5,39.1,,25.43,,25.43,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
346,ZAC 530,mad39,breu,Protium hebetatum Daly,protium,burseraceae,0,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.7,0.2,-33,45.6,-31.8,40,,25.39,,25.39,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
347,ZAC 531,mad39,breu,Protium hebetatum Daly,protium,burseraceae,25,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.4,0.2,-32.1,45.3,-31,40,,25.59,,25.59,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
348,ZAC 532,mad39,breu,Protium hebetatum Daly,protium,burseraceae,50,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,4.2,0.1,-32.2,45.5,-31,40.2,,25.51,,25.51,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
349,ZAC 533,mad39,breu,Protium hebetatum Daly,protium,burseraceae,75,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.8,0.1,-32.2,45.6,-30.7,37,,26.07,,26.07,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
350,ZAC 534,mad39,breu,Protium hebetatum Daly,protium,burseraceae,100,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,4.4,0.2,-32.5,45.7,-31.4,44,,26.83,,26.83,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
351,ZAC 570,mad40,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,0,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,2.5,0.2,-31.9,47.1,-30.4,39.7,,26.29,,26.29,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
352,ZAC 571,mad40,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,25,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.4,0.2,-30.4,49.9,-28.8,40,,23.7,,23.7,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
353,ZAC 572,mad40,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,50,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.2,0.2,-29.7,50.2,-27.4,40.6,,25.48,,25.48,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
354,ZAC 573,mad40,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,75,jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.6,0.2,-29.7,46.2,-27.9,39.8,,26.82,,26.82,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
355,ZAC 574,mad40,ucuuba_puna,Iryanthera coriacea Ducke,iryanthera,myristicaceae,100,Jurua,amazonas,Jul-13,-3.907183314,-66.05514576,3.5,0.3,-28.8,47.6,-27.3,41,,28.01,,28.01,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
356,ZAC 410,mad41,ucuuba_puna,Iryanthera cf. hostmannii (Benth.) Warb.,iryanthera,myristicaceae,0,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2,0.1,-29.8,46.7,-28.6,44,21.808025,23.66,-0.08492172033,23.66,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
357,ZAC 411,mad41,ucuuba_puna,Iryanthera cf. hostmannii (Benth.) Warb.,iryanthera,myristicaceae,25,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2.2,0.1,-32.2,45.7,-30.2,42.2,23.235175,23.09,0.006248070006,23.09,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
358,ZAC 412,mad41,ucuuba_puna,Iryanthera cf. hostmannii (Benth.) Warb.,iryanthera,myristicaceae,50,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2.2,0.2,-32.6,46.2,-31.8,40.5,22.42385,24.58,-0.09615431784,24.58,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
359,ZAC 413,mad41,ucuuba_puna,Iryanthera cf. hostmannii (Benth.) Warb.,iryanthera,myristicaceae,75,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2.6,0.1,-31,46.2,-29.9,41.5,21.48545,22.59,-0.05140920949,22.59,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
360,ZAC 414,mad41,ucuuba_puna,Iryanthera cf. hostmannii (Benth.) Warb.,iryanthera,myristicaceae,100,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,3.3,0.2,-31,46.5,-29.6,41.2,20.58615,22.72,-0.1036546416,22.72,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
361,,mad42,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,0,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,0.9,0.1,-33,44.7,-32.4,41.1,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
362,,mad42,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,25,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,0.4,0.1,-31.5,44.7,-30.6,41.4,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
363,,mad42,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,50,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,-0.3,0.1,-29.6,45.6,-28.7,41.1,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
364,,mad42,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,75,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,0.5,0.1,-30.1,45.8,-28.5,41.1,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
365,,mad42,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,100,atalaia_do_norte,amazonas,Jul-13,-4.303698078,-70.29106779,0.9,0.1,-31,45.8,-29.7,40.6,,,,,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
366,ZAC 660,mad43,breu,Protium cf. elegans Engl.,protium,burseraceae,0,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2.1,0.2,-29.1,48.4,-28,41.3,,26.24,,26.24,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
367,ZAC 661,mad43,breu,Protium cf. elegans Engl.,protium,burseraceae,25,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2,0.1,-28.3,49,-26.7,40.7,,10.18,,10.18,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
368,ZAC 662,mad43,breu,Protium cf. elegans Engl.,protium,burseraceae,50,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2,0.2,-28.6,49.2,-26.6,39.5,,25.17,,25.17,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
369,ZAC 663,mad43,breu,Protium cf. elegans Engl.,protium,burseraceae,75,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,0.8,0.2,-28.9,46.8,-27.4,41.3,,25.67,,25.67,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
370,ZAC 664,mad43,breu,Protium cf. elegans Engl.,protium,burseraceae,100,atalaia_do_norte,amazonas,Aug-13,-4.303698078,-70.29106779,2.9,0.2,-28.3,45.9,-26.5,41.9,,25.58,,25.58,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.95343,-5.12316,-5.456932
371,ZAC 645,mad44,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,0,zf2,amazonas,Feb-22,-2.63795,-60.14972,1.4,0.1,-28.6,51.6,-28.3,42.1,,27.09,,27.09,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
372,ZAC 646,mad44,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,25,zf2,amazonas,Feb-22,-2.63795,-60.14972,1.5,0.1,-27.9,52,-27.1,41.5,,26.82,,26.82,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
373,ZAC 647,mad44,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,50,zf2,amazonas,Feb-22,-2.63795,-60.14972,0.2,0.1,-27.6,48.5,-26.5,40.3,,27.55,,27.55,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
374,ZAC 648,mad44,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,75,zf2,amazonas,Feb-22,-2.63795,-60.14972,0.9,0.1,-28.5,50.1,-26.4,41.5,,26.49,,26.49,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
375,ZAC 649,mad44,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,100,zf2,amazonas,Feb-22,-2.63795,-60.14972,1.2,0.2,-28.5,48.8,-26.7,41.3,,26.86,,26.86,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
376,ZAC 650,mad46,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,0,zf2,amazonas,Feb-22,-2.63795,-60.14972,,,,,,,,26.52,,26.52,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
377,ZAC 651,mad46,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,25,zf2,amazonas,Feb-22,-2.63795,-60.14972,,,,,,,,26.67,,26.67,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
378,ZAC 652,mad46,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,50,zf2,amazonas,Feb-22,-2.63795,-60.14972,,,,,,,,27.23,,27.23,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
379,ZAC 653,mad46,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,75,zf2,amazonas,Feb-22,-2.63795,-60.14972,,,,,,,,24.89,,24.89,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
380,ZAC 654,mad46,mata_mata,Eschweilera coriacea (DC.) S.A. Mori,eschweilera,lecythidaceae,100,zf2,amazonas,Feb-22,-2.63795,-60.14972,,,,,,,,25,,25,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
381,ZAC 655,mad47,piaozinho,Micrandropsis scleroxylon,micrandropsis,fabaceae,0,zf2,amazonas,Feb-22,-2.63795,-60.14972,3.8,0.2,-30.2,45.8,-28.8,41.6,,25.43,,25.43,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
382,ZAC 656,mad47,piaozinho,Micrandropsis scleroxylon,micrandropsis,fabaceae,25,zf2,amazonas,Feb-22,-2.63795,-60.14972,4.1,0.3,-29.1,46.5,-27.8,40.1,,25.04,,25.04,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
383,ZAC 657,mad47,piaozinho,Micrandropsis scleroxylon,micrandropsis,fabaceae,50,zf2,amazonas,Feb-22,-2.63795,-60.14972,3.8,0.3,-28.4,47.7,-26.5,41.7,,24.5,,24.5,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
384,ZAC 658,mad47,piaozinho,Micrandropsis scleroxylon,micrandropsis,fabaceae,75,zf2,amazonas,Feb-22,-2.63795,-60.14972,3.8,0.3,-29,48.7,-26.7,41.4,,25.36,,25.36,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
385,ZAC 659,mad47,piaozinho,Micrandropsis scleroxylon,micrandropsis,fabaceae,100,zf2,amazonas,Feb-22,-2.63795,-60.14972,3.3,0.3,-28.7,47.7,-26.9,41.9,,25.12,,25.12,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
386,ZAC 630,mad48,cajui,Anacardium spp,anacardium,anacardiaceae,0,zf2,amazonas,Feb-22,-2.63795,-60.14972,1.8,0.8,-30.3,48.8,-29.6,40.4,,26.05,,26.05,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
387,ZAC 631,mad48,cajui,Anacardium spp,anacardium,anacardiaceae,25,zf2,amazonas,Feb-22,-2.63795,-60.14972,1.6,0.9,-30.5,48.3,-29.4,41.5,,26.92,,26.92,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
388,ZAC 632,mad48,cajui,Anacardium spp,anacardium,anacardiaceae,50,zf2,amazonas,Feb-22,-2.63795,-60.14972,1.6,0.6,-29.6,47.5,-28.9,39.4,,26.52,,26.52,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
389,ZAC 633,mad48,cajui,Anacardium spp,anacardium,anacardiaceae,75,zf2,amazonas,Feb-22,-2.63795,-60.14972,2.5,0.3,-29.1,46.5,-28.3,40.8,,26.04,,26.04,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
390,ZAC 634,mad48,cajui,Anacardium spp,anacardium,anacardiaceae,100,zf2,amazonas,Feb-22,-2.63795,-60.14972,2.7,0.2,-28.3,47,-27.1,41.7,,26.46,,26.46,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
391,ZAC 635,mad49,cardeiro,Scleronema micranthum,scleronema,malvaceae,0,zf2,amazonas,Feb-22,-2.63795,-60.14972,-1.2,0.1,-31.2,47.9,-29.9,41.4,,27.48,,27.48,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
392,ZAC 636,mad49,cardeiro,Scleronema micranthum,scleronema,malvaceae,25,zf2,amazonas,Feb-22,-2.63795,-60.14972,-1.7,0.1,-29.9,47.4,-28.7,40.8,,26.56,,26.56,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
393,ZAC 637,mad49,cardeiro,Scleronema micranthum,scleronema,malvaceae,50,zf2,amazonas,Feb-22,-2.63795,-60.14972,-0.4,0.1,-30,47.8,-28.2,40,,26.41,,26.41,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
394,ZAC 638,mad49,cardeiro,Scleronema micranthum,scleronema,malvaceae,75,zf2,amazonas,Feb-22,-2.63795,-60.14972,4.3,0.1,-30.6,47.2,-29.9,42.1,,26.61,,26.61,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
395,ZAC 639,mad49,cardeiro,Scleronema micranthum,scleronema,malvaceae,100,zf2,amazonas,Feb-22,-2.63795,-60.14972,-0.8,0.1,-29.3,46.8,-26.9,41.5,,25.4,,25.4,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
396,ZAC 665,mad50,angelim,Hymenolobium spp/Anadenanthera spp,hymenaea,fabaceae,0,zf2,amazonas,Feb-22,-2.63795,-60.14972,2.9,0.2,-27.3,48.6,-25,41.6,,24.44,,24.44,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
397,ZAC 666,mad50,angelim,Hymenolobium spp/Anadenanthera spp,hymenaea,fabaceae,25,zf2,amazonas,Feb-22,-2.63795,-60.14972,2.7,0.2,-26.7,48.6,-25,42.5,,24.87,,24.87,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
398,ZAC 667,mad50,angelim,Hymenolobium spp/Anadenanthera spp,hymenaea,fabaceae,50,zf2,amazonas,Feb-22,-2.63795,-60.14972,3.1,0.2,-26.8,49.6,-25.2,41.9,,24.24,,24.24,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
399,ZAC 668,mad50,angelim,Hymenolobium spp/Anadenanthera spp,hymenaea,fabaceae,75,zf2,amazonas,Feb-22,-2.63795,-60.14972,3.2,0.2,-26.3,43.9,-24.5,43.2,,23.75,,23.75,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
400,ZAC 669,mad50,angelim,Hymenolobium spp/Anadenanthera spp,hymenaea,fabaceae,100,zf2,amazonas,Feb-22,-2.63795,-60.14972,4.1,0.2,-27.1,47.2,-24.8,41.3,,24.2,,24.2,0.71,0.80446,96.09167,90,1002.20557,26.67083,2354,-3.82334,-4.17565,-4.08643,-3.98924
401,,mad499,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,6.9,0.1,-28.8,45.2,,,28.337725,,,28.337725,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
402,,mad499,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,6.4,0.1,-27.1,46.2,,,26.773725,,,26.773725,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
403,,mad499,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,6.7,0.1,-27.2,46.6,,,27.23315,,,27.23315,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
404,,mad499,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,6.9,0.1,-27,46.1,,,27.106075,,,27.106075,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
405,,mad499,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,8.3,0.1,-27.9,46.9,,,26.7053,,,26.7053,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
406,,mad500,cupiuba,goupeia spp.,goupeia,euphorbiaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,6,0.1,-28.3,46.7,,,27.5655,,,27.5655,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
407,,mad500,cupiuba,goupeia spp.,goupeia,euphorbiaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,7.4,0.1,-25.8,47.5,,,26.3143,,,26.3143,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
408,,mad500,cupiuba,goupeia spp.,goupeia,euphorbiaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,7,0.1,-26.7,47.5,,,24.779625,,,24.779625,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
409,,mad500,cupiuba,goupeia spp.,goupeia,euphorbiaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,5.9,0.1,-26.6,47.7,,,26.6662,,,26.6662,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
410,,mad500,cupiuba,goupeia spp.,goupeia,euphorbiaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,6,0.1,-27.3,46.7,,,27.6828,,,27.6828,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
411,,mad501,cupiuba,goupeia spp.,goupeia,euphorbiaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,4.8,0.1,-26.8,48.1,,,27.164725,,,27.164725,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
412,,mad501,cupiuba,goupeia spp.,goupeia,euphorbiaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,4.5,0.1,-26.9,48.2,,,26.7835,,,26.7835,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
413,,mad501,cupiuba,goupeia spp.,goupeia,euphorbiaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,5.3,0.1,-27.5,48.2,,,26.089475,,,26.089475,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
414,,mad501,cupiuba,goupeia spp.,goupeia,euphorbiaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,5.4,0.1,-28,47.5,,,26.324075,,,26.324075,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
415,,mad501,cupiuba,goupeia spp.,goupeia,euphorbiaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,4.4,0.1,-28.9,48.1,,,26.4707,,,26.4707,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
416,,mad502,angelim_pedra,dinizia excelsa,dinizia,fabaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,3.5,0.1,-28.9,47.7,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
417,,mad502,angelim_pedra,dinizia excelsa,dinizia,fabaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,2.8,0.2,-28,47.4,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
418,,mad502,angelim_pedra,dinizia excelsa,dinizia,fabaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,3.2,0.2,-27.8,48,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
419,,mad502,angelim_pedra,dinizia excelsa,dinizia,fabaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,3.7,0.2,-27.6,48.5,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
420,,mad502,angelim_pedra,dinizia excelsa,dinizia,fabaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,3.7,0.2,-27.4,47.3,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
421,,mad503,angelim_pedra,dinizia excelsa,dinizia,fabaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,3.5,0.2,-27.2,47.9,,,27.379775,,,27.379775,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
422,,mad503,angelim_pedra,dinizia excelsa,dinizia,fabaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,3.8,0.1,-27.9,48,,,27.8392,,,27.8392,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
423,,mad503,angelim_pedra,dinizia excelsa,dinizia,fabaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,2.9,0.1,-27.9,47.7,,,27.848975,,,27.848975,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
424,,mad503,angelim_pedra,dinizia excelsa,dinizia,fabaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,3.3,0.2,-28.5,48.1,,,26.988775,,,26.988775,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
425,,mad503,angelim_pedra,dinizia excelsa,dinizia,fabaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,3.3,0.3,-28.1,46.7,,,29.452075,,,29.452075,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
426,,mad504,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,4.9,0.1,-28.3,46.8,,,25.346575,,,25.346575,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
427,,mad504,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,6.5,0.1,-26.6,47.3,,,26.304525,,,26.304525,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
428,,mad504,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,5.3,0.1,-25.7,47.1,,,27.0572,,,27.0572,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
429,,mad504,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,6.4,0.1,-26.9,47.3,,,26.089475,,,26.089475,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
430,,mad504,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,5.5,0.1,-27,47.4,,,28.474575,,,28.474575,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
431,,mad505,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,6.9,0.1,-26.7,46.2,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
432,,mad505,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,7,0.1,-27.1,45.4,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
433,,mad505,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,7.9,0.1,-27.3,45.4,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
434,,mad505,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,8,0.1,-27.9,47.7,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
435,,mad505,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,9.4,0.1,-29.4,47.7,,,,,,,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
436,,mad506,ipe,handroanthus spp.,handroanthus,bignoniaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,4.6,0.2,-27.9,48.5,,,26.37295,,,26.37295,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
437,,mad506,ipe,handroanthus spp.,handroanthus,bignoniaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,6.1,0.2,-27.6,49.2,,,27.2918,,,27.2918,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
438,,mad506,ipe,handroanthus spp.,handroanthus,bignoniaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,6.8,0.2,-27,48.8,,,28.650525,,,28.650525,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
439,,mad506,ipe,handroanthus spp.,handroanthus,bignoniaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,7,0.3,-26.4,49.1,,,28.17155,,,28.17155,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
440,,mad506,ipe,handroanthus spp.,handroanthus,bignoniaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,8.1,0.4,-26.6,46.9,,,28.806925,,,28.806925,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
441,,mad507,ipe,handroanthus spp.,handroanthus,bignoniaceae,0,rorainopolis,roraima,2022,-0.90361,-60.43406,4.2,0.2,-27.4,50.8,,,27.008325,,,27.008325,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
442,,mad507,ipe,handroanthus spp.,handroanthus,bignoniaceae,25,rorainopolis,roraima,2022,-0.90361,-60.43406,3.9,0.3,-27.5,49.6,,,28.152,,,28.152,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
443,,mad507,ipe,handroanthus spp.,handroanthus,bignoniaceae,50,rorainopolis,roraima,2022,-0.90361,-60.43406,4.9,0.3,-26.1,49.5,,,27.0963,,,27.0963,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
444,,mad507,ipe,handroanthus spp.,handroanthus,bignoniaceae,75,rorainopolis,roraima,2022,-0.90361,-60.43406,6,0.4,-26.9,49.9,,,26.930125,,,26.930125,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
445,,mad507,ipe,handroanthus spp.,handroanthus,bignoniaceae,100,rorainopolis,roraima,2022,-0.90361,-60.43406,6.3,0.4,-27.8,47.9,,,29.002425,,,29.002425,0.77583,0.78936,99.88333,84,1002.9223,26.87083,2217,-3.60859,-3.94084,-3.86175,-3.87313
446,,mad612,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.318,-60.978,6.1,0.1,-26,48.6,,,25.229275,,,25.229275,0.72583,0.79095,93.08334,151,994.9425,25.45833,2190,-4.23224,-4.11592,-4.64169,-4.69205
447,,mad612,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.318,-60.978,6.3,0.1,-26.5,47.9,,,24.4375,,,24.4375,0.72583,0.79095,93.08334,151,994.9425,25.45833,2190,-4.23224,-4.11592,-4.64169,-4.69205
448,,mad612,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.318,-60.978,4.9,0.1,-27,48.5,,,25.55185,,,25.55185,0.72583,0.79095,93.08334,151,994.9425,25.45833,2190,-4.23224,-4.11592,-4.64169,-4.69205
449,,mad612,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.318,-60.978,3.6,0.2,-26.6,47.4,,,23.430675,,,23.430675,0.72583,0.79095,93.08334,151,994.9425,25.45833,2190,-4.23224,-4.11592,-4.64169,-4.69205
450,,mad612,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.318,-60.978,4.6,0.3,-28,47.5,,,25.23905,,,25.23905,0.72583,0.79095,93.08334,151,994.9425,25.45833,2190,-4.23224,-4.11592,-4.64169,-4.69205
451,,mad613,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.312,-62.982,6.2,0.1,-26.3,47.6,,,25.307475,,,25.307475,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
452,,mad613,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.312,-62.982,5.9,0.1,-26.7,47.7,,,24.84805,,,24.84805,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
453,,mad613,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.312,-62.982,5,0.1,-26.6,48.4,,,25.463875,,,25.463875,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
454,,mad613,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.312,-62.982,5.2,0.2,-26.9,49,,,25.796225,,,25.796225,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
455,,mad613,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.312,-62.982,5,0.2,-27.4,46.9,,,26.343625,,,26.343625,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
456,,mad614,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.311,-62.974,6.2,0.1,-30.2,48.3,,,25.3759,,,25.3759,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
457,,mad614,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.311,-62.974,5.3,0.1,-28,47.4,,,25.1022,,,25.1022,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
458,,mad614,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.311,-62.974,6.2,0.1,-27.3,47.8,,,25.1804,,,25.1804,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
459,,mad614,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.311,-62.974,4.7,0.2,-27.9,47.4,,,25.04355,,,25.04355,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
460,,mad614,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.311,-62.974,4.1,0.2,-28.2,46.6,,,24.877375,,,24.877375,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
461,,mad615,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.317,-62.981,7.3,0.1,-29,46,,,24.26155,,,24.26155,0.63333,0.81416,91.68333,138,996.48682,25.375,2344,-4.4706,-4.41339,-4.90163,-4.92778
462,,mad615,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.317,-62.981,6.5,0.1,-27.6,47.1,,,30.019025,,,30.019025,0.63333,0.81416,91.68333,138,996.48682,25.375,2344,-4.4706,-4.41339,-4.90163,-4.92778
463,,mad615,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.317,-62.981,6.6,0.1,-26.3,47.4,,,25.1804,,,25.1804,0.63333,0.81416,91.68333,138,996.48682,25.375,2344,-4.4706,-4.41339,-4.90163,-4.92778
464,,mad615,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.317,-62.981,6.4,0.2,-27.3,46.6,,,25.483425,,,25.483425,0.63333,0.81416,91.68333,138,996.48682,25.375,2344,-4.4706,-4.41339,-4.90163,-4.92778
465,,mad615,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.317,-62.981,5.3,0.2,-27.9,45.3,,,24.994675,,,24.994675,0.63333,0.81416,91.68333,138,996.48682,25.375,2344,-4.4706,-4.41339,-4.90163,-4.92778
466,,mad616,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.299,-62.977,8.4,0.1,-28.4,49,,,23.32315,,,23.32315,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
467,,mad616,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.299,-62.977,6.6,0.1,-27.8,48.1,,,24.0465,,,24.0465,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
468,,mad616,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.299,-62.977,5.4,0.1,-27.2,48.9,,,24.349525,,,24.349525,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
469,,mad616,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.299,-62.977,6.4,0.2,-28,47.6,,,24.80895,,,24.80895,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
470,,mad616,roxinho,peltogyne paniculata benth.,peltogyne,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.299,-62.977,5.5,0.3,-28,46.8,,,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
471,ZAC 1235,mad83,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,5.1,0.1,-29.4,48,-27.35,43.54,,25.16,,25.16,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
472,ZAC 1236,mad83,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,3.3,0.1,-28.7,47.9,-27.32,43.75,,24.38,,24.38,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
473,ZAC 1237,mad83,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,3.1,0.1,-27.6,48,-26.23,43.26,,24.61,,24.61,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
474,ZAC 1238,mad83,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,4.8,0.1,-27.9,48.1,-26.53,43.9,,22.96,,22.96,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
475,ZAC 1239,mad83,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,4.1,0.1,-28.9,47.8,-27.57,43.75,,26.85,,26.85,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
476,ZAC 1240,mad84,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,3.8,0.1,-28.9,47.9,-27.04,43.52,,25.12,,25.12,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
477,ZAC 1241,mad84,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,4.4,0.1,-27.3,47,-26.57,44.61,,27.1,,27.1,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
478,ZAC 1242,mad84,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,4,0.1,-27,47.9,-26.74,44.38,,26.31,,26.31,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
479,ZAC 1243,mad84,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,3.3,0.1,-27.8,47.1,-27.03,41.56,,26.03,,26.03,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
480,ZAC 1244,mad84,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,4.2,0.1,-29.7,46.2,-28.68,40.35,,26.54,,26.54,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
481,ZAC 1245,mad85,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,4.5,0.1,-29.4,46.8,-29.03,43.31,,23.11,,23.11,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
482,ZAC 1246,mad85,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,4,0.1,-28.6,47,-28.15,44.14,,25.94,,25.94,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
483,ZAC 1247,mad85,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,4.2,0.1,-28.5,47.6,-27.91,43.01,,26.41,,26.41,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
484,ZAC 1248,mad85,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,4.7,0.2,-28.7,47.1,-28.32,42.12,,25.91,,25.91,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
485,ZAC 1249,mad85,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,5.4,0.2,-29.5,46.1,-28.97,41.16,,27.49,,27.49,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
486,ZAC 1250,mad86,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,4.3,0.1,-29.3,49.2,-28.46,44.27,,27.24,,27.24,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
487,ZAC 1251,mad86,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,5.5,0.1,-27.7,47.9,-27.29,44.43,,24.02,,24.02,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
488,ZAC 1252,mad86,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,4.3,0.2,-26.9,50.3,-26.83,44.86,,25.72,,25.72,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
489,ZAC 1253,mad86,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,4.2,0.1,-27.5,47.6,-27.19,44.77,,26.81,,26.81,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
490,ZAC 1254,mad86,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,4.4,0.1,-29.3,47.1,-28.32,43.15,,25.96,,25.96,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
491,ZAC 1255,mad87,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,3.2,0.1,-28.1,47.7,-27.38,45.12,,27.26,,27.26,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
492,ZAC 1256,mad87,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,3.6,0.1,-27.7,47.4,-27.31,43.39,,26.9,,26.9,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
493,ZAC 1257,mad87,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,3.2,0.1,-28,46.9,-27.34,43.7,,26.15,,26.15,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
494,ZAC 1258,mad87,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,5.1,0.1,-28.8,47.1,-27.55,43.65,,27.1,,27.1,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
495,ZAC 1259,mad87,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,5.3,0.2,-29.5,46.5,-28.2,40.48,,26.69,,26.69,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
496,ZAC 1260,mad88,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,2.6,0.1,-29.4,48.9,-29.06,44.51,,,,,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
497,ZAC 1261,mad88,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,2.7,0.1,-28.2,49,-27.51,43.11,,25.76,,25.76,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
498,ZAC 1262,mad88,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,3.1,0.1,-28.4,49,-27.39,42.72,,25.51,,25.51,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
499,ZAC 1263,mad88,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,2.9,0.1,-28.7,49.1,-27.79,43.84,,25.49,,25.49,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
500,ZAC 1264,mad88,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,3,0.2,-29.6,46.8,-28.73,42.47,,25.93,,25.93,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
501,ZAC 1265,mad89,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,3.4,0.1,-26.7,47.3,-26.69,44.69,,26.96,,26.96,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
502,ZAC 1266,mad89,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,4.3,0.1,-27,48.3,-26.72,44.83,,26.11,,26.11,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
503,ZAC 1267,mad89,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,5.2,0.1,-27.5,48.9,-27.23,44.44,,25,,25,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
504,ZAC 1268,mad89,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,4.7,0.2,-27.9,47.6,-27.2,43.54,,24.9,,24.9,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
505,ZAC 1269,mad89,jutai_mirim,Hymenaea spp.,hymenaea,fabaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,4,0.2,-29.8,47.9,-28.96,42.15,,26.53,,26.53,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
506,ZAC 1270,mad90,cumaru,Dipteryx spp.,dipteryx,fabaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,0,0.4,-31,48.3,-30.64,43.56,,25.79,,25.79,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
507,ZAC 1271,mad90,cumaru,Dipteryx spp.,dipteryx,fabaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,0.1,0.5,-29,48.6,-28.58,43.76,,24.21,,24.21,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
508,ZAC 1272,mad90,cumaru,Dipteryx spp.,dipteryx,fabaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,0.5,0.5,-28.9,49.3,-28.21,42.31,,25.51,,25.51,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
509,ZAC 1273,mad90,cumaru,Dipteryx spp.,dipteryx,fabaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,0.5,0.5,-28.4,49.5,-27.71,44.58,,25.44,,25.44,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
510,ZAC 1274,mad90,cumaru,Dipteryx spp.,dipteryx,fabaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,2.2,0.5,-29,47.7,-27.86,42.74,,24.14,,24.14,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
511,ZAC 1275,mad91,cumaru,Dipteryx spp.,dipteryx,fabaceae,0,flona_tapajos,para,2021,-3.3983972,-54.973982,-0.2,0.4,-28.5,48.2,-28.65,44.31,,25.7,,25.7,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
512,ZAC 1276,mad91,cumaru,Dipteryx spp.,dipteryx,fabaceae,25,flona_tapajos,para,2021,-3.3983972,-54.973982,1.1,0.4,-27,48.1,-26.49,43.13,,25.26,,25.26,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
513,ZAC 1277,mad91,cumaru,Dipteryx spp.,dipteryx,fabaceae,50,flona_tapajos,para,2021,-3.3983972,-54.973982,1.1,0.4,-27.9,48.2,-27.41,43.03,,24.55,,24.55,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
514,ZAC 1278,mad91,cumaru,Dipteryx spp.,dipteryx,fabaceae,75,flona_tapajos,para,2021,-3.3983972,-54.973982,1.6,0.4,-29.4,47.6,-28.56,42.95,,25.45,,25.45,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
515,ZAC 1279,mad91,cumaru,Dipteryx spp.,dipteryx,fabaceae,100,flona_tapajos,para,2021,-3.3983972,-54.973982,4.4,0.3,-30.4,47.4,-29.08,40.27,,24.84,,24.84,0.645,0.81744,97.93333,139,996.36792,26,1840,-3.30055,-4.49297,-3.56381,-3.42629
516,ZAC 1280,mad96,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,3.7,0.1,-26.6,50.9,-25.16,41.76,,24.82,,24.82,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
517,ZAC 1281,mad96,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,3,0.1,-26.3,50.4,-25.3,41.31,,26.7,,26.7,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
518,ZAC 1282,mad96,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,3.5,0.1,-26.9,47.7,-24.13,39.9,,27.05,,27.05,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
519,ZAC 1283,mad96,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,3.5,0.2,-25.8,51,-25.49,41.85,,27.27,,27.27,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
520,ZAC 1284,mad96,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,4.9,0.3,-28.3,48,-27.15,40.89,,26.87,,26.87,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
521,ZAC 1285,mad97,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,5.8,0.2,-26.8,50.7,-25.77,41.88,,27.12,,27.12,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
522,ZAC 1286,mad97,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,5.6,0.1,-26.1,50.5,-24.89,41.54,,26.51,,26.51,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
523,ZAC 1287,mad97,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,4.3,0.5,-25.5,50.1,-24.43,42.51,,27.47,,27.47,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
524,ZAC 1288,mad97,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,4.7,0.2,-25.5,49.2,-23.64,40.82,,27.48,,27.48,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
525,ZAC 1289,mad97,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,5.2,0.3,-26.6,47.4,-25.32,41.48,,26.9,,26.9,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
526,ZAC 1290,mad98,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,2.8,0.2,-26,48.2,-25.11,42.41,,26.25,,26.25,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
527,ZAC 1291,mad98,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,2.4,0.1,-25.5,48.7,-24.31,41.2,,26.68,,26.68,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
528,ZAC 1292,mad98,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,3.7,0.1,-26.1,48.7,-24.33,41.5,,26.06,,26.06,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
529,ZAC 1293,mad98,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,3.2,0.2,-25.7,49.9,-25.58,40.86,,26.19,,26.19,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
530,ZAC 1294,mad98,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,4.7,0.2,-28.1,46.8,-26.43,40.14,,25.47,,25.47,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
531,ZAC 1295,mad99,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,3.2,0.2,-28.6,48.9,-27.46,41.57,,27.46,,27.46,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
532,ZAC 1296,mad99,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,2.5,0.2,-26.2,48.3,-25.5,40.63,,27.78,,27.78,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
533,ZAC 1297,mad99,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,3.8,0.1,-25.2,47.8,-24.02,40.54,,26.54,,26.54,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
534,ZAC 1298,mad99,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,3.9,0.2,-25.7,47.7,-24.29,40.93,,26.25,,26.25,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
535,ZAC 1299,mad99,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,3.8,0.2,-26.2,47.5,-24.97,41.54,,26.89,,26.89,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
536,ZAC 1300,mad100,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,4.2,0.2,-26.5,45,-25.88,40.68,,26.29,,26.29,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
537,ZAC 1301,mad100,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,3.4,0.2,-25.6,48.8,-24.28,40.86,,27.74,,27.74,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
538,ZAC 1302,mad100,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,4.3,0.2,-27,49.3,-25.99,41.74,,26.27,,26.27,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
539,ZAC 1303,mad100,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,3.6,0.2,-27.8,50.6,-26.45,40.99,,26.52,,26.52,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
540,ZAC 1304,mad100,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,3.5,0.2,-27.5,48.7,-26.26,40.02,,27.49,,27.49,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
541,ZAC 1305,mad101,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,4.2,0.2,-26.8,51.2,-26.37,40.79,,26.31,,26.31,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
542,ZAC 1306,mad101,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,4.2,0.2,-27.9,51.4,-25.67,40.47,,26.62,,26.62,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
543,ZAC 1307,mad101,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,4.5,0.2,-27.7,53.1,-25.84,41.21,,27.54,,27.54,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
544,ZAC 1308,mad101,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,4.3,0.2,-28.2,50.4,-26.68,42.3,,25.9,,25.9,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
545,ZAC 1309,mad101,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,6.2,0.3,-28.4,49.6,-27.02,40.82,,25.08,,25.08,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
546,ZAC 1310,mad102,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,5.5,0.2,-26.7,49.9,-25.8,44.83,,27.31,,27.31,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
547,ZAC 1311,mad102,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,5.7,0.2,-26.3,50.2,-25.16,42.32,,27.05,,27.05,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
548,ZAC 1312,mad102,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,5.8,0.2,-27,49.6,-25.87,42.4,,27.46,,27.46,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
549,ZAC 1313,mad102,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,5.9,0.2,-27.2,49.2,-25.99,43.62,,26.63,,26.63,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
550,ZAC 1314,mad102,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,4.6,0.2,-27.7,47.5,-26.66,42.09,,25.97,,25.97,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
551,ZAC 1315,mad103,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,7,0.2,-27.2,48.9,-25.86,41.29,,26.81,,26.81,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
552,ZAC 1316,mad103,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,6.9,0.2,-27,47.1,-26.34,43.22,,27.38,,27.38,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
553,ZAC 1317,mad103,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,7.8,0.2,-27.7,48.7,-27.01,44.23,,26.77,,26.77,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
554,ZAC 1318,mad103,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,6.9,0.2,-27.9,48.5,-27.38,45.03,,26.5,,26.5,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
555,ZAC 1319,mad103,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,6.6,0.2,-27.1,47.2,-26.28,43.89,,25.22,,25.22,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
556,ZAC 1320,mad104,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,5.7,0.2,-26.5,49,-25.86,45.11,,25.85,,25.85,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
557,ZAC 1321,mad104,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,6,0.2,-26.2,49.5,-25.55,44.13,,25.37,,25.37,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
558,ZAC 1322,mad104,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,5.9,0.2,-25.8,49,-25.05,43.9,,25.67,,25.67,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
559,ZAC 1323,mad104,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,5.8,0.2,-26.4,48.4,-25.49,42.75,,26.26,,26.26,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
560,ZAC 1324,mad104,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,6.7,0.2,-26.5,47,-25.69,44.73,,26.9,,26.9,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
561,ZAC 1325,mad105,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,6.9,0.2,-26,49.7,-24.59,41.96,,28.47,,28.47,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
562,ZAC 1326,mad105,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,7.3,0.2,-26.2,50.3,-25.05,41.33,,26.68,,26.68,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
563,ZAC 1327,mad105,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,7.1,0.2,-26.2,50.4,-25.04,41.23,,28.13,,28.13,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
564,ZAC 1328,mad105,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,6.5,0.2,-26.3,49.1,-24.97,39.7,,26.14,,26.14,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
565,ZAC 1329,mad105,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,7.5,0.3,-26.1,43,-24.99,40.37,,26.61,,26.61,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
566,ZAC 1330,mad106,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,santarem,para,2021,-3.9923,-54.908,4.2,0.2,-26.7,49.6,-25.68,38.26,,26.1,,26.1,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
567,ZAC 1331,mad106,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,santarem,para,2021,-3.9923,-54.908,3.2,0.2,-26.1,47,-25.6,39.76,,26.21,,26.21,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
568,ZAC 1332,mad106,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,santarem,para,2021,-3.9923,-54.908,3.5,0.2,-27.2,51,-24.76,40.6,,27.37,,27.37,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
569,ZAC 1333,mad106,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,santarem,para,2021,-3.9923,-54.908,3.1,0.1,-26.8,46.1,-25.41,40.7,,26.84,,26.84,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
570,ZAC 1334,mad106,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,santarem,para,2021,-3.9923,-54.908,2.9,0.2,-28.5,45.5,-27.3,39.67,,27.29,,27.29,0.7075,0.80339,97.88333,108,1000.05792,26.29583,1897,-3.27639,-4.25936,-3.50834,-3.48101
571,ZAC 1335,mad589,breu_sucuruba_branco,trattinnickia burseraefolia mart.,trattinnickia,burseraceae,0,tanguro,mato_grosso,2022,-13.0813,-52.3771,1.9,0.1,-26.5,45.7,-24.53,39.69,,27.05,,27.05,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
572,ZAC 1336,mad589,breu_sucuruba_branco,trattinnickia burseraefolia mart.,trattinnickia,burseraceae,25,tanguro,mato_grosso,2022,-13.0813,-52.3771,1.3,0.1,-28,45.7,-25.99,40.99,,27.66,,27.66,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
573,ZAC 1337,mad589,breu_sucuruba_branco,trattinnickia burseraefolia mart.,trattinnickia,burseraceae,50,tanguro,mato_grosso,2022,-13.0813,-52.3771,1.2,0.1,-28,45.7,-26.08,38.43,,28.55,,28.55,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
574,ZAC 1338,mad589,breu_sucuruba_branco,trattinnickia burseraefolia mart.,trattinnickia,burseraceae,75,tanguro,mato_grosso,2022,-13.0813,-52.3771,1.5,0.1,-27.9,45.5,-26.03,38.45,,26.73,,26.73,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
575,ZAC 1339,mad589,breu_sucuruba_branco,trattinnickia burseraefolia mart.,trattinnickia,burseraceae,100,tanguro,mato_grosso,2022,-13.0813,-52.3771,2.2,0.2,-28.2,45.9,-26.28,38.93,,27.18,,27.18,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
576,ZAC 1340,mad590,breu_sucuruba,trattinnickia rhoifolia willd.,trattinnickia,burseraceae,0,tanguro,mato_grosso,2022,-13.079,-52.3864,1.9,0.2,-27,45.6,-25,38.79,,26.97,,26.97,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
577,ZAC 1341,mad590,breu_sucuruba,trattinnickia rhoifolia willd.,trattinnickia,burseraceae,25,tanguro,mato_grosso,2022,-13.079,-52.3864,1.5,0.2,-28.7,46.1,-26.84,40.09,,27.93,,27.93,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
578,ZAC 1342,mad590,breu_sucuruba,trattinnickia rhoifolia willd.,trattinnickia,burseraceae,50,tanguro,mato_grosso,2022,-13.079,-52.3864,2.2,0.2,-29.3,45.6,-27.54,39.99,,28.36,,28.36,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
579,ZAC 1343,mad590,breu_sucuruba,trattinnickia rhoifolia willd.,trattinnickia,burseraceae,75,tanguro,mato_grosso,2022,-13.079,-52.3864,3,0.2,-29.4,45.8,-27.6,38.53,,27.82,,27.82,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
580,ZAC 1344,mad590,breu_sucuruba,trattinnickia rhoifolia willd.,trattinnickia,burseraceae,100,tanguro,mato_grosso,2022,-13.079,-52.3864,5.5,0.5,-29.3,45.4,-27.53,37.29,,29.7,,29.7,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
581,ZAC 1345,mad591,tento,ormosia paraensis ducke.,ormosia,fabaceae,0,tanguro,mato_grosso,2022,-13.079,-52.3864,2.6,0.2,-25.6,48.3,-24.54,43.73,,26.5,,26.5,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
582,ZAC 1346,mad591,tento,ormosia paraensis ducke.,ormosia,fabaceae,25,tanguro,mato_grosso,2022,-13.079,-52.3864,2.4,0.2,-26.1,47.7,-24.81,40.24,,27.2,,27.2,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
583,ZAC 1347,mad591,tento,ormosia paraensis ducke.,ormosia,fabaceae,50,tanguro,mato_grosso,2022,-13.079,-52.3864,2.7,0.2,-26.4,47.4,-23.72,46.23,,26.76,,26.76,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
584,ZAC 1348,mad591,tento,ormosia paraensis ducke.,ormosia,fabaceae,75,tanguro,mato_grosso,2022,-13.079,-52.3864,2.8,0.2,-26.4,46.9,-25.23,46.43,,26.52,,26.52,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
585,ZAC 1349,mad591,tento,ormosia paraensis ducke.,ormosia,fabaceae,100,tanguro,mato_grosso,2022,-13.079,-52.3864,1.4,0.3,-27.7,46.4,-26.15,42.14,,27.2,,27.2,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
586,ZAC 1350,mad592,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,0,tanguro,mato_grosso,2022,-13.079,-52.3864,5,0.6,-29.4,46.4,-28.83,46.02,,28.88,,28.88,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
587,ZAC 1351,mad592,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,25,tanguro,mato_grosso,2022,-13.079,-52.3864,2.2,0.3,-28.8,46.9,-27.58,44.05,,28.8,,28.8,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
588,ZAC 1352,mad592,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,50,tanguro,mato_grosso,2022,-13.079,-52.3864,2.4,0.2,-28.8,47.1,-28.29,46.1,,28.79,,28.79,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
589,ZAC 1353,mad592,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,75,tanguro,mato_grosso,2022,-13.079,-52.3864,3.8,0.4,-29,46.5,-28.14,44.33,,27.26,,27.26,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
590,ZAC 1354,mad592,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,100,tanguro,mato_grosso,2022,-13.079,-52.3864,5.7,0.5,-29.3,46.4,-28.37,44.65,,27.17,,27.17,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
591,ZAC 1355,mad593,carapanauba,aspidosperma  excelsum benth.,aspidosperma,apocynaceae,0,tanguro,mato_grosso,2022,-13.079,-52.3864,-0.9,0.4,-24.9,44,-24.21,45.52,,27.01,,27.01,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
592,ZAC 1356,mad593,carapanauba,aspidosperma  excelsum benth.,aspidosperma,apocynaceae,25,tanguro,mato_grosso,2022,-13.079,-52.3864,-0.5,0.4,-24.8,47,-23.4,44.6,,27.22,,27.22,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
593,ZAC 1357,mad593,carapanauba,aspidosperma  excelsum benth.,aspidosperma,apocynaceae,50,tanguro,mato_grosso,2022,-13.079,-52.3864,0.6,0.3,-25.6,46.6,-24.11,45.19,,26.88,,26.88,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
594,ZAC 1358,mad593,carapanauba,aspidosperma  excelsum benth.,aspidosperma,apocynaceae,75,tanguro,mato_grosso,2022,-13.079,-52.3864,0.9,0.3,-25.8,45.9,-24.49,45.07,,26.61,,26.61,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
595,ZAC 1359,mad593,carapanauba,aspidosperma  excelsum benth.,aspidosperma,apocynaceae,100,tanguro,mato_grosso,2022,-13.079,-52.3864,3.6,0.4,-27.1,46.6,-26.13,41.91,,26.45,,26.45,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
596,ZAC 1360,mad594,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,0,tanguro,mato_grosso,2022,-13.079,-52.3864,1.3,0.2,-29.2,46.4,-28.32,38.6,,25.63,,25.63,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
597,ZAC 1361,mad594,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,25,tanguro,mato_grosso,2022,-13.079,-52.3864,0.6,0.2,-29.1,46.1,-28.54,39.47,,26.46,,26.46,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
598,ZAC 1362,mad594,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,50,tanguro,mato_grosso,2022,-13.079,-52.3864,-0.1,0.2,-28.6,46.6,-27.62,39.39,,26.52,,26.52,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
599,ZAC 1363,mad594,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,75,tanguro,mato_grosso,2022,-13.079,-52.3864,2.6,0.2,-28.6,46.7,-27.56,39.18,,27.27,,27.27,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
600,ZAC 1364,mad594,cuiarana,terminalia tetraphylla (aublet) howard,terminalia,combretaceae,100,tanguro,mato_grosso,2022,-13.079,-52.3864,4.2,0.2,-29.6,46.3,-28.76,40.93,,27.34,,27.34,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
601,ZAC 1365,mad595,miraoba_folha_grande,mouriri brachyanthera ducke,mouriri,melastomataceae,0,tanguro,mato_grosso,2022,-13.079,-52.3864,1,0.2,-28.4,46.9,-28.36,45.17,,26.99,,26.99,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
602,ZAC 1366,mad595,miraoba_folha_grande,mouriri brachyanthera ducke,mouriri,melastomataceae,25,tanguro,mato_grosso,2022,-13.079,-52.3864,2.1,0.2,-29.8,46.7,-29.21,42.69,,26.42,,26.42,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
603,ZAC 1367,mad595,miraoba_folha_grande,mouriri brachyanthera ducke,mouriri,melastomataceae,50,tanguro,mato_grosso,2022,-13.079,-52.3864,2.4,0.2,-27.8,45.4,-26.53,44.52,,26.54,,26.54,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
604,ZAC 1368,mad595,miraoba_folha_grande,mouriri brachyanthera ducke,mouriri,melastomataceae,75,tanguro,mato_grosso,2022,-13.079,-52.3864,1.7,0.2,-29.3,46.2,-28.42,46.07,,26.18,,26.18,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
605,ZAC 1369,mad595,miraoba_folha_grande,mouriri brachyanthera ducke,mouriri,melastomataceae,100,tanguro,mato_grosso,2022,-13.079,-52.3864,1.9,0.2,-29.9,47,-28.96,44.75,,27.73,,27.73,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
606,ZAC 1370,mad596,breu_sucurubinha,trattinnickia glaziovii swart.,trattinnickia,burseraceae,0,tanguro,mato_grosso,2022,-13.0813,-52.3771,1.2,0.1,-29.7,45.9,-28.34,45.01,26.7835,26.33,0.01693206638,26.33,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
607,ZAC 1371,mad596,breu_sucurubinha,trattinnickia glaziovii swart.,trattinnickia,burseraceae,25,tanguro,mato_grosso,2022,-13.0813,-52.3771,-0.2,0.1,-28.4,45.5,-26.82,43.16,28.6994,27.7,0.03482302766,27.7,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
608,ZAC 1372,mad596,breu_sucurubinha,trattinnickia glaziovii swart.,trattinnickia,burseraceae,50,tanguro,mato_grosso,2022,-13.0813,-52.3771,0.3,0.1,-28.2,45.7,-26.79,45.06,28.103125,27.54,0.02003780718,27.54,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
609,,mad596,breu_sucurubinha,trattinnickia glaziovii swart.,trattinnickia,burseraceae,75,tanguro,mato_grosso,2022,-13.0813,-52.3771,0.6,0.1,-28.1,45.4,-27.12,43.69,27.4482,,,27.4482,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
610,,mad596,breu_sucurubinha,trattinnickia glaziovii swart.,trattinnickia,burseraceae,100,tanguro,mato_grosso,2022,-13.0813,-52.3771,1.3,0.1,-27.3,45.3,-26.27,44.28,29.4032,,,29.4032,0.99167,0.71167,105.925,391,966.77917,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
611,,mad597,ucuu_da_terra_firme,virola sebifera aubl.,virola,myristicaceae,0,tanguro,mato_grosso,2022,-13.079,-52.3864,2.1,0.3,-26.9,46.9,-25.48,43.27,29.021975,,,29.021975,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
612,,mad597,ucuu_da_terra_firme,virola sebifera aubl.,virola,myristicaceae,25,tanguro,mato_grosso,2022,-13.079,-52.3864,3,0.2,-27.3,46,-26.02,46.94,26.617325,,,26.617325,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
613,,mad597,ucuu_da_terra_firme,virola sebifera aubl.,virola,myristicaceae,50,tanguro,mato_grosso,2022,-13.079,-52.3864,2.9,0.2,-27.8,44.3,-26.59,39.62,27.03765,,,27.03765,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
614,,mad597,ucuu_da_terra_firme,virola sebifera aubl.,virola,myristicaceae,75,tanguro,mato_grosso,2022,-13.079,-52.3864,2.5,0.2,-27.9,46.1,-26.74,40.71,27.379775,,,27.379775,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
615,,mad597,ucuu_da_terra_firme,virola sebifera aubl.,virola,myristicaceae,100,tanguro,mato_grosso,2022,-13.079,-52.3864,1.9,0.2,-28.4,45.7,-27.35,41.31,26.95945,,,26.95945,0.99167,0.71167,105.925,381,967.93964,25.06667,1593,-3.54571,-4.554,-3.71678,-4.11961
616,,mad617,cumaru,Dipteryx spp.,dipteryx,fabaceae,0,mosqueiro,para,Oct-22,-1.0888389,-48.373225,-0.6,0.1,-27.5,46.3,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
617,,mad617,cumaru,Dipteryx spp.,dipteryx,fabaceae,25,mosqueiro,para,Oct-22,-1.0888389,-48.373225,-1.5,0.1,-27.4,47.3,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
618,,mad617,cumaru,Dipteryx spp.,dipteryx,fabaceae,50,mosqueiro,para,Oct-22,-1.0888389,-48.373225,1,0.1,-28.7,46.9,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
619,,mad617,cumaru,Dipteryx spp.,dipteryx,fabaceae,75,mosqueiro,para,Oct-22,-1.0888389,-48.373225,-1.6,0.1,-29,48.3,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
620,,mad617,cumaru,Dipteryx spp.,dipteryx,fabaceae,100,mosqueiro,para,Oct-22,-1.0888389,-48.373225,0.1,0.2,-29.3,46.7,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
621,,mad618,pitaica,swartzia spp,swartzia,fabaceae,0,mosqueiro,para,Oct-22,-1.0888389,-48.373225,0.1,0.3,-26.8,46.3,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
622,,mad618,pitaica,swartzia spp,swartzia,fabaceae,25,mosqueiro,para,Oct-22,-1.0888389,-48.373225,0,0.3,-25.1,47,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
623,,mad618,pitaica,swartzia spp,swartzia,fabaceae,50,mosqueiro,para,Oct-22,-1.0888389,-48.373225,-0.2,0.4,-24.8,47.1,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
624,,mad618,pitaica,swartzia spp,swartzia,fabaceae,75,mosqueiro,para,Oct-22,-1.0888389,-48.373225,0,0.4,-26.1,47.2,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
625,,mad618,pitaica,swartzia spp,swartzia,fabaceae,100,mosqueiro,para,Oct-22,-1.0888389,-48.373225,-0.4,0.4,-26.3,47.5,,,,,,,0.72167,0.80116,111.73333,14,1011.3147,26.61667,2708,-1.9879,-3.41812,-1.93196,-2.26624475
626,,mad562,pequi,caryocar spp.,caryocar,caryocaraceae,0,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,2.2,0.1,-26,46.5,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
627,,mad562,pequi,caryocar spp.,caryocar,caryocaraceae,25,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,1.8,0.1,-26.6,46.1,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
628,,mad562,pequi,caryocar spp.,caryocar,caryocaraceae,50,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,3.2,0.2,-26.9,46.7,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
629,,mad562,pequi,caryocar spp.,caryocar,caryocaraceae,75,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,1.8,0.2,-27.1,46.6,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
630,,mad562,pequi,caryocar spp.,caryocar,caryocaraceae,100,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,2,0.3,-27.7,45.6,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
631,,mad567,canela,nectandra spp.,nectandra,lauraceae,0,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,0.7,0.3,-27.2,47.6,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
632,,mad567,canela,nectandra spp.,nectandra,lauraceae,25,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,0.6,0.3,-27.1,48.7,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
633,,mad567,canela,nectandra spp.,nectandra,lauraceae,50,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,2.2,0.3,-27.5,48.6,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
634,,mad567,canela,nectandra spp.,nectandra,lauraceae,75,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,3,0.3,-27.8,48.6,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
635,,mad567,canela,nectandra spp.,nectandra,lauraceae,100,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,1.7,0.3,-28.5,48.7,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
636,,mad575,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,0,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,-3,0.3,-29.1,47.9,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
637,,mad575,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,25,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,-2.8,0.3,-28.6,47.7,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
638,,mad575,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,50,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,-2.3,0.3,-28.9,48,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
639,,mad575,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,75,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,-2.7,0.3,-27.6,48.8,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
640,,mad575,louro_itauba,Mezilaurus itauba (Meisn.) Taub. ex Mez,mezilaurus,lauraceae,100,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,-2.2,0.4,-27.5,49.5,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
641,,mad571,sucupira,bowdichia spp,bowdichia ,fabaceae,0,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,0.1,0.4,-26.9,47.7,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
642,,mad571,sucupira,bowdichia spp,bowdichia ,fabaceae,25,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,0.5,0.4,-26.3,47.5,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
643,,mad571,sucupira,bowdichia spp,bowdichia ,fabaceae,50,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,0.5,0.4,-25.8,47.2,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
644,,mad571,sucupira,bowdichia spp,bowdichia ,fabaceae,75,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,0.4,0.4,-26.4,47.8,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
645,,mad571,sucupira,bowdichia spp,bowdichia ,fabaceae,100,vilhena_ceos,rondonia,2022,-13.230025,-60.267356,0,0.9,-26.7,49.2,,,,,,,0.86833,0.74468,105.55833,244,983.95142,25.01667,1853,-4.40876,-4.3663,-4.82933,-5.10264
646,,mad635,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,0,flona_do_jamari,rondonia,May-19,-9.172,-63.008,4.3,0.3,-27.1,49.1,-25.65,38.69,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
647,,mad635,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,25,flona_do_jamari,rondonia,May-19,-9.172,-63.008,6,0.3,-26.8,48.9,-25.39,38.16,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
648,,mad635,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,50,flona_do_jamari,rondonia,May-19,-9.172,-63.008,6.6,0.3,-27.9,49.5,-26.17,39.24,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
649,,mad635,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,75,flona_do_jamari,rondonia,May-19,-9.172,-63.008,6.3,0.3,-26.9,49.7,-26.42,39.03,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
650,,mad635,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,100,flona_do_jamari,rondonia,May-19,-9.172,-63.008,6.4,0.4,-27.3,49.5,-27.48,38.15,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
651,,mad636,embireira,couratari stellata A. C. Sm.,couratari,lecythidaceae,0,flona_do_jamari,rondonia,May-19,-9.294,-62.981,4.8,0.7,-28.5,47.9,-27.07,37.34,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
652,,mad636,embireira,couratari stellata A. C. Sm.,couratari,lecythidaceae,25,flona_do_jamari,rondonia,May-19,-9.294,-62.981,6.5,0.4,-26.7,47.3,-25.02,38.29,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
653,,mad636,embireira,couratari stellata A. C. Sm.,couratari,lecythidaceae,50,flona_do_jamari,rondonia,May-19,-9.294,-62.981,8.1,0.3,-25.4,47.1,-24.55,38.87,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
654,,mad636,embireira,couratari stellata A. C. Sm.,couratari,lecythidaceae,75,flona_do_jamari,rondonia,May-19,-9.294,-62.981,7.7,0.3,-26,47.4,-25.46,39.09,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
655,,mad636,embireira,couratari stellata A. C. Sm.,couratari,lecythidaceae,100,flona_do_jamari,rondonia,May-19,-9.294,-62.981,5.1,0.3,-27.9,47.8,-26.43,39.97,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
656,,mad637,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,0,flona_do_jamari,rondonia,May-19,-9.295,-62.981,5.8,0.2,-28.7,49.8,-27.69,40.3,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
657,,mad637,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,25,flona_do_jamari,rondonia,May-19,-9.295,-62.981,5.4,0.2,-27.9,50.6,-26.84,43.68,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
658,,mad637,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,50,flona_do_jamari,rondonia,May-19,-9.295,-62.981,5.3,0.2,-27.7,48.8,-26.62,40.91,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
659,,mad637,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,75,flona_do_jamari,rondonia,May-19,-9.295,-62.981,7.3,0.2,-28.4,47.2,-27.29,41.66,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
660,,mad637,muirapiranga,Brosimum rubescens Taubert,brosimum,moraceae,100,flona_do_jamari,rondonia,May-19,-9.295,-62.981,7.4,0.2,-29.1,47.2,-27,40.37,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
661,,mad638,abiu,pouteria guianensis Aubl.,pouteria,Sapotaceae,0,flona_do_jamari,rondonia,May-19,-9.296,-62.979,4,0.9,-26.9,45.5,-26.1,38.67,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
662,,mad638,abiu,pouteria guianensis Aubl.,pouteria,Sapotaceae,25,flona_do_jamari,rondonia,May-19,-9.296,-62.979,4.9,0.6,-25.5,45.8,-24.56,40.01,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
663,,mad638,abiu,pouteria guianensis Aubl.,pouteria,Sapotaceae,50,flona_do_jamari,rondonia,May-19,-9.296,-62.979,4.5,0.5,-26.9,45.7,-26.2,40.84,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
664,,mad638,abiu,pouteria guianensis Aubl.,pouteria,Sapotaceae,75,flona_do_jamari,rondonia,May-19,-9.296,-62.979,4.9,0.4,-26.7,43.6,-25.49,40.53,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
665,,mad638,abiu,pouteria guianensis Aubl.,pouteria,Sapotaceae,100,flona_do_jamari,rondonia,May-19,-9.296,-62.979,5.3,0.5,-27.8,46.1,-26.08,40.06,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
666,,mad639,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.298,-62.981,6.5,0.1,-29.1,47.1,-28.21,40.02,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
667,,mad639,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.298,-62.981,6.9,0.1,-27.2,46.9,-26.45,40.47,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
668,,mad639,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.298,-62.981,8.7,0.1,-27.8,47.6,-26.62,40.59,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
669,,mad639,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.298,-62.981,7.5,0.1,-27.9,46.8,-26.79,41.84,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
670,,mad639,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.298,-62.981,8.2,0.1,-28.6,46.7,-27.26,40.98,,,,,0.63333,0.81416,91.68333,127,997.79504,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
671,,mad640,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.166,-63.014,5.5,0.2,-26.6,48.6,-25.39,40.45,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
672,,mad640,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.166,-63.014,6.8,0.1,-27.1,48.4,-25.9,40.03,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
673,,mad640,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.166,-63.014,5.4,0.1,-27.2,48.4,-25.84,40.98,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
674,,mad640,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.166,-63.014,5.4,0.1,-27.9,47.4,-26.52,39.93,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
675,,mad640,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.166,-63.014,6.6,0.2,-28.7,46.9,-27.11,43.42,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
676,,mad641,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.311,-62.981,4.5,0.1,-28,46.9,-26.84,42.51,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
677,,mad641,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.311,-62.981,7.2,0.1,-27.4,46.6,-26.25,40.17,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
678,,mad641,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.311,-62.981,6.5,0.1,-27.5,47.2,-26.19,40.82,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
679,,mad641,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.311,-62.981,5.8,0.1,-28.6,47,-27.1,40.97,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
680,,mad641,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.311,-62.981,4.8,0.2,-29.5,46.5,-27.86,43.72,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
681,,mad642,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.163,-63.011,6.6,0.1,-27.3,48.2,-26.21,45.69,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
682,,mad642,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.163,-63.011,6.4,0.1,-26.8,48.2,-25.75,42.94,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
683,,mad642,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.163,-63.011,5.7,0.2,-27.2,48.6,-26.11,41.82,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
684,,mad642,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.163,-63.011,6,0.2,-27.6,47.5,-26.26,41.19,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
685,,mad642,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.163,-63.011,6.7,0.3,-28.4,45.7,-27.17,42.62,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
686,,mad643,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.298,-62.992,4,0.1,-27.9,46.9,-27.04,41.64,,,,,0.63333,0.81416,91.68333,119,998.74731,25.375,2344,-4.45845,-4.39838,-4.90163,-4.92778
687,,mad643,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.298,-62.992,4.2,0.1,-27.8,47.2,-27.32,40.18,,,,,0.63333,0.81416,91.68333,119,998.74731,25.375,2344,-4.45845,-4.39838,-4.90163,-4.92778
688,,mad643,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.298,-62.992,4.4,0.1,-27.2,47.6,-25.95,40.94,,,,,0.63333,0.81416,91.68333,119,998.74731,25.375,2344,-4.45845,-4.39838,-4.90163,-4.92778
689,,mad643,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.298,-62.992,3.7,0.1,-28.3,47,-26.72,41.06,,,,,0.63333,0.81416,91.68333,119,998.74731,25.375,2344,-4.45845,-4.39838,-4.90163,-4.92778
690,,mad643,jatoba,Hymenaea courbaril L.,hymenaea,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.298,-62.992,3.1,0.1,-29.3,46.1,-27.91,41.7,,,,,0.63333,0.81416,91.68333,119,998.74731,25.375,2344,-4.45845,-4.39838,-4.90163,-4.92778
691,,mad644,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.173,-63.012,4.3,0.2,-27.7,46.9,-26.51,41.04,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
692,,mad644,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.173,-63.012,5.7,0.2,-28.3,47.7,-27.14,40.72,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
693,,mad644,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.173,-63.012,4.9,0.2,-28.7,47.6,-27.4,41.82,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
694,,mad644,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.173,-63.012,4.6,0.2,-28.6,48,-27.77,41.23,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
695,,mad644,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.173,-63.012,4.7,0.3,-29.3,47,-28.21,40.36,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
696,,mad645,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.167,-63.013,5.3,0.2,-27.5,46.5,-26.25,40.11,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
697,,mad645,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.167,-63.013,4.4,0.2,-27.2,47.2,-25.86,45.23,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
698,,mad645,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.167,-63.013,4.7,0.2,-27.8,47.8,-26.27,45.14,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
699,,mad645,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.167,-63.013,4.2,0.2,-28,47.3,-26.56,40.39,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
700,,mad645,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.167,-63.013,3.8,0.3,-29.1,46.4,-27.8,39.93,,,,,0.6275,0.81639,91.49167,113,999.46204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
701,,mad646,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.166,-63.01,6.5,0.2,-27,47.1,-28.35,40.4,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
702,,mad646,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.166,-63.01,7.2,0.2,-27.3,47.5,-26.73,40.85,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
703,,mad646,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.166,-63.01,5.7,0.2,-27.7,47.6,-26.85,40.32,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
704,,mad646,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.166,-63.01,4.9,0.1,-28.2,47.1,-26.28,42.39,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
705,,mad646,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.166,-63.01,5.8,0.2,-29.2,47.3,-25.78,40.9,,,,,0.62667,0.8168,91.45834,112,999.58118,25.4625,2386,-4.44897,-4.42081,-4.89532,-4.92425
706,,mad647,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.167,-63.008,8.1,0.3,-26.4,41.1,-26.58,40.36,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
707,,mad647,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.167,-63.008,5.8,0.1,-27.7,47.7,-26.44,40.76,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
708,,mad647,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.167,-63.008,5.3,0.2,-27.7,48.4,-26.88,40.81,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
709,,mad647,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.167,-63.008,4.7,0.2,-28.2,48,-26.77,44.81,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
710,,mad647,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.167,-63.008,3.3,0.2,-29.2,47.6,-27.57,45.7,,,,,0.6275,0.81639,91.49167,124,998.15204,25.44167,2375,-4.44897,-4.42081,-4.89532,-4.94142
711,,mad648,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.17,-63.017,4.7,0.2,-27.7,48.1,-28.53,41.13,,,,,0.6275,0.81639,91.49167,111,999.70038,25.44167,2375,-4.45616,-4.42081,-4.89532,-4.94142
712,,mad648,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.17,-63.017,5.2,0.2,-27.7,47.9,-27.08,40,,,,,0.6275,0.81639,91.49167,111,999.70038,25.44167,2375,-4.45616,-4.42081,-4.89532,-4.94142
713,,mad648,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.17,-63.017,4,0.2,-28,48.5,-27.04,40.28,,,,,0.6275,0.81639,91.49167,111,999.70038,25.44167,2375,-4.45616,-4.42081,-4.89532,-4.94142
714,,mad648,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.17,-63.017,5.1,0.2,-28.2,48.6,-26.59,40.34,,,,,0.6275,0.81639,91.49167,111,999.70038,25.44167,2375,-4.45616,-4.42081,-4.89532,-4.94142
715,,mad648,cedromara,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.17,-63.017,3.5,0.2,-29.9,48,-26.41,40.42,,,,,0.6275,0.81639,91.49167,111,999.70038,25.44167,2375,-4.45616,-4.42081,-4.89532,-4.94142
716,,mad649,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.313,-62.974,4.2,0.1,-31.1,47.8,-29.14,41.91,,,,,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
717,,mad649,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.313,-62.974,5.8,0.1,-29.3,47.6,-27.81,39.98,,,,,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
718,,mad649,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.313,-62.974,4.3,0.1,-29.2,48.3,-27.49,40.53,,,,,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
719,,mad649,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.313,-62.974,3.2,0.1,-30.3,48.6,-28.19,39.53,,,,,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
720,,mad649,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.313,-62.974,5.7,0.2,-29.5,47.4,-27.78,41.79,,,,,0.63333,0.81416,91.68333,124,998.15204,25.375,2344,-4.45328,-4.37377,-4.90163,-4.92778
721,,mad650,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.315,-62.98,4.6,0.1,-29.6,47.8,-28.09,45.76,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
722,,mad650,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.315,-62.98,4.6,0.1,-28.6,48,-26.88,40.47,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
723,,mad650,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.315,-62.98,4.2,0.1,-29,47.9,-27.33,40.22,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
724,,mad650,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.315,-62.98,5.3,0.1,-28.3,48.6,-26.49,40.26,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
725,,mad650,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.315,-62.98,6,0.1,-29.4,47,-27.94,39.95,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
726,,mad651,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.304,-63.031,3.9,0.1,-28.3,46.8,-27,39.87,,,,,0.63417,0.81413,91.8,124,998.15204,25.39167,2325,-4.46223,-4.39838,-4.908,-4.94142
727,,mad651,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.304,-63.031,3.9,0.1,-29.3,47.6,-27.47,40.59,,,,,0.63417,0.81413,91.8,124,998.15204,25.39167,2325,-4.46223,-4.39838,-4.908,-4.94142
728,,mad651,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.304,-63.031,4.1,0.1,-28.3,48.3,-26.75,44.85,,,,,0.63417,0.81413,91.8,124,998.15204,25.39167,2325,-4.46223,-4.39838,-4.908,-4.94142
729,,mad651,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.304,-63.031,7.7,0.1,-29.5,48.2,-27.62,41.01,,,,,0.63417,0.81413,91.8,124,998.15204,25.39167,2325,-4.46223,-4.39838,-4.908,-4.94142
730,,mad651,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.304,-63.031,5.8,0.2,-29.2,46.9,-27.88,40.31,,,,,0.63417,0.81413,91.8,124,998.15204,25.39167,2325,-4.46223,-4.39838,-4.908,-4.94142
731,,mad652,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.309,-62.978,3.6,0.1,-29.3,47,-28.01,40.91,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
732,,mad652,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.309,-62.978,6.9,0.1,-28.5,47.9,-27,40.1,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
733,,mad652,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.309,-62.978,3.9,0.1,-27.7,47.4,-26.33,40.97,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
734,,mad652,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.309,-62.978,5.7,0.1,-28.2,48.5,-26.65,46.38,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
735,,mad652,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.309,-62.978,5.6,0.1,-28.9,47.7,-27.56,40.08,,,,,0.63333,0.81416,91.68333,129,997.55707,25.375,2344,-4.45845,-4.37377,-4.90163,-4.92778
736,,mad653,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,0,flona_do_jamari,rondonia,May-19,-9.306,-63.002,2.9,0.1,-28.2,46.9,-26.9,40.65,,,,,0.63417,0.81413,91.8,120,998.62823,25.39167,2325,-4.45845,-4.39838,-4.90163,-4.94142
737,,mad653,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,25,flona_do_jamari,rondonia,May-19,-9.306,-63.002,5.2,0.1,-28.2,48.4,-26.43,39.3,,,,,0.63417,0.81413,91.8,120,998.62823,25.39167,2325,-4.45845,-4.39838,-4.90163,-4.94142
738,,mad653,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,50,flona_do_jamari,rondonia,May-19,-9.306,-63.002,3.2,0.1,-28.2,48.3,-26.73,40.49,,,,,0.63417,0.81413,91.8,120,998.62823,25.39167,2325,-4.45845,-4.39838,-4.90163,-4.94142
739,,mad653,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,75,flona_do_jamari,rondonia,May-19,-9.306,-63.002,4.5,0.1,-28.4,48.4,-27.1,46.13,,,,,0.63417,0.81413,91.8,120,998.62823,25.39167,2325,-4.45845,-4.39838,-4.90163,-4.94142
740,,mad653,cedro_rosa,cedrelinga catenaeformis,cedrelinga,fabaceae,100,flona_do_jamari,rondonia,May-19,-9.306,-63.002,5.5,0.2,-29.5,47.7,-27.85,44.01,,,,,0.63417,0.81413,91.8,120,998.62823,25.39167,2325,-4.45845,-4.39838,-4.90163,-4.94142
741,,mad623,cupiuba,goupeia spp.,goupeia,euphorbiaceae,0,alta_floresta,mato_grosso,Nov-22,-9.365,-59.019,5.9,0.1,-28.1,46.4,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
742,,mad623,cupiuba,goupeia spp.,goupeia,euphorbiaceae,25,alta_floresta,mato_grosso,Nov-22,-9.365,-59.019,4.5,0.1,-27,47.1,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
743,,mad623,cupiuba,goupeia spp.,goupeia,euphorbiaceae,50,alta_floresta,mato_grosso,Nov-22,-9.365,-59.019,5,0.1,-26.2,48.8,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
744,,mad623,cupiuba,goupeia spp.,goupeia,euphorbiaceae,75,alta_floresta,mato_grosso,Nov-22,-9.365,-59.019,4.8,0.1,-27.2,47.4,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
745,,mad623,cupiuba,goupeia spp.,goupeia,euphorbiaceae,100,alta_floresta,mato_grosso,Nov-22,-9.365,-59.019,4.1,0.1,-28.2,47.2,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
746,,mad624,cupiuba,goupeia spp.,goupeia,euphorbiaceae,0,alta_floresta,mato_grosso,Nov-22,-9.365,-59.02,3.5,0.1,-28.4,47.3,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
747,,mad624,cupiuba,goupeia spp.,goupeia,euphorbiaceae,25,alta_floresta,mato_grosso,Nov-22,-9.365,-59.02,3.8,0.1,-27.7,47.2,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
748,,mad624,cupiuba,goupeia spp.,goupeia,euphorbiaceae,50,alta_floresta,mato_grosso,Nov-22,-9.365,-59.02,4.3,0.1,-28.2,47,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
749,,mad624,cupiuba,goupeia spp.,goupeia,euphorbiaceae,75,alta_floresta,mato_grosso,Nov-22,-9.365,-59.02,4.1,0.1,-28.4,47.2,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
750,,mad624,cupiuba,goupeia spp.,goupeia,euphorbiaceae,100,alta_floresta,mato_grosso,Nov-22,-9.365,-59.02,4.2,0.1,-29.8,46.2,,,,,,,0.88083,0.75595,96.81667,164,993.40021,25.98333,1957,-4.02318,-4.11534,-4.40177,-4.50394
751,,mad625,cupiuba,goupeia spp.,goupeia,euphorbiaceae,0,alta_floresta,mato_grosso,Nov-22,-9.375,-59.042,5.3,0.2,-26.1,47.3,,,,,,,0.88833,0.75516,97.1,165,993.28162,26.0875,1960,-4.02318,-4.11534,-4.40177,-4.50394
752,,mad625,cupiuba,goupeia spp.,goupeia,euphorbiaceae,25,alta_floresta,mato_grosso,Nov-22,-9.375,-59.042,4.8,0.1,-26.2,45.7,,,,,,,0.88833,0.75516,97.1,165,993.28162,26.0875,1960,-4.02318,-4.11534,-4.40177,-4.50394
753,,mad625,cupiuba,goupeia spp.,goupeia,euphorbiaceae,50,alta_floresta,mato_grosso,Nov-22,-9.375,-59.042,6.1,0.1,-26.2,46.7,,,,,,,0.88833,0.75516,97.1,165,993.28162,26.0875,1960,-4.02318,-4.11534,-4.40177,-4.50394
754,,mad625,cupiuba,goupeia spp.,goupeia,euphorbiaceae,75,alta_floresta,mato_grosso,Nov-22,-9.375,-59.042,5.2,0.1,-26.4,46.7,,,,,,,0.88833,0.75516,97.1,165,993.28162,26.0875,1960,-4.02318,-4.11534,-4.40177,-4.50394
755,,mad625,cupiuba,goupeia spp.,goupeia,euphorbiaceae,100,alta_floresta,mato_grosso,Nov-22,-9.375,-59.042,5.4,0.1,-27.5,46.1,,,,,,,0.88833,0.75516,97.1,165,993.28162,26.0875,1960,-4.02318,-4.11534,-4.40177,-4.50394
756,,mad626,cupiuba,goupeia spp.,goupeia,euphorbiaceae,0,alta_floresta,mato_grosso,Nov-22,-9.367,-59.144,2.9,0.1,-27.7,46.4,,,,,,,0.87667,0.75658,96.49167,198,989.37549,25.94167,1977,-4.05716,-4.20474,-4.41924,-4.50394
757,,mad626,cupiuba,goupeia spp.,goupeia,euphorbiaceae,25,alta_floresta,mato_grosso,Nov-22,-9.367,-59.144,4.1,0.1,-26.8,47,,,,,,,0.87667,0.75658,96.49167,198,989.37549,25.94167,1977,-4.05716,-4.20474,-4.41924,-4.50394
758,,mad626,cupiuba,goupeia spp.,goupeia,euphorbiaceae,50,alta_floresta,mato_grosso,Nov-22,-9.367,-59.144,4.6,0.1,-26.7,46.3,,,,,,,0.87667,0.75658,96.49167,198,989.37549,25.94167,1977,-4.05716,-4.20474,-4.41924,-4.50394
759,,mad626,cupiuba,goupeia spp.,goupeia,euphorbiaceae,75,alta_floresta,mato_grosso,Nov-22,-9.367,-59.144,4.7,0.1,-27.3,46.8,,,,,,,0.87667,0.75658,96.49167,198,989.37549,25.94167,1977,-4.05716,-4.20474,-4.41924,-4.50394
760,,mad626,cupiuba,goupeia spp.,goupeia,euphorbiaceae,100,alta_floresta,mato_grosso,Nov-22,-9.367,-59.144,6,0.1,-27.8,46.2,,,,,,,0.87667,0.75658,96.49167,198,989.37549,25.94167,1977,-4.05716,-4.20474,-4.41924,-4.50394
761,,mad627,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,alta_floresta,mato_grosso,Nov-22,-9.375,-59.04,0.9,0.1,-29.5,46.6,,,,,,,0.88917,0.75439,97.05,168,992.92603,26.02917,1961,-4.02318,-4.11534,-4.40177,-4.50394
762,,mad627,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,alta_floresta,mato_grosso,Nov-22,-9.375,-59.04,1.1,0.1,-26.9,47,,,,,,,0.88917,0.75439,97.05,168,992.92603,26.02917,1961,-4.02318,-4.11534,-4.40177,-4.50394
763,,mad627,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,alta_floresta,mato_grosso,Nov-22,-9.375,-59.04,2.7,0.1,-26.2,47.1,,,,,,,0.88917,0.75439,97.05,168,992.92603,26.02917,1961,-4.02318,-4.11534,-4.40177,-4.50394
764,,mad627,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,alta_floresta,mato_grosso,Nov-22,-9.375,-59.04,1.2,0.1,-27.5,46.8,,,,,,,0.88917,0.75439,97.05,168,992.92603,26.02917,1961,-4.02318,-4.11534,-4.40177,-4.50394
765,,mad627,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,alta_floresta,mato_grosso,Nov-22,-9.375,-59.04,2.6,0.1,-28.4,46.4,,,,,,,0.88917,0.75439,97.05,168,992.92603,26.02917,1961,-4.02318,-4.11534,-4.40177,-4.50394
766,,mad628,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,alta_floresta,mato_grosso,Nov-22,-9.355,-58.89,4.2,0.2,-26.8,46.1,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
767,,mad628,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,alta_floresta,mato_grosso,Nov-22,-9.355,-58.89,3.6,0.1,-26.6,46,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
768,,mad628,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,alta_floresta,mato_grosso,Nov-22,-9.355,-58.89,4.3,0.1,-26.5,45.9,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
769,,mad628,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,alta_floresta,mato_grosso,Nov-22,-9.355,-58.89,4.6,0.1,-27.8,47,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
770,,mad628,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,alta_floresta,mato_grosso,Nov-22,-9.355,-58.89,3.3,0.1,-28.8,46.1,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
771,,mad629,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,alta_floresta,mato_grosso,Nov-22,-9.371,-59.062,3.2,0.1,-26.4,46.4,,,,,,,0.88083,0.75633,96.75,155,994.46777,26.00833,1959,-4.0143,-4.11534,-4.40177,-4.50394
772,,mad629,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,alta_floresta,mato_grosso,Nov-22,-9.371,-59.062,2.7,0.1,-24.8,46.7,,,,,,,0.88083,0.75633,96.75,155,994.46777,26.00833,1959,-4.0143,-4.11534,-4.40177,-4.50394
773,,mad629,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,alta_floresta,mato_grosso,Nov-22,-9.371,-59.062,1.6,0.1,-25.9,46.7,,,,,,,0.88083,0.75633,96.75,155,994.46777,26.00833,1959,-4.0143,-4.11534,-4.40177,-4.50394
774,,mad629,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,alta_floresta,mato_grosso,Nov-22,-9.371,-59.062,3.4,0.1,-26.3,46.8,,,,,,,0.88083,0.75633,96.75,155,994.46777,26.00833,1959,-4.0143,-4.11534,-4.40177,-4.50394
775,,mad629,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,alta_floresta,mato_grosso,Nov-22,-9.371,-59.062,4.9,0.3,-27.2,47,,,,,,,0.88083,0.75633,96.75,155,994.46777,26.00833,1959,-4.0143,-4.11534,-4.40177,-4.50394
776,,mad630,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,alta_floresta,mato_grosso,Nov-22,-9.371,-58.76,1.3,0.4,-26.3,46.5,,,,,,,0.86667,0.75582,96.45,248,983.4809,25.65833,1933,-4.07572,-4.31636,-4.47391,-4.51994
777,,mad630,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,alta_floresta,mato_grosso,Nov-22,-9.371,-58.76,1.6,0.2,-26.1,46.9,,,,,,,0.86667,0.75582,96.45,248,983.4809,25.65833,1933,-4.07572,-4.31636,-4.47391,-4.51994
778,,mad630,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,alta_floresta,mato_grosso,Nov-22,-9.371,-58.76,1.7,0.2,-26.4,47.2,,,,,,,0.86667,0.75582,96.45,248,983.4809,25.65833,1933,-4.07572,-4.31636,-4.47391,-4.51994
779,,mad630,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,alta_floresta,mato_grosso,Nov-22,-9.371,-58.76,2.4,0.1,-27.5,46.4,,,,,,,0.86667,0.75582,96.45,248,983.4809,25.65833,1933,-4.07572,-4.31636,-4.47391,-4.51994
780,,mad630,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,alta_floresta,mato_grosso,Nov-22,-9.371,-58.76,2.5,0.1,-27.3,44.9,,,,,,,0.86667,0.75582,96.45,248,983.4809,25.65833,1933,-4.07572,-4.31636,-4.47391,-4.51994
781,,mad631,cedrinho,erisma spp.,erisma,vochysiaceae,0,alta_floresta,mato_grosso,Nov-22,-9.36,-59.116,8,0.3,-28.2,48.8,,,,,,,0.87833,0.7563,96.50833,203,988.78479,25.96667,1967,-4.04829,-4.20474,-4.41924,-4.50394
782,,mad631,cedrinho,erisma spp.,erisma,vochysiaceae,25,alta_floresta,mato_grosso,Nov-22,-9.36,-59.116,7.5,0.3,-28.3,48.9,,,,,,,0.87833,0.7563,96.50833,203,988.78479,25.96667,1967,-4.04829,-4.20474,-4.41924,-4.50394
783,,mad631,cedrinho,erisma spp.,erisma,vochysiaceae,50,alta_floresta,mato_grosso,Nov-22,-9.36,-59.116,7.2,0.2,-28.2,46.7,,,,,,,0.87833,0.7563,96.50833,203,988.78479,25.96667,1967,-4.04829,-4.20474,-4.41924,-4.50394
784,,mad631,cedrinho,erisma spp.,erisma,vochysiaceae,75,alta_floresta,mato_grosso,Nov-22,-9.36,-59.116,7.2,0.3,-28.2,45.8,,,,,,,0.87833,0.7563,96.50833,203,988.78479,25.96667,1967,-4.04829,-4.20474,-4.41924,-4.50394
785,,mad631,cedrinho,erisma spp.,erisma,vochysiaceae,100,alta_floresta,mato_grosso,Nov-22,-9.36,-59.116,9,0.2,-28.9,45.6,,,,,,,0.87833,0.7563,96.50833,203,988.78479,25.96667,1967,-4.04829,-4.20474,-4.41924,-4.50394
786,,mad632,cedrinho,erisma spp.,erisma,vochysiaceae,0,alta_floresta,mato_grosso,Nov-22,-9.366,-58.73,5.5,0.2,-28.7,46.2,,,,,,,0.87417,0.75493,96.725,249,983.36328,25.74167,1931,-4.07572,-4.2815,-4.43249,-4.51994
787,,mad632,cedrinho,erisma spp.,erisma,vochysiaceae,25,alta_floresta,mato_grosso,Nov-22,-9.366,-58.73,6.1,0.2,-28.8,46.8,,,,,,,0.87417,0.75493,96.725,249,983.36328,25.74167,1931,-4.07572,-4.2815,-4.43249,-4.51994
788,,mad632,cedrinho,erisma spp.,erisma,vochysiaceae,50,alta_floresta,mato_grosso,Nov-22,-9.366,-58.73,5.7,0.2,-27,46.5,,,,,,,0.87417,0.75493,96.725,249,983.36328,25.74167,1931,-4.07572,-4.2815,-4.43249,-4.51994
789,,mad632,cedrinho,erisma spp.,erisma,vochysiaceae,75,alta_floresta,mato_grosso,Nov-22,-9.366,-58.73,7,0.2,-29,46.9,,,,,,,0.87417,0.75493,96.725,249,983.36328,25.74167,1931,-4.07572,-4.2815,-4.43249,-4.51994
790,,mad632,cedrinho,erisma spp.,erisma,vochysiaceae,100,alta_floresta,mato_grosso,Nov-22,-9.366,-58.73,5.9,0.2,-28.5,46.2,,,,,,,0.87417,0.75493,96.725,249,983.36328,25.74167,1931,-4.07572,-4.2815,-4.43249,-4.51994
791,,mad633,cedrinho,erisma spp.,erisma,vochysiaceae,0,alta_floresta,mato_grosso,Nov-22,-9.354,-58.866,4.4,0.2,-29.7,46.6,,,,,,,0.85333,0.75793,96.18333,260,982.07043,25.5375,1944,-4.10954,-4.3343,-4.50665,-4.54157
792,,mad633,cedrinho,erisma spp.,erisma,vochysiaceae,25,alta_floresta,mato_grosso,Nov-22,-9.354,-58.866,4.6,0.1,-28.1,46.5,,,,,,,0.85333,0.75793,96.18333,260,982.07043,25.5375,1944,-4.10954,-4.3343,-4.50665,-4.54157
793,,mad633,cedrinho,erisma spp.,erisma,vochysiaceae,50,alta_floresta,mato_grosso,Nov-22,-9.354,-58.866,5.7,0.2,-29.1,45.4,,,,,,,0.85333,0.75793,96.18333,260,982.07043,25.5375,1944,-4.10954,-4.3343,-4.50665,-4.54157
794,,mad633,cedrinho,erisma spp.,erisma,vochysiaceae,75,alta_floresta,mato_grosso,Nov-22,-9.354,-58.866,6.1,0.2,-28,46.6,,,,,,,0.85333,0.75793,96.18333,260,982.07043,25.5375,1944,-4.10954,-4.3343,-4.50665,-4.54157
795,,mad633,cedrinho,erisma spp.,erisma,vochysiaceae,100,alta_floresta,mato_grosso,Nov-22,-9.354,-58.866,5.9,0.2,-28.9,47,,,,,,,0.85333,0.75793,96.18333,260,982.07043,25.5375,1944,-4.10954,-4.3343,-4.50665,-4.54157
796,,mad634,cedrinho,erisma spp.,erisma,vochysiaceae,0,alta_floresta,mato_grosso,Nov-22,-9.355,-58.885,4.8,0.2,-27.9,46.4,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
797,,mad634,cedrinho,erisma spp.,erisma,vochysiaceae,25,alta_floresta,mato_grosso,Nov-22,-9.355,-58.885,3,0.2,-27.7,45.3,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
798,,mad634,cedrinho,erisma spp.,erisma,vochysiaceae,50,alta_floresta,mato_grosso,Nov-22,-9.355,-58.885,4.6,0.2,-27.7,45.7,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
799,,mad634,cedrinho,erisma spp.,erisma,vochysiaceae,75,alta_floresta,mato_grosso,Nov-22,-9.355,-58.885,4.3,0.3,-28.1,45.6,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
800,,mad634,cedrinho,erisma spp.,erisma,vochysiaceae,100,alta_floresta,mato_grosso,Nov-22,-9.355,-58.885,4.9,0.3,-29.2,46.2,,,,,,,0.86917,0.75558,96.575,270,980.8963,25.6875,1958,-4.10954,-4.3343,-4.50665,-4.54157
801,,mad654,cerejeira,amburana spp,amburana,fabaceae,0,sena_madureira,acre,Sep-22,-8.632,-70.134,0.3,0.1,-27.8,48.2,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
802,,mad654,cerejeira,amburana spp,amburana,fabaceae,25,sena_madureira,acre,Sep-22,-8.632,-70.134,1.2,0.1,-25.8,47.5,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
803,,mad654,cerejeira,amburana spp,amburana,fabaceae,50,sena_madureira,acre,Sep-22,-8.632,-70.134,0.3,0.1,-26.5,48.8,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
804,,mad654,cerejeira,amburana spp,amburana,fabaceae,75,sena_madureira,acre,Sep-22,-8.632,-70.134,-0.5,0.1,-26.5,48.2,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
805,,mad654,cerejeira,amburana spp,amburana,fabaceae,100,sena_madureira,acre,Sep-22,-8.632,-70.134,0.9,0.2,-27.3,45.2,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
806,,mad655,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,0,sena_madureira,acre,Jul-22,-8.641,-70.149,1.1,0.1,-28.2,47.1,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
807,,mad655,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,25,sena_madureira,acre,Jul-22,-8.641,-70.149,1.1,0.1,-28.2,46,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
808,,mad655,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,50,sena_madureira,acre,Jul-22,-8.641,-70.149,1.9,0.1,-28.5,46.9,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
809,,mad655,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,75,sena_madureira,acre,Jul-22,-8.641,-70.149,1.4,0.1,-29.1,48.5,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
810,,mad655,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,100,sena_madureira,acre,Jul-22,-8.641,-70.149,1.3,0.2,-31.1,46.8,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
811,,mad656,cerejeira,amburana spp,amburana,fabaceae,0,sena_madureira,acre,Sep-22,-8.638,-70.141,0.8,0.1,-30.1,48.4,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
812,,mad656,cerejeira,amburana spp,amburana,fabaceae,25,sena_madureira,acre,Sep-22,-8.638,-70.141,0.9,0.1,-27.5,49.3,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
813,,mad656,cerejeira,amburana spp,amburana,fabaceae,50,sena_madureira,acre,Sep-22,-8.638,-70.141,0,0.1,-27.4,49.3,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
814,,mad656,cerejeira,amburana spp,amburana,fabaceae,75,sena_madureira,acre,Sep-22,-8.638,-70.141,0.7,0.1,-28.2,45.8,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
815,,mad656,cerejeira,amburana spp,amburana,fabaceae,100,sena_madureira,acre,Sep-22,-8.638,-70.141,1.3,0.3,-28.2,44.7,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
816,,mad657,cerejeira,amburana spp,amburana,fabaceae,0,sena_madureira,acre,Oct/2022,-8.651,-70.137,0.7,0.1,-27.6,46.7,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
817,,mad657,cerejeira,amburana spp,amburana,fabaceae,25,sena_madureira,acre,Oct/2022,-8.651,-70.137,0.5,0.1,-26.6,46.5,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
818,,mad657,cerejeira,amburana spp,amburana,fabaceae,50,sena_madureira,acre,Oct/2022,-8.651,-70.137,0.1,0.1,-26,46.8,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
819,,mad657,cerejeira,amburana spp,amburana,fabaceae,75,sena_madureira,acre,Oct/2022,-8.651,-70.137,0.6,0.1,-26.5,48,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
820,,mad657,cerejeira,amburana spp,amburana,fabaceae,100,sena_madureira,acre,Oct/2022,-8.651,-70.137,-0.5,0.1,-26.6,48.7,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
821,,mad658,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,0,sena_madureira,acre,Sep-22,-8.646,-70.146,1.7,0.1,-29.8,47.8,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
822,,mad658,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,25,sena_madureira,acre,Sep-22,-8.646,-70.146,1.4,0.1,-29.5,47.7,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
823,,mad658,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,50,sena_madureira,acre,Sep-22,-8.646,-70.146,1.2,0.1,-30.3,47.7,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
824,,mad658,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,75,sena_madureira,acre,Sep-22,-8.646,-70.146,0,0.1,-29.3,46.6,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
825,,mad658,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,100,sena_madureira,acre,Sep-22,-8.646,-70.146,0.9,0.1,-30.6,46.6,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
826,,mad659,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,0,sena_madureira,acre,Jul-22,-8.641,-70.148,0.7,0.1,-29.3,47.2,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
827,,mad659,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,25,sena_madureira,acre,Jul-22,-8.641,-70.148,1.1,0.1,-29.9,47.8,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
828,,mad659,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,50,sena_madureira,acre,Jul-22,-8.641,-70.148,1.7,0.1,-29.7,48.6,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
829,,mad659,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,75,sena_madureira,acre,Jul-22,-8.641,-70.148,1,0.1,-30.4,48.2,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
830,,mad659,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,100,sena_madureira,acre,Jul-22,-8.641,-70.148,1.4,0.1,-30.4,47.2,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
831,,mad660,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,0,sena_madureira,acre,Sep-22,-8.633,-70.139,0.6,0.1,-29.7,46.8,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
832,,mad660,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,25,sena_madureira,acre,Sep-22,-8.633,-70.139,1.7,0.1,-29.8,46.8,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
833,,mad660,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,50,sena_madureira,acre,Sep-22,-8.633,-70.139,0.6,0.1,-29.8,46.4,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
834,,mad660,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,75,sena_madureira,acre,Sep-22,-8.633,-70.139,0.7,0.1,-30.5,46.8,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
835,,mad660,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,100,sena_madureira,acre,Sep-22,-8.633,-70.139,0.5,0.2,-30,46.6,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
836,,mad661,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,0,sena_madureira,acre,Sep-22,-8.634,-70.139,1.8,0.1,-28.8,47.4,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
837,,mad661,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,25,sena_madureira,acre,Sep-22,-8.634,-70.139,1,0.1,-29.7,46.5,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
838,,mad661,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,50,sena_madureira,acre,Sep-22,-8.634,-70.139,1.3,0.1,-28.2,47.9,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
839,,mad661,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,75,sena_madureira,acre,Sep-22,-8.634,-70.139,0.9,0.1,-28.5,48,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
840,,mad661,cedro_rosa,Cedrela fissilis,cedrela,meliaceae,100,sena_madureira,acre,Sep-22,-8.634,-70.139,0.3,0.1,-28.5,45.5,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
841,,mad662,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,sena_madureira,acre,Sep-22,-8.63,-70.147,1.7,0.1,-29.2,47,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
842,,mad662,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,sena_madureira,acre,Sep-22,-8.63,-70.147,1.2,0.1,-28.6,46.4,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
843,,mad662,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,sena_madureira,acre,Sep-22,-8.63,-70.147,2.4,0.1,-29,47.1,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
844,,mad662,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,sena_madureira,acre,Sep-22,-8.63,-70.147,1.6,0.1,-28.8,46.5,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
845,,mad662,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,sena_madureira,acre,Sep-22,-8.63,-70.147,1.3,0.1,-28.5,45.8,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
846,,mad663,cerejeira,amburana spp,amburana,fabaceae,0,sena_madureira,acre,Sep-22,-8.641,-70.142,4.1,0.1,-28.7,47.7,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
847,,mad663,cerejeira,amburana spp,amburana,fabaceae,25,sena_madureira,acre,Sep-22,-8.641,-70.142,1.7,0.1,-26.3,46.5,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
848,,mad663,cerejeira,amburana spp,amburana,fabaceae,50,sena_madureira,acre,Sep-22,-8.641,-70.142,0.5,0.1,-26.2,47.3,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
849,,mad663,cerejeira,amburana spp,amburana,fabaceae,75,sena_madureira,acre,Sep-22,-8.641,-70.142,0.5,0.1,-25.9,47.4,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
850,,mad663,cerejeira,amburana spp,amburana,fabaceae,100,sena_madureira,acre,Sep-22,-8.641,-70.142,-0.3,0.2,-27,45.5,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
851,,mad664,cerejeira,amburana spp,amburana,fabaceae,0,sena_madureira,acre,Sep-22,-8.657,-70.196,0.3,0.1,-26.5,47.9,,,,,,,0.605,0.81671,86.69167,222,986.54254,24.78333,2183,-4.73126,-4.26556,-5.53004,-5.93853
852,,mad664,cerejeira,amburana spp,amburana,fabaceae,25,sena_madureira,acre,Sep-22,-8.657,-70.196,0.1,0.1,-26.7,47.9,,,,,,,0.605,0.81671,86.69167,222,986.54254,24.78333,2183,-4.73126,-4.26556,-5.53004,-5.93853
853,,mad664,cerejeira,amburana spp,amburana,fabaceae,50,sena_madureira,acre,Sep-22,-8.657,-70.196,0.8,0.1,-27.8,48.1,,,,,,,0.605,0.81671,86.69167,222,986.54254,24.78333,2183,-4.73126,-4.26556,-5.53004,-5.93853
854,,mad664,cerejeira,amburana spp,amburana,fabaceae,75,sena_madureira,acre,Sep-22,-8.657,-70.196,1.2,0.1,-28.4,45.9,,,,,,,0.605,0.81671,86.69167,222,986.54254,24.78333,2183,-4.73126,-4.26556,-5.53004,-5.93853
855,,mad664,cerejeira,amburana spp,amburana,fabaceae,100,sena_madureira,acre,Sep-22,-8.657,-70.196,1.3,0.2,-28.2,45.5,,,,,,,0.605,0.81671,86.69167,222,986.54254,24.78333,2183,-4.73126,-4.26556,-5.53004,-5.93853
856,,mad665,cerejeira,amburana spp,amburana,fabaceae,0,sena_madureira,acre,Oct/2022,-8.646,-70.133,0.6,0.1,-29.3,48.4,,,,,,,0.6075,0.81643,86.63333,217,987.1322,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
857,,mad665,cerejeira,amburana spp,amburana,fabaceae,25,sena_madureira,acre,Oct/2022,-8.646,-70.133,2,0.1,-27.5,48,,,,,,,0.6075,0.81643,86.63333,217,987.1322,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
858,,mad665,cerejeira,amburana spp,amburana,fabaceae,50,sena_madureira,acre,Oct/2022,-8.646,-70.133,1.4,0.1,-27.4,48.7,,,,,,,0.6075,0.81643,86.63333,217,987.1322,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
859,,mad665,cerejeira,amburana spp,amburana,fabaceae,75,sena_madureira,acre,Oct/2022,-8.646,-70.133,0.8,0.1,-27.9,47.9,,,,,,,0.6075,0.81643,86.63333,217,987.1322,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
860,,mad665,cerejeira,amburana spp,amburana,fabaceae,100,sena_madureira,acre,Oct/2022,-8.646,-70.133,1.2,0.2,-28.1,45.2,,,,,,,0.6075,0.81643,86.63333,217,987.1322,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
861,,mad666,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,0,sena_madureira,acre,Jul-22,-8.632,-70.145,4.3,0.2,-27,49.8,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
862,,mad666,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,25,sena_madureira,acre,Jul-22,-8.632,-70.145,4.1,0.2,-26,49.6,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
863,,mad666,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,50,sena_madureira,acre,Jul-22,-8.632,-70.145,4.3,0.2,-25.2,47.9,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
864,,mad666,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,75,sena_madureira,acre,Jul-22,-8.632,-70.145,5.3,0.2,-25.3,48.7,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
865,,mad666,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,100,sena_madureira,acre,Jul-22,-8.632,-70.145,3.3,0.2,-26.2,45.7,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
866,,mad667,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,sena_madureira,acre,Oct/2022,-8.65,-70.15,3.7,0.3,-26.9,45.2,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
867,,mad667,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,sena_madureira,acre,Oct/2022,-8.65,-70.15,7.8,0.2,-25.5,46,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
868,,mad667,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,sena_madureira,acre,Oct/2022,-8.65,-70.15,3.8,0.2,-25.3,46.9,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
869,,mad667,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,sena_madureira,acre,Oct/2022,-8.65,-70.15,4.2,0.2,-26.2,46.3,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
870,,mad667,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,sena_madureira,acre,Oct/2022,-8.65,-70.15,3.5,0.3,-27.2,45.9,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
871,,mad668,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,0,sena_madureira,acre,Sep-22,-8.646,-70.145,5.2,0.2,-27.5,49.2,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
872,,mad668,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,25,sena_madureira,acre,Sep-22,-8.646,-70.145,5.4,0.2,-26.7,46.5,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
873,,mad668,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,50,sena_madureira,acre,Sep-22,-8.646,-70.145,4.7,0.2,-25.9,48.6,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
874,,mad668,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,75,sena_madureira,acre,Sep-22,-8.646,-70.145,4.3,0.2,-25.3,46.1,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
875,,mad668,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,100,sena_madureira,acre,Sep-22,-8.646,-70.145,4.4,0.3,-25.5,45.2,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
876,,mad669,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,0,sena_madureira,acre,Oct/2022,-8.654,-70.137,2.9,0.2,-26.6,47.6,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
877,,mad669,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,25,sena_madureira,acre,Oct/2022,-8.654,-70.137,2.8,0.2,-27,46.2,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
878,,mad669,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,50,sena_madureira,acre,Oct/2022,-8.654,-70.137,2.4,0.2,-26.5,47,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
879,,mad669,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,75,sena_madureira,acre,Oct/2022,-8.654,-70.137,2,0.2,-26.1,46.5,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
880,,mad669,ipe_roxo,handroanthus spp,handroanthus,bignoniaceae,100,sena_madureira,acre,Oct/2022,-8.654,-70.137,3.4,0.3,-27.6,46.2,,,,,,,0.6075,0.81643,86.63333,215,987.36816,24.79583,2191,-4.71461,-4.25633,-5.50565,-5.90316
881,,mad670,cabreuva,myroxylon balsamum,myroxylon,fabaceae,0,sena_madureira,acre,Sep-22,-8.631,-70.143,1.9,0.1,-29.3,46.2,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
882,,mad670,cabreuva,myroxylon balsamum,myroxylon,fabaceae,25,sena_madureira,acre,Sep-22,-8.631,-70.143,1.1,0.1,-28.1,47.2,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
883,,mad670,cabreuva,myroxylon balsamum,myroxylon,fabaceae,50,sena_madureira,acre,Sep-22,-8.631,-70.143,1.7,0.1,-28.3,47.7,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
884,,mad670,cabreuva,myroxylon balsamum,myroxylon,fabaceae,75,sena_madureira,acre,Sep-22,-8.631,-70.143,1.2,0.2,-28.2,46.5,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
885,,mad670,cabreuva,myroxylon balsamum,myroxylon,fabaceae,100,sena_madureira,acre,Sep-22,-8.631,-70.143,1.7,0.2,-28.1,45.7,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
886,,mad671,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,sena_madureira,acre,Sep-22,-8.626,-70.132,1.1,0.1,-27.7,47.6,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
887,,mad671,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,sena_madureira,acre,Sep-22,-8.626,-70.132,1.9,0.1,-27.9,46.1,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
888,,mad671,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,sena_madureira,acre,Sep-22,-8.626,-70.132,2,0.1,-26.9,46.1,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
889,,mad671,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,sena_madureira,acre,Sep-22,-8.626,-70.132,2.2,0.1,-28,46.3,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
890,,mad671,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,sena_madureira,acre,Sep-22,-8.626,-70.132,1.6,0.2,-28.5,45,,,,,,,0.6075,0.81643,86.63333,223,986.42462,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
891,,mad672,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,sena_madureira,acre,Sep-22,-8.651,-70.151,0.6,0.1,-27.8,46.3,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
892,,mad672,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,sena_madureira,acre,Sep-22,-8.651,-70.151,2.7,0.1,-27,47.2,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
893,,mad672,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,sena_madureira,acre,Sep-22,-8.651,-70.151,2,0.1,-27.1,47.8,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
894,,mad672,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,sena_madureira,acre,Sep-22,-8.651,-70.151,1.1,0.1,-27,47.5,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
895,,mad672,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,sena_madureira,acre,Sep-22,-8.651,-70.151,1.4,0.2,-28.1,45.2,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
896,,mad673,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,sena_madureira,acre,Sep-22,-8.658,-70.148,2.8,0.1,-27.3,48.6,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.7191,-4.25633,-5.50565,-5.90316
897,,mad673,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,sena_madureira,acre,Sep-22,-8.658,-70.148,2,0.1,-27.2,46.4,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.7191,-4.25633,-5.50565,-5.90316
898,,mad673,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,sena_madureira,acre,Sep-22,-8.658,-70.148,1.3,0.1,-26.6,47.4,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.7191,-4.25633,-5.50565,-5.90316
899,,mad673,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,sena_madureira,acre,Sep-22,-8.658,-70.148,1.1,0.1,-27,48.8,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.7191,-4.25633,-5.50565,-5.90316
900,,mad673,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,sena_madureira,acre,Sep-22,-8.658,-70.148,1.4,0.2,-28.5,46,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.7191,-4.25633,-5.50565,-5.90316
901,,mad674,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,sena_madureira,acre,Sep-22,-8.656,-70.153,4.6,0.2,-25.8,48.3,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
902,,mad674,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,sena_madureira,acre,Sep-22,-8.656,-70.153,5.3,0.1,-26.7,46.4,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
903,,mad674,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,sena_madureira,acre,Sep-22,-8.656,-70.153,2.5,0.1,-27,47.1,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
904,,mad674,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,sena_madureira,acre,Sep-22,-8.656,-70.153,5,0.2,-26.9,48.3,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
905,,mad674,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,sena_madureira,acre,Sep-22,-8.656,-70.153,4.2,0.1,-27.8,44.8,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
906,,mad675,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,sena_madureira,acre,Sep-22,-8.644,-70.143,1.6,0.1,-27.5,46.7,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
907,,mad675,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,sena_madureira,acre,Sep-22,-8.644,-70.143,1,0.1,-27.1,47.2,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
908,,mad675,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,sena_madureira,acre,Sep-22,-8.644,-70.143,1.6,0.1,-27.4,47.7,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
909,,mad675,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,sena_madureira,acre,Sep-22,-8.644,-70.143,0.6,0.1,-28.1,46.5,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
910,,mad675,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,sena_madureira,acre,Sep-22,-8.644,-70.143,1.5,0.2,-28.8,46,,,,,,,0.6075,0.81643,86.63333,216,987.25018,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
911,,mad676,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,0,sena_madureira,acre,Sep-22,-8.633,-70.142,3.6,0.3,-26.7,49.5,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
912,,mad676,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,25,sena_madureira,acre,Sep-22,-8.633,-70.142,3.9,0.3,-26.4,48.3,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
913,,mad676,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,50,sena_madureira,acre,Sep-22,-8.633,-70.142,3.6,0.4,-26.6,50,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
914,,mad676,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,75,sena_madureira,acre,Sep-22,-8.633,-70.142,3.9,0.4,-27.4,48.6,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
915,,mad676,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,100,sena_madureira,acre,Sep-22,-8.633,-70.142,4.8,0.4,-28.7,46.5,,,,,,,0.6075,0.81643,86.63333,219,986.8963,24.79583,2191,-4.73255,-4.29245,-5.50565,-5.90316
916,,mad677,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,0,sena_madureira,acre,Sep-22,-8.643,-70.14,1,0.1,-26.9,47.9,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
917,,mad677,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,25,sena_madureira,acre,Sep-22,-8.643,-70.14,0.9,0.1,-26.2,47.4,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
918,,mad677,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,50,sena_madureira,acre,Sep-22,-8.643,-70.14,1.1,0.1,-26.6,47.4,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
919,,mad677,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,75,sena_madureira,acre,Sep-22,-8.643,-70.14,1.6,0.1,-28.3,46.1,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
920,,mad677,cumaru_ferro,Dipteryx odorata (Aubl.) Willd.,dipteryx,fabaceae,100,sena_madureira,acre,Sep-22,-8.643,-70.14,1.6,0.2,-29.4,46.2,,,,,,,0.6075,0.81643,86.63333,222,986.54254,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
921,,mad678,cabreuva,myroxylon balsamum,myroxylon,fabaceae,0,sena_madureira,acre,Aug/2022,-8.674,-70.196,2.4,0.2,-27.7,46.7,,,,,,,0.61,0.81588,86.71667,212,987.72217,24.82917,2178,-4.73126,-4.26556,-5.53004,-5.9431
922,,mad678,cabreuva,myroxylon balsamum,myroxylon,fabaceae,25,sena_madureira,acre,Aug/2022,-8.674,-70.196,3,0.2,-28.1,48.6,,,,,,,0.61,0.81588,86.71667,212,987.72217,24.82917,2178,-4.73126,-4.26556,-5.53004,-5.9431
923,,mad678,cabreuva,myroxylon balsamum,myroxylon,fabaceae,50,sena_madureira,acre,Aug/2022,-8.674,-70.196,3.1,0.2,-27.3,48.2,,,,,,,0.61,0.81588,86.71667,212,987.72217,24.82917,2178,-4.73126,-4.26556,-5.53004,-5.9431
924,,mad678,cabreuva,myroxylon balsamum,myroxylon,fabaceae,75,sena_madureira,acre,Aug/2022,-8.674,-70.196,1,0.1,-27.2,47.5,,,,,,,0.61,0.81588,86.71667,212,987.72217,24.82917,2178,-4.73126,-4.26556,-5.53004,-5.9431
925,,mad678,cabreuva,myroxylon balsamum,myroxylon,fabaceae,100,sena_madureira,acre,Aug/2022,-8.674,-70.196,1.3,0.2,-28.1,46.4,,,,,,,0.61,0.81588,86.71667,212,987.72217,24.82917,2178,-4.73126,-4.26556,-5.53004,-5.9431
926,,mad679,cabreuva,myroxylon balsamum,myroxylon,fabaceae,0,sena_madureira,acre,Aug/2022,-8.651,-70.165,0.5,0.1,-28.5,47.8,,,,,,,0.6075,0.81643,86.63333,218,987.01422,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
927,,mad679,cabreuva,myroxylon balsamum,myroxylon,fabaceae,25,sena_madureira,acre,Aug/2022,-8.651,-70.165,1.1,0.2,-28.3,48.1,,,,,,,0.6075,0.81643,86.63333,218,987.01422,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
928,,mad679,cabreuva,myroxylon balsamum,myroxylon,fabaceae,50,sena_madureira,acre,Aug/2022,-8.651,-70.165,1,0.1,-29.2,47.2,,,,,,,0.6075,0.81643,86.63333,218,987.01422,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
929,,mad679,cabreuva,myroxylon balsamum,myroxylon,fabaceae,75,sena_madureira,acre,Aug/2022,-8.651,-70.165,0.9,0.2,-29,46.7,,,,,,,0.6075,0.81643,86.63333,218,987.01422,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
930,,mad679,cabreuva,myroxylon balsamum,myroxylon,fabaceae,100,sena_madureira,acre,Aug/2022,-8.651,-70.165,1.6,0.3,-28.5,46.5,,,,,,,0.6075,0.81643,86.63333,218,987.01422,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
931,,mad680,cabreuva,myroxylon balsamum,myroxylon,fabaceae,0,sena_madureira,acre,Sep-22,-8.646,-70.197,0.5,0.1,-28.6,47.7,,,,,,,0.605,0.81671,86.69167,228,985.83533,24.78333,2183,-4.7435,-4.30279,-5.53004,-5.93853
932,,mad680,cabreuva,myroxylon balsamum,myroxylon,fabaceae,25,sena_madureira,acre,Sep-22,-8.646,-70.197,-1.1,0.1,-28.4,47,,,,,,,0.605,0.81671,86.69167,228,985.83533,24.78333,2183,-4.7435,-4.30279,-5.53004,-5.93853
933,,mad680,cabreuva,myroxylon balsamum,myroxylon,fabaceae,50,sena_madureira,acre,Sep-22,-8.646,-70.197,-0.7,0.1,-28.4,48.3,,,,,,,0.605,0.81671,86.69167,228,985.83533,24.78333,2183,-4.7435,-4.30279,-5.53004,-5.93853
934,,mad680,cabreuva,myroxylon balsamum,myroxylon,fabaceae,75,sena_madureira,acre,Sep-22,-8.646,-70.197,-0.5,0.1,-28.5,46.1,,,,,,,0.605,0.81671,86.69167,228,985.83533,24.78333,2183,-4.7435,-4.30279,-5.53004,-5.93853
935,,mad680,cabreuva,myroxylon balsamum,myroxylon,fabaceae,100,sena_madureira,acre,Sep-22,-8.646,-70.197,-0.5,0.2,-28.7,45.5,,,,,,,0.605,0.81671,86.69167,228,985.83533,24.78333,2183,-4.7435,-4.30279,-5.53004,-5.93853
936,,mad681,cabreuva,myroxylon balsamum,myroxylon,fabaceae,0,sena_madureira,acre,Sep-22,-8.629,-70.139,0.4,0.1,-29.3,47.8,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
937,,mad681,cabreuva,myroxylon balsamum,myroxylon,fabaceae,25,sena_madureira,acre,Sep-22,-8.629,-70.139,-1.5,0.1,-28.1,47.8,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
938,,mad681,cabreuva,myroxylon balsamum,myroxylon,fabaceae,50,sena_madureira,acre,Sep-22,-8.629,-70.139,0.9,0.1,-27.5,47.9,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
939,,mad681,cabreuva,myroxylon balsamum,myroxylon,fabaceae,75,sena_madureira,acre,Sep-22,-8.629,-70.139,0.8,0.1,-27.8,48.1,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
940,,mad681,cabreuva,myroxylon balsamum,myroxylon,fabaceae,100,sena_madureira,acre,Sep-22,-8.629,-70.139,0.7,0.2,-28,46.4,,,,,,,0.6075,0.81643,86.63333,233,985.24628,24.79583,2191,-4.73232,-4.29245,-5.50565,-5.90316
941,,mad682,cabreuva,myroxylon balsamum,myroxylon,fabaceae,0,sena_madureira,acre,Sep-22,-8.652,-70.156,1.9,0.2,-27.5,46.2,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
942,,mad682,cabreuva,myroxylon balsamum,myroxylon,fabaceae,25,sena_madureira,acre,Sep-22,-8.652,-70.156,1,0.1,-26.8,46.1,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
943,,mad682,cabreuva,myroxylon balsamum,myroxylon,fabaceae,50,sena_madureira,acre,Sep-22,-8.652,-70.156,1.1,0.1,-26.8,47.9,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
944,,mad682,cabreuva,myroxylon balsamum,myroxylon,fabaceae,75,sena_madureira,acre,Sep-22,-8.652,-70.156,0.8,0.1,-26.7,46.9,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
945,,mad682,cabreuva,myroxylon balsamum,myroxylon,fabaceae,100,sena_madureira,acre,Sep-22,-8.652,-70.156,-0.4,0.2,-27.4,44.5,,,,,,,0.6075,0.81643,86.63333,209,988.07623,24.79583,2191,-4.7191,-4.26556,-5.50565,-5.90316
946,,mad683,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,0,sena_madureira,acre,Sep-22,-8.649,-70.158,3,0.5,-28.8,48.1,,,,,,,0.6075,0.81643,86.63333,212,987.72217,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
947,,mad683,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,25,sena_madureira,acre,Sep-22,-8.649,-70.158,3.1,0.4,-27,48.1,,,,,,,0.6075,0.81643,86.63333,212,987.72217,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
948,,mad683,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,50,sena_madureira,acre,Sep-22,-8.649,-70.158,2.9,0.4,-27.1,47.7,,,,,,,0.6075,0.81643,86.63333,212,987.72217,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
949,,mad683,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,75,sena_madureira,acre,Sep-22,-8.649,-70.158,3.6,0.5,-27.4,47.4,,,,,,,0.6075,0.81643,86.63333,212,987.72217,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
950,,mad683,ipe_amarelo,handroanthus spp,handroanthus,bignoniaceae,100,sena_madureira,acre,Sep-22,-8.649,-70.158,5.4,0.6,-28.5,46.3,,,,,,,0.6075,0.81643,86.63333,212,987.72217,24.79583,2191,-4.73255,-4.30279,-5.50565,-5.90316
951,,mad708,cedrinho,erisma lanceolatum,erisma,vochysiaceae,0,flona_jacunda,rondonia,2022,-8.617,-62.854,7.08,0.22,-27.75,46.51,,,,,,,0.61917,0.81976,91.08334,119,998.74731,25.5375,2300,-4.42063,-4.41403,-4.84743,-4.84901
952,,mad708,cedrinho,erisma lanceolatum,erisma,vochysiaceae,25,flona_jacunda,rondonia,2022,-8.617,-62.854,8.57,0.21,-28.12,45.97,,,,,,,0.61917,0.81976,91.08334,119,998.74731,25.5375,2300,-4.42063,-4.41403,-4.84743,-4.84901
953,,mad708,cedrinho,erisma lanceolatum,erisma,vochysiaceae,50,flona_jacunda,rondonia,2022,-8.617,-62.854,9.4,0.19,-27.69,46.38,,,,,,,0.61917,0.81976,91.08334,119,998.74731,25.5375,2300,-4.42063,-4.41403,-4.84743,-4.84901
954,,mad708,cedrinho,erisma lanceolatum,erisma,vochysiaceae,75,flona_jacunda,rondonia,2022,-8.617,-62.854,9.06,0.21,-27.65,46.29,,,,,,,0.61917,0.81976,91.08334,119,998.74731,25.5375,2300,-4.42063,-4.41403,-4.84743,-4.84901
955,,mad708,cedrinho,erisma lanceolatum,erisma,vochysiaceae,100,flona_jacunda,rondonia,2022,-8.617,-62.854,10.09,0.31,-29.18,45.01,,,,,,,0.61917,0.81976,91.08334,119,998.74731,25.5375,2300,-4.42063,-4.41403,-4.84743,-4.84901
956,,mad709,cedrinho,erisma lanceolatum,erisma,vochysiaceae,0,flona_jacunda,rondonia,2022,-8.627,-62.853,8.14,0.23,-27.04,46.31,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
957,,mad709,cedrinho,erisma lanceolatum,erisma,vochysiaceae,25,flona_jacunda,rondonia,2022,-8.627,-62.853,10.23,0.29,-27.7,45.8,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
958,,mad709,cedrinho,erisma lanceolatum,erisma,vochysiaceae,50,flona_jacunda,rondonia,2022,-8.627,-62.853,9.95,0.31,-27.73,45.63,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
959,,mad709,cedrinho,erisma lanceolatum,erisma,vochysiaceae,75,flona_jacunda,rondonia,2022,-8.627,-62.853,9.87,0.35,-27.55,46.28,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
960,,mad709,cedrinho,erisma lanceolatum,erisma,vochysiaceae,100,flona_jacunda,rondonia,2022,-8.627,-62.853,11.11,0.47,-29.09,47.19,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
961,,mad710,cedrinho,erisma lanceolatum,erisma,vochysiaceae,0,flona_jacunda,rondonia,2022,-8.625,-62.852,6.54,0.26,-26.22,46.36,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
962,,mad710,cedrinho,erisma lanceolatum,erisma,vochysiaceae,25,flona_jacunda,rondonia,2022,-8.625,-62.852,8.89,0.27,-26.74,47.71,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
963,,mad710,cedrinho,erisma lanceolatum,erisma,vochysiaceae,50,flona_jacunda,rondonia,2022,-8.625,-62.852,9.24,0.25,-26.6,47.31,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
964,,mad710,cedrinho,erisma lanceolatum,erisma,vochysiaceae,75,flona_jacunda,rondonia,2022,-8.625,-62.852,9.11,0.28,-26.51,46.33,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
965,,mad710,cedrinho,erisma lanceolatum,erisma,vochysiaceae,100,flona_jacunda,rondonia,2022,-8.625,-62.852,10.75,0.39,-28.47,47.28,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
966,,mad711,cedrinho,erisma lanceolatum,erisma,vochysiaceae,0,flona_jacunda,rondonia,2022,-8.617,-62.848,7.28,0.36,-28.14,45.89,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
967,,mad711,cedrinho,erisma lanceolatum,erisma,vochysiaceae,25,flona_jacunda,rondonia,2022,-8.617,-62.848,9.74,0.22,-27.56,46.74,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
968,,mad711,cedrinho,erisma lanceolatum,erisma,vochysiaceae,50,flona_jacunda,rondonia,2022,-8.617,-62.848,10.06,0.23,-27.85,46.95,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
969,,mad711,cedrinho,erisma lanceolatum,erisma,vochysiaceae,75,flona_jacunda,rondonia,2022,-8.617,-62.848,10.58,0.24,-28.87,46.76,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
970,,mad711,cedrinho,erisma lanceolatum,erisma,vochysiaceae,100,flona_jacunda,rondonia,2022,-8.617,-62.848,10.87,0.25,-28.82,47.49,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
971,,mad712,cedrinho,erisma lanceolatum,erisma,vochysiaceae,0,flona_jacunda,rondonia,2022,-8.632,-62.836,7.55,0.31,-27.24,47.31,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
972,,mad712,cedrinho,erisma lanceolatum,erisma,vochysiaceae,25,flona_jacunda,rondonia,2022,-8.632,-62.836,7.54,0.22,-26.82,47.78,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
973,,mad712,cedrinho,erisma lanceolatum,erisma,vochysiaceae,50,flona_jacunda,rondonia,2022,-8.632,-62.836,7.09,0.22,-27.48,47.1,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
974,,mad712,cedrinho,erisma lanceolatum,erisma,vochysiaceae,75,flona_jacunda,rondonia,2022,-8.632,-62.836,7.02,0.25,-28.54,47.2,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
975,,mad712,cedrinho,erisma lanceolatum,erisma,vochysiaceae,100,flona_jacunda,rondonia,2022,-8.632,-62.836,7.09,0.3,-28.85,47.17,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
976,,mad713,cedrinho,erisma lanceolatum,erisma,vochysiaceae,0,flona_jacunda,rondonia,2022,-8.633,-62.844,7.57,0.28,-26.34,47.49,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
977,,mad713,cedrinho,erisma lanceolatum,erisma,vochysiaceae,25,flona_jacunda,rondonia,2022,-8.633,-62.844,11.56,0.25,-26.5,47.15,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
978,,mad713,cedrinho,erisma lanceolatum,erisma,vochysiaceae,50,flona_jacunda,rondonia,2022,-8.633,-62.844,8.77,0.26,-27.03,47.8,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
979,,mad713,cedrinho,erisma lanceolatum,erisma,vochysiaceae,75,flona_jacunda,rondonia,2022,-8.633,-62.844,8.23,0.26,-27.24,46.68,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
980,,mad713,cedrinho,erisma lanceolatum,erisma,vochysiaceae,100,flona_jacunda,rondonia,2022,-8.633,-62.844,9.89,0.5,-28.37,46.21,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
981,,mad714,cupiuba,goupeia glabra,goupeia,euphorbiaceae,0,flona_jacunda,rondonia,2022,-8.629,-62.853,8.41,0.09,-26.13,48.1,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
982,,mad714,cupiuba,goupeia glabra,goupeia,euphorbiaceae,25,flona_jacunda,rondonia,2022,-8.629,-62.853,8.99,0.09,-26.41,47.21,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
983,,mad714,cupiuba,goupeia glabra,goupeia,euphorbiaceae,50,flona_jacunda,rondonia,2022,-8.629,-62.853,7.8,0.09,-26.14,50.24,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
984,,mad714,cupiuba,goupeia glabra,goupeia,euphorbiaceae,75,flona_jacunda,rondonia,2022,-8.629,-62.853,8.19,0.11,-27.48,48.86,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
985,,mad714,cupiuba,goupeia glabra,goupeia,euphorbiaceae,100,flona_jacunda,rondonia,2022,-8.629,-62.853,8.91,0.15,-27.18,47.75,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
986,,mad715,cupiuba,goupeia glabra,goupeia,euphorbiaceae,0,flona_jacunda,rondonia,2022,-8.626,-62.852,8.85,0.09,-27.44,47.99,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
987,,mad715,cupiuba,goupeia glabra,goupeia,euphorbiaceae,25,flona_jacunda,rondonia,2022,-8.626,-62.852,7.51,0.09,-26.2,47.44,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
988,,mad715,cupiuba,goupeia glabra,goupeia,euphorbiaceae,50,flona_jacunda,rondonia,2022,-8.626,-62.852,8.06,0.1,-26.6,46.18,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
989,,mad715,cupiuba,goupeia glabra,goupeia,euphorbiaceae,75,flona_jacunda,rondonia,2022,-8.626,-62.852,8.65,0.11,-26.76,47.01,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
990,,mad715,cupiuba,goupeia glabra,goupeia,euphorbiaceae,100,flona_jacunda,rondonia,2022,-8.626,-62.852,6.99,0.15,-28.34,47.15,,,,,,,0.61833,0.81981,91.075,157,994.23041,25.50833,2311,-4.42063,-4.41403,-4.84743,-4.84901
991,,mad716,cupiuba,goupeia glabra,goupeia,euphorbiaceae,0,flona_jacunda,rondonia,2022,-8.863,-62.852,8.65,0.08,-28.16,46.09,,,,,,,0.6175,0.81979,91.35,103,1000.65411,25.4875,2359,-4.41166,-4.34829,-4.85079,-4.88444
992,,mad716,cupiuba,goupeia glabra,goupeia,euphorbiaceae,25,flona_jacunda,rondonia,2022,-8.863,-62.852,7.5,0.06,-26.27,47.36,,,,,,,0.6175,0.81979,91.35,103,1000.65411,25.4875,2359,-4.41166,-4.34829,-4.85079,-4.88444
993,,mad716,cupiuba,goupeia glabra,goupeia,euphorbiaceae,50,flona_jacunda,rondonia,2022,-8.863,-62.852,9.11,0.07,-25.81,48.19,,,,,,,0.6175,0.81979,91.35,103,1000.65411,25.4875,2359,-4.41166,-4.34829,-4.85079,-4.88444
994,,mad716,cupiuba,goupeia glabra,goupeia,euphorbiaceae,75,flona_jacunda,rondonia,2022,-8.863,-62.852,7.28,0.08,-26,48.25,,,,,,,0.6175,0.81979,91.35,103,1000.65411,25.4875,2359,-4.41166,-4.34829,-4.85079,-4.88444
995,,mad716,cupiuba,goupeia glabra,goupeia,euphorbiaceae,100,flona_jacunda,rondonia,2022,-8.863,-62.852,7.53,0.1,-26.76,47.5,,,,,,,0.6175,0.81979,91.35,103,1000.65411,25.4875,2359,-4.41166,-4.34829,-4.85079,-4.88444
996,,mad717,cupiuba,goupeia glabra,goupeia,euphorbiaceae,0,flona_jacunda,rondonia,2022,-8.625,-62.844,8.6,0.1,-27.29,47.8,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
997,,mad717,cupiuba,goupeia glabra,goupeia,euphorbiaceae,25,flona_jacunda,rondonia,2022,-8.625,-62.844,9.88,0.07,-25.59,48.62,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
998,,mad717,cupiuba,goupeia glabra,goupeia,euphorbiaceae,50,flona_jacunda,rondonia,2022,-8.625,-62.844,7.8,0.09,-26.08,47.78,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
999,,mad717,cupiuba,goupeia glabra,goupeia,euphorbiaceae,75,flona_jacunda,rondonia,2022,-8.625,-62.844,11.19,0.09,-27.01,46.91,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1000,,mad717,cupiuba,goupeia glabra,goupeia,euphorbiaceae,100,flona_jacunda,rondonia,2022,-8.625,-62.844,7.53,0.2,-29.27,46.67,,,,,,,0.61833,0.81981,91.075,137,996.60565,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1001,,mad718,cupiuba,goupeia glabra,goupeia,euphorbiaceae,0,flona_jacunda,rondonia,2022,-8.624,-62.842,9.54,0.08,-28.31,47.13,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1002,,mad718,cupiuba,goupeia glabra,goupeia,euphorbiaceae,25,flona_jacunda,rondonia,2022,-8.624,-62.842,8.1,0.07,-28.05,48.02,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1003,,mad718,cupiuba,goupeia glabra,goupeia,euphorbiaceae,50,flona_jacunda,rondonia,2022,-8.624,-62.842,7.42,0.09,-27.24,46.97,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1004,,mad718,cupiuba,goupeia glabra,goupeia,euphorbiaceae,75,flona_jacunda,rondonia,2022,-8.624,-62.842,7.92,0.1,-27.92,46.98,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1005,,mad718,cupiuba,goupeia glabra,goupeia,euphorbiaceae,100,flona_jacunda,rondonia,2022,-8.624,-62.842,6,0.12,-28.44,46.19,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1006,,mad719,cupiuba,goupeia glabra,goupeia,euphorbiaceae,0,flona_jacunda,rondonia,2022,-8.623,-62.839,6.72,0.08,-25.99,48.19,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1007,,mad719,cupiuba,goupeia glabra,goupeia,euphorbiaceae,25,flona_jacunda,rondonia,2022,-8.623,-62.839,8.16,0.07,-25.73,48.02,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1008,,mad719,cupiuba,goupeia glabra,goupeia,euphorbiaceae,50,flona_jacunda,rondonia,2022,-8.623,-62.839,9.12,0.08,-26.04,48.05,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1009,,mad719,cupiuba,goupeia glabra,goupeia,euphorbiaceae,75,flona_jacunda,rondonia,2022,-8.623,-62.839,7.62,0.07,-27.36,48.81,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1010,,mad719,cupiuba,goupeia glabra,goupeia,euphorbiaceae,100,flona_jacunda,rondonia,2022,-8.623,-62.839,7.62,0.11,-28.23,47.64,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1011,,mad720,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,flona_jacunda,rondonia,2022,-8.625,-62.836,5.12,0.13,-26.85,46.2,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1012,,mad720,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,flona_jacunda,rondonia,2022,-8.625,-62.836,5.43,0.12,-26.64,46.72,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1013,,mad720,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,flona_jacunda,rondonia,2022,-8.625,-62.836,4.69,0.12,-27.85,46.79,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1014,,mad720,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,flona_jacunda,rondonia,2022,-8.625,-62.836,4.99,0.12,-26.88,46.9,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1015,,mad720,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,flona_jacunda,rondonia,2022,-8.625,-62.836,3.68,0.13,-28.19,46.55,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1016,,mad721,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,flona_jacunda,rondonia,2022,-8.625,-62.84,5.98,0.16,-26.1,46.33,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1017,,mad721,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,flona_jacunda,rondonia,2022,-8.625,-62.84,7.86,0.11,-26.5,45.86,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1018,,mad721,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,flona_jacunda,rondonia,2022,-8.625,-62.84,7.63,0.11,-26.96,46.24,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1019,,mad721,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,flona_jacunda,rondonia,2022,-8.625,-62.84,6.53,0.11,-27.21,46.66,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1020,,mad721,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,flona_jacunda,rondonia,2022,-8.625,-62.84,5.86,0.24,-28.85,47.24,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1021,,mad722,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,flona_jacunda,rondonia,2022,-8.636,-62.839,7.43,0.14,-25.74,46.9,,,,,,,0.61833,0.81981,91.075,109,999.93872,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1022,,mad722,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,flona_jacunda,rondonia,2022,-8.636,-62.839,4.49,0.11,-26.02,47.16,,,,,,,0.61833,0.81981,91.075,109,999.93872,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1023,,mad722,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,flona_jacunda,rondonia,2022,-8.636,-62.839,4.43,0.12,-26.17,47.36,,,,,,,0.61833,0.81981,91.075,109,999.93872,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1024,,mad722,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,flona_jacunda,rondonia,2022,-8.636,-62.839,5.27,0.13,-27.13,46.11,,,,,,,0.61833,0.81981,91.075,109,999.93872,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1025,,mad722,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,flona_jacunda,rondonia,2022,-8.636,-62.839,5.23,0.18,-27.75,47.23,,,,,,,0.61833,0.81981,91.075,109,999.93872,25.50833,2311,-4.39013,-4.41403,-4.81583,-4.84901
1026,,mad723,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,flona_jacunda,rondonia,2022,-8.622,-62.836,6.74,0.13,-27.8,47.08,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1027,,mad723,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,flona_jacunda,rondonia,2022,-8.622,-62.836,6.5,0.11,-27.16,46.56,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1028,,mad723,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,flona_jacunda,rondonia,2022,-8.622,-62.836,7.92,0.1,-27.38,46.24,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1029,,mad723,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,flona_jacunda,rondonia,2022,-8.622,-62.836,6.14,0.1,-27.84,47.16,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1030,,mad723,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,flona_jacunda,rondonia,2022,-8.622,-62.836,5.6,0.16,-28.09,46.42,,,,,,,0.61917,0.81976,91.08334,104,1000.53485,25.5375,2300,-4.39013,-4.41403,-4.81583,-4.84901
1031,,mad724,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,flona_jacunda,rondonia,2022,-8.617,-62.843,5.18,0.11,-26.74,46.56,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1032,,mad724,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,flona_jacunda,rondonia,2022,-8.617,-62.843,5.67,0.11,-26.69,46.27,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1033,,mad724,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,flona_jacunda,rondonia,2022,-8.617,-62.843,4.58,0.1,-27.28,47.21,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1034,,mad724,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,flona_jacunda,rondonia,2022,-8.617,-62.843,4.08,0.1,-27.66,46.93,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1035,,mad724,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,flona_jacunda,rondonia,2022,-8.617,-62.843,3.34,0.13,-29.06,46.68,,,,,,,0.61917,0.81976,91.08334,124,998.15204,25.5375,2300,-4.39013,-4.41403,-4.84743,-4.84901
1036,,mad725,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,0,flona_jacunda,rondonia,2022,-8.626,-62.84,5.69,0.18,-28.88,46.35,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1037,,mad725,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,25,flona_jacunda,rondonia,2022,-8.626,-62.84,7.54,0.09,-28.27,46.07,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1038,,mad725,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,50,flona_jacunda,rondonia,2022,-8.626,-62.84,7.43,0.1,-27.08,46.74,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1039,,mad725,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,75,flona_jacunda,rondonia,2022,-8.626,-62.84,6.52,0.11,-27.41,46.85,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1040,,mad725,macaranduba,Manilkara huberi (Ducke) A.Chev.,manilkara,sapotaceae,100,flona_jacunda,rondonia,2022,-8.626,-62.84,6.51,0.13,-28.4,46.56,,,,,,,0.61833,0.81981,91.075,123,998.27106,25.50833,2311,-4.39013,-4.41403,-4.84743,-4.84901
1041,ZAC 405,madsd0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,são gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,,24.91,,24.91,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1042,ZAC 406,madsd0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,são gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,,24.51,,24.51,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1043,ZAC 407,madsd0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,são gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,,23.98,,23.98,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1044,ZAC 408,madsd0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,são gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,,24.89,,24.89,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1045,ZAC 409,madsd0,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,são gabriel da cachoeira,amazonas,Sep-13,-0.121,-67.013,,,,,,,,24.99,,24.99,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1046,ZAC 420,madsd01,breu,Protium hebetatum Daly,protium,burseraceae,0,são gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,,26.35,,26.35,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
1047,ZAC 421,madsd01,breu,Protium hebetatum Daly,protium,burseraceae,25,são gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,,26.62,,26.62,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
1048,ZAC 422,madsd01,breu,Protium hebetatum Daly,protium,burseraceae,50,são gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,,26.17,,26.17,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
1049,ZAC 423,madsd01,breu,Protium hebetatum Daly,protium,burseraceae,75,são gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,,26.16,,26.16,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
1050,ZAC 424,madsd01,breu,Protium hebetatum Daly,protium,burseraceae,100,são gabriel da cachoeira,amazonas,Sep-13,-0.041,-66.872,,,,,,,,26.48,,26.48,0.62083,0.82559,92.275,93,1001.84741,26.375,2764,-3.92301,-4.02441,-3.92461,-4.67525
1051,ZAC 435,madsd02,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,25.346575,24.51,0.03300544551,24.51,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1052,ZAC 436,madsd02,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,24.623225,24.27,0.01434519646,24.27,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1053,ZAC 437,madsd02,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,24.10515,24.27,-0.006838787562,24.27,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1054,ZAC 438,madsd02,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,23.293825,23.57,-0.01185614643,23.57,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1055,ZAC 439,madsd02,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,jurua,amazonas,Jul-13,-3.907,-66.055,,,,,,,22.7562,23.19,-0.01906293669,23.19,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1056,ZAC 445,madsd03,breu,Protium hebetatum Daly,protium,burseraceae,0,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,24.76985,24.51,0.01049057624,24.51,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1057,ZAC 446,madsd03,breu,Protium hebetatum Daly,protium,burseraceae,25,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,26.9399,24.73,0.0820307425,24.73,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1058,ZAC 447,madsd03,breu,Protium hebetatum Daly,protium,burseraceae,50,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,26.21655,25.49,0.02771341004,25.49,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1059,ZAC 448,madsd03,breu,Protium hebetatum Daly,protium,burseraceae,75,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,23.411125,24.95,-0.0657326378,24.95,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1060,ZAC 449,madsd03,breu,Protium hebetatum Daly,protium,burseraceae,100,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,24.681875,24.56,0.004937833937,24.56,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1061,ZAC 450,madsd04,breu,Protium hebetatum Daly,protium,burseraceae,0,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,25.9624,25.14,0.03167657844,25.14,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1062,ZAC 451,madsd04,breu,Protium hebetatum Daly,protium,burseraceae,25,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,25.74735,25.18,0.02203527742,25.18,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1063,ZAC 452,madsd04,breu,Protium hebetatum Daly,protium,burseraceae,50,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,23.430675,24.91,-0.06313625194,24.91,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1064,ZAC 453,madsd04,breu,Protium hebetatum Daly,protium,burseraceae,75,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,27.3309,25.37,0.0717466311,25.37,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1065,ZAC 454,madsd04,breu,Protium hebetatum Daly,protium,burseraceae,100,atalia do norte,amazonas,Aug-13,-4.304,-70.291,,,,,,,25.8842,24.8,0.04188655628,24.8,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.4728
1066,ZAC 520,madsd05,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,Jurua,amazonas,Jul-13,-3.907183314,-66.05514576,,,,,,,,27.01,,27.01,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1067,ZAC 521,madsd05,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,Jurua,amazonas,Jul-13,-3.907183314,-66.05514576,,,,,,,,28.2,,28.2,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1068,ZAC 522,madsd05,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,Jurua,amazonas,Jul-13,-3.907183314,-66.05514576,,,,,,,,27.02,,27.02,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1069,ZAC 523,madsd05,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,Jurua,amazonas,Jul-13,-3.907183314,-66.05514576,,,,,,,,26.26,,26.26,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1070,ZAC 524,madsd05,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,Jurua,amazonas,Jul-13,-3.907183314,-66.05514576,,,,,,,,24.24,,24.24,0.64,0.82437,89.98333,88,1002.44446,26.71667,2795,-4.16807,-3.90939,-4.56441,-4.86485
1071,ZAC 540,madsd06,breu,Protium hebetatum Daly,protium,burseraceae,0,são gabriel da cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,,,,,,,,26.16,,26.16,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1072,ZAC 541,madsd06,breu,Protium hebetatum Daly,protium,burseraceae,25,são gabriel da cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,,,,,,,,26,,26,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1073,ZAC 542,madsd06,breu,Protium hebetatum Daly,protium,burseraceae,50,são gabriel da cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,,,,,,,,26.51,,26.51,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1074,ZAC 543,madsd06,breu,Protium hebetatum Daly,protium,burseraceae,75,são gabriel da cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,,,,,,,,25.72,,25.72,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1075,ZAC 544,madsd06,breu,Protium hebetatum Daly,protium,burseraceae,100,são gabriel da cachoeira,amazonas,Sep-13,-0.121229892,-67.01310259,,,,,,,,24.02,,24.02,0.58917,0.83284,91.16666,101,1000.8927,26.20833,2830,-3.96045,-4.02056,-3.91908,-4.69293
1076,ZAC 545,madsd07,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,24.7,,24.7,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
1077,ZAC 546,madsd07,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,24.36,,24.36,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
1078,ZAC 547,madsd07,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,23.94,,23.94,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
1079,ZAC 548,madsd07,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,24.22,,24.22,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
1080,ZAC 549,madsd07,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,atalia do norte,amazonas,Aug-13,-4.303698078,-70.29106779,,,,,,,,22.92,,22.92,0.73583,0.79822,89.56667,108,1000.05792,26.64583,2708,-4.36128,-3.96684,-5.12316,-5.47280393
1081,ZAC 565,madsd08,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,23.83,,23.83,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1082,ZAC 566,madsd08,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,24.42,,24.42,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1083,ZAC 567,madsd08,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,24.21,,24.21,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1084,ZAC 568,madsd08,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,23.38,,23.38,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1085,ZAC 569,madsd08,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,23.88,,23.88,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1086,ZAC 595,madsd09,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,Jurua,amazonas,Jul-13,-3.758533554,-66.0737537,,,,,,,,24.52,,24.52,0.64667,0.82247,90.44167,84,1002.9223,26.71667,2856,-4.1379,-3.92563,-4.54196,-4.84537
1087,ZAC 596,madsd09,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,Jurua,amazonas,Jul-13,-3.758533554,-66.0737537,,,,,,,,22.56,,22.56,0.64667,0.82247,90.44167,84,1002.9223,26.71667,2856,-4.1379,-3.92563,-4.54196,-4.84537
1088,ZAC 597,madsd09,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,Jurua,amazonas,Jul-13,-3.758533554,-66.0737537,,,,,,,,24,,24,0.64667,0.82247,90.44167,84,1002.9223,26.71667,2856,-4.1379,-3.92563,-4.54196,-4.84537
1089,ZAC 598,madsd09,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,Jurua,amazonas,Jul-13,-3.758533554,-66.0737537,,,,,,,,23.91,,23.91,0.64667,0.82247,90.44167,84,1002.9223,26.71667,2856,-4.1379,-3.92563,-4.54196,-4.84537
1090,ZAC 599,madsd09,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,Jurua,amazonas,Jul-13,-3.758533554,-66.0737537,,,,,,,,23.77,,23.77,0.64667,0.82247,90.44167,84,1002.9223,26.71667,2856,-4.1379,-3.92563,-4.54196,-4.84537
1091,ZAC 620,madsd10,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,0,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,27.35,,27.35,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1092,ZAC 621,madsd10,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,25,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,25.06,,25.06,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1093,ZAC 622,madsd10,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,50,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,26.33,,26.33,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1094,ZAC 623,madsd10,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,75,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,25.03,,25.03,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
1095,ZAC 624,madsd10,ucuuba_puna,Iryanthera  laevis Markgr,iryanthera,myristicaceae,100,manicoré,amazonas,Mar-14,-6.009706576,-61.8686565,,,,,,,,26.32,,26.32,0.77083,0.79509,93.975,71,1004.47662,27.2,1996,-4.05694,-3.91077,-4.45402,-4.46622
"""

In [6]:
from pandas.util.testing import assert_frame_equal

# Average and variance test
def average_variance_test():
  test_df = pd.DataFrame({
      "lat": [0.0, 0.0, 1.0],
      "long": [3.0, 3.0, 6.0],
      "x": [3.0, 3.0, 3.0],
      "y": [8.0, 0.0, -3.0]}
  )
  feature_columns = ["lat", "long"]
  label_columns = ["x", "y"]
  aggregate_columns = ["lat", "long"]
  keep_grouping=True

  test_sample = dataset.preprocess_sample_data(test_df, feature_columns, label_columns, aggregate_columns, keep_grouping)
  expected_df = pd.DataFrame({
      "lat": [0.0, 1.0],
      "long": [3.0, 6.0],
      "x_mean": [3.0, 3.0],
      "x_variance": [0.0, np.nan],
      "y_mean": [4.0, -3.0],
      "y_variance": [32.0, np.nan]
  })
  assert_frame_equal(expected_df, test_sample)

def average_variance_test_no_grouping():
  test_df = pd.DataFrame({
      "lat": [0.0, 0.0, 1.0],
      "long": [3.0, 3.0, 6.0],
      "x": [3.0, 3.0, 3.0],
      "y": [8.0, 0.0, -3.0]}
  )
  feature_columns = ["lat", "long"]
  label_columns = ["x", "y"]
  aggregate_columns = ["lat", "long"]
  keep_grouping=False

  test_sample = dataset.preprocess_sample_data(test_df, feature_columns, label_columns, aggregate_columns, keep_grouping)
  expected_df = pd.DataFrame({
      "lat": [0.0, 0.0, 1.0],
      "long": [3.0, 3.0, 6.0],
      "x_mean": [3.0, 3.0, 3.0],
      "x_variance": [0.0, 0.0, np.nan],
      "y_mean": [4.0, 4.0, -3.0],
      "y_variance": [32.0, 32.0, np.nan]
  })
  assert_frame_equal(expected_df, test_sample)

def average_variance_test_keep_nonnumerical_columns():
  test_df = pd.DataFrame({
      "code": ["a", "b", "c"],
      "lat": [0.0, 0.0, 1.0],
      "long": [3.0, 3.0, 6.0],
      "x": [3.0, 3.0, 3.0],
      "y": [8.0, 0.0, -3.0]}
  )
  feature_columns = ["code", "lat", "long"]
  label_columns = ["x", "y"]
  aggregate_columns = ["lat", "long"]
  keep_grouping=True

  test_sample = dataset.preprocess_sample_data(test_df, feature_columns, label_columns, aggregate_columns, keep_grouping)
  expected_df = pd.DataFrame({
      "lat": [0.0, 1.0],
      "long": [3.0, 6.0],
      "code": ["a", "c"],
      "x_mean": [3.0, 3.0],
      "x_variance": [0.0, np.nan],
      "y_mean": [4.0, -3.0],
      "y_variance": [32.0, np.nan]
  })
  assert_frame_equal(expected_df, test_sample)

def xgb_equivalence():
  with open(raster.get_sample_db_path("scratch_file.txt"), "w") as f:
    f.write(jun23_reference_data)

  pds1 = dataset.partitioned_reference_data("scratch_file.txt")

  # The equivalent...
  df = pd.read_csv(raster.get_sample_db_path("2023_06_23_Results_Google.csv"),
    encoding="ISO-8859-1", sep=',')
  df = dataset.partition(df.rename(
      columns={'long': 'lon' })).train
  df = df[['Code', 'lat', 'lon', 'd18O_cel']]
  df = df[df['d18O_cel'].notna()]
  df = df.groupby(['lat', 'lon'])
  df = df.mean().reset_index().rename(
      columns={'d18O_cel': 'cellulose_oxygen_ratio' }).reset_index()
  df.drop('index', inplace=True, axis=1)

  fdf = dataset.add_features_from_rasters(df, [raster.relative_humidity_geotiff(),
      raster.temperature_geotiff(),
      raster.vapor_pressure_deficit_geotiff(),
      raster.atmosphere_isoscape_geotiff()])

  fdf1 = fdf.sort_index(axis=1).reset_index(drop=True)
  fdf2 = pds1.train.sort_index(axis=1).reset_index(drop=True)

  assert_frame_equal(fdf1, fdf2)

average_variance_test()
average_variance_test_no_grouping()
average_variance_test_keep_nonnumerical_columns()
xgb_equivalence()

/content/gdrive/MyDrive/xgb2/ddf_common/dataset.py:183: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  means = grouped.mean().reset_index()
/content/gdrive/MyDrive/xgb2/ddf_common/dataset.py:187: FutureWarning: The default value of numeric_only in DataFrameGroupBy.var is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  variances = grouped.var().reset_index()
/content/gdrive/MyDrive/xgb2/ddf_common/dataset.py:183: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  means = grouped.mean().reset_index()
/c

Driver: GTiff/GeoTIFF
Size is 941 x 937 x 12
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0000000000241, 5.29166666665704)
Pixel Size = (0.04166666666665718, -0.04166666666667143)
Driver: GTiff/GeoTIFF
Size is 941 x 937 x 12
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Origin = (-74.0000000000241, 5.29166666665704)
Pixel Size = (0.04166666666665718, -0.04166666666667143)
Driver: GTiff/GeoTIFF
Size is 941 x 937 x 12
Projection is GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,

100%|██████████| 46/46 [00:00<00:00, 6242.13it/s]


Train: 91.30% (42)
Test: 0.00% (0)
Validation: 8.70% (4)


<ipython-input-6-2b29dcb30f0e>:89: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.mean().reset_index().rename(
